# Match Ids and import

In [14]:
from statsbombpy import sb
from mplsoccer import Pitch, VerticalPitch
from IPython.display import display
import seaborn as sns
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats
import os
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import matplotlib as mpl
from matplotlib.colors import to_rgba
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import os
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.io as pio
from plotly.subplots import make_subplots
import json
import re
import pycountry
import matplotlib.colors as mcolors 
import pandas as pd
import numpy as np
from scipy import stats
import json
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
import pycountry
from datetime import datetime

In [15]:
# Fetch the list of competitions
competitions = sb.competitions()

# Filter for World Cup competitions and make a copy to safely modify
world_cup = competitions[competitions['competition_name'].str.contains("World Cup", case=False, na=False)].copy()

# Now it's safe to assign new columns
world_cup['year'] = world_cup['season_name'].str.extract(r'(\d{4})')

# Display the result
print(world_cup[['competition_name', 'competition_id', 'season_id', 'year']])

      competition_name  competition_id  season_id  year
28  FIFA U20 World Cup            1470        274  1979
29      FIFA World Cup              43        106  2022
30      FIFA World Cup              43          3  2018
31      FIFA World Cup              43         55  1990
32      FIFA World Cup              43         54  1986
33      FIFA World Cup              43         51  1974
34      FIFA World Cup              43        272  1970
35      FIFA World Cup              43        270  1962
36      FIFA World Cup              43        269  1958
72   Women's World Cup              72        107  2023
73   Women's World Cup              72         30  2019


In [16]:
class TeamColorManager:
    """Manages color assignment based on national team shirt colors."""
    
    def __init__(self):
        self.national_colors = {
            'Argentina': '#87CEEB',  # Light blue and white stripes 
            'France': '#0055A4',     # Navy blue
            'Brazil': '#FFD700',     # Yellow
            'Netherlands': '#FF4500', # Orange
            'England': '#D3D3D3',    # Light gray 
            'Croatia': '#FF0000',    # Red and white checkered 
            'Morocco': '#C1272D',    # Red
            'Portugal': '#CE1126',   # Red
            'Spain': '#AA151B',      # Red
            'Germany': '#D3D3D3',    # Light gray 
            'Belgium': '#000000',    # Black
            'Poland': '#D3D3D3',     # Light gray 
            'Switzerland': '#FF0000', # Red
            'Uruguay': '#55AAFF',    # Light blue
            'Denmark': '#C8102E',    # Red
            'Senegal': '#00A651',    # Green
            'Australia': '#FDD835',  # Yellow 
            'Japan': '#002868',      # Navy blue
            'South Korea': '#C9151E', # Red
            'Canada': '#FF0000',     # Red
            'Mexico': '#006633',     # Green
            'Saudi Arabia': '#006C35', # Green
            'Tunisia': '#CE1126',    # Red
            'Ecuador': '#FFD100',    # Yellow
            'Iran': '#239F40',       # Green
            'Qatar': '#8A1538',      # Maroon
            'Serbia': '#C6363C',     # Red
            'Cameroon': '#007A3D',   # Green
            'Wales': '#C8102E',      # Red
            'Ghana': '#CE1126',      # Red
            'Costa Rica': '#002147', # Navy blue
            'USA': '#D3D3D3',        # Light gray 
            'United States': '#D3D3D3', # Light gray 
            'Italy': '#0066CC',      # Blue
            'Russia': '#C8102E',     # Red
            'Colombia': '#FFCD00',   # Yellow
            'Nigeria': '#008751',    # Green
            'Algeria': '#007A3D',    # Green
            'Egypt': '#CE1126',      # Red
            'Turkey': '#E30A17',     # Red
            'India': '#FF9933',      # Orange
            'Chile': '#C8102E',      # Red
            'Peru': '#D3D3D3',       # Light gray 
            'Venezuela': '#FFCE00',  # Yellow
            'Paraguay': '#CE1126',   # Red
            'Bolivia': '#007A3D',    # Green
            'China': '#DE2910',      # Red
            'Thailand': '#ED1C24',   # Red
            'Iraq': '#C8102E',       # Red
            'Vietnam': '#DA020E',    # Red
            'Indonesia': '#CE1126',  # Red
            'Norway': '#EF2B2D',     # Red
            'Sweden': '#006AA7',     # Blue
            'Finland': '#002F6C',    # Blue
            'Iceland': '#0048E0',    # Blue
            'Czech Republic': '#D7141A', # Red
            'Slovakia': '#0B4EA2',   # Blue
            'Hungary': '#436F4D',    # Green
            'Romania': '#003DA5',    # Blue
            'Bulgaria': '#00966E',   # Green
            'Ukraine': '#FFD500',    # Yellow
            'Greece': '#0D5EAF',     # Blue
            'Austria': '#C8102E',    # Red
            'Slovenia': '#007A3D',   # Green
            'North Macedonia': '#CE1126', # Red
            'Albania': '#E41E20',    # Red
            'Montenegro': '#C8102E', # Red
            'Luxembourg': '#00A1DE', # Light blue
            'Andorra': '#10069C',    # Blue
            'Malta': '#CF142B',      # Red
            'Gibraltar': '#C8102E',  # Red
            'Faroe Islands': '#ED2E38', # Red
            'San Marino': '#0099CC', # Light blue
            'Liechtenstein': '#002B7F', # Blue
        }
        
        self.fallback_colors = ['#808080', '#A0A0A0', '#909090', '#B0B0B0']
        self.fallback_index = 0
    
    def get_color_for_team(self, team_name):
        """Get the national color for a team."""
        if team_name in self.national_colors:
            return self.national_colors[team_name]
        else:
            # For unknown teams, use a fallback color and increment for next unknown team
            color = self.fallback_colors[self.fallback_index % len(self.fallback_colors)]
            self.fallback_index += 1
            print(f"Warning: No color defined for {team_name}, using fallback color {color}")
            return color
    
    def add_team_color(self, team_name, color):
        """Add or update a team's color."""
        self.national_colors[team_name] = color
    
    def get_all_defined_colors(self):
        """Get all defined team colors."""
        return self.national_colors.copy()


color_manager = TeamColorManager()

In [17]:
def get_matches_by_team(team_name, competition_id={"competition_id": 43, "season_id": 106}):
    """
    Get matches for a specific team in a competition and return as a dictionary
    with opponent details and national shirt colors.
    
    Args:
        team_name (str): Name of the team to get matches for
        competition_id (dict): Dictionary with competition_id and season_id
    
    Returns:
        dict: Dictionary with match_id as key and opponent info as value
    """
    # Define stage order for sorting
    stage_order = [
        "Group Stage",
        "Round of 16",
        "Quarter-finals",
        "Semi-finals",
        "Final"
    ]
    
    # Fetch matches for the competition
    matches = sb.matches(
        competition_id=competition_id['competition_id'],
        season_id=competition_id['season_id']
    )
    
    # Map stages to order for sorting
    matches['stage_order'] = matches['competition_stage'].map(
        lambda x: stage_order.index(x) if x in stage_order else -1
    )
    
    # Filter matches for the specific team
    team_matches = matches[
        (matches['home_team'] == team_name) | (matches['away_team'] == team_name)
    ].sort_values(by='stage_order')
    
    # Create the matches dictionary
    matches_dict = {}
    
    for _, match in team_matches.iterrows():
        match_id = match['match_id']
        
        # Determine the opponent
        if match['home_team'] == team_name:
            opponent = match['away_team']
        else:
            opponent = match['home_team']
        
        # Get national color for the opponent
        color = color_manager.get_color_for_team(opponent)
        
        # Add to dictionary
        matches_dict[match_id] = {
            'team': opponent,
            'color': color
        }
    
    return matches_dict


arg_matches = get_matches_by_team("Argentina")
fr_matches = get_matches_by_team("France")



### TRB calculation

In [18]:
def analyze_time_to_recover(home_team, matches, output_dir="output"):
    """
    Analyze time to recover for matches between home team and opponents.
    Creates global variables for each match's time-to-recover dataframe.
    """    
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    for match_id, details in matches.items():
        print(f"\nProcessing match with Match ID: {match_id} ({details['team']})...")

        # ------------------------------------------------------------------------------
        # 1. Pull events for the match
        # ------------------------------------------------------------------------------
        # Assuming the events are fetched outside the function or with a global sb object
        try:
            events = sb.events(match_id=match_id)
        except NameError:
            print(f"Error: StatsBomb client 'sb' not defined. Please define it before calling this function.")
            continue

        # ------------------------------------------------------------------------------
        # 2. Parse and sort events
        # ------------------------------------------------------------------------------
        events['timestamp'] = pd.to_datetime(events['timestamp'], errors='coerce')
        events = events[events['period'].isin([1, 2, 3, 4])]
        
        if events.empty:
            print(f"No valid events found for Match {match_id}. Skipping...")
            continue

        events = events.sort_values(by=['period', 'timestamp']).reset_index(drop=True)

        # ------------------------------------------------------------------------------
        # 3. Identify Goal and Own Goal Events
        # ------------------------------------------------------------------------------
        goal_events = events[
            ((events['type'] == 'Shot') & (events['shot_outcome'] == 'Goal')) |
            (events['type'] == 'Own Goal For')
        ].copy()

        # ------------------------------------------------------------------------------
        # 4. Define Conditions for Ball-Loss and Ball-Recovery Events
        # ------------------------------------------------------------------------------
        ball_loss_conditions = [
            (events['type'] == 'Pass') & (events['pass_outcome'] == 'Incomplete'),
            (events['type'] == 'Pass') & (events['pass_outcome'] == 'Intercepted'),
            (events['type'] == 'Pass') & (events['pass_type'].isin(['Long Ball', 'Through Ball'])) & (events['pass_outcome'] == 'Incomplete'),
            (events['type'] == 'Carry') & (events['under_pressure'] == True),
            (events['type'] == 'Duel') & (events['duel_outcome'] == 'Lost'),
            (events['type'] == 'Dribble') & (events['dribble_outcome'] == 'Unsuccessful'),
            (events['type'] == 'Goalkeeper Distribution') & (events['goalkeeper_outcome'] == 'Unsuccessful'),
            events['type'].isin(['Dispossessed', 'Turnover', 'Interception', 'Error']),
            events['type'] == 'Foul Committed'
        ]
        combined_ball_loss_cond = np.logical_or.reduce(ball_loss_conditions)

        ball_recovery_conditions = [
            (events['type'] == 'Duel') & (events['duel_outcome'] == 'Won'),
            (events['type'] == 'Pass') & (events['pass_outcome'] == 'Successful'),
            (events['type'] == 'Interception') & (events['interception_outcome'] == 'Successful'),
            (events['type'] == 'Goalkeeper Action') & (events['goalkeeper_outcome'].isin(['Saved', 'Caught'])),
            events['type'].isin(['Ball Recovery', 'Interception', 'Tackle', 'Goalkeeper Possession']),
            events['type'] == 'Foul Won'
        ]
        combined_recovery_cond = np.logical_or.reduce(ball_recovery_conditions)

        # ------------------------------------------------------------------------------
        # 5. Compute Time-to-Recover in Segments Between Goals
        # ------------------------------------------------------------------------------
        period_results_list = []

        for period_val, period_df in events.groupby('period'):
            if period_val not in [1, 2, 3, 4]:
                continue

            period_df = period_df.sort_values(by='timestamp').reset_index(drop=True)

            # Get goal timestamps in this period
            goal_timestamps = goal_events.loc[goal_events['period'] == period_val, 'timestamp'].tolist()
            goal_timestamps.append(pd.Timestamp.max)  # Ensure last segment is covered

            # Split period into goal segments
            start_time = period_df['timestamp'].min()
            for goal_time in goal_timestamps:
                segment_df = period_df[(period_df['timestamp'] >= start_time) & (period_df['timestamp'] < goal_time)]

                # Identify ball-loss and ball-recovery events in this segment
                segment_loss_events = segment_df[
                    (segment_df['type'] == 'Pass') & (segment_df['pass_outcome'] == 'Incomplete') |
                    (segment_df['type'] == 'Pass') & (segment_df['pass_outcome'] == 'Intercepted') |
                    (segment_df['type'] == 'Pass') & (segment_df['pass_type'].isin(['Long Ball', 'Through Ball'])) & (segment_df['pass_outcome'] == 'Incomplete') |
                    (segment_df['type'] == 'Carry') & (segment_df['under_pressure'] == True) |
                    (segment_df['type'] == 'Duel') & (segment_df['duel_outcome'] == 'Lost') |
                    (segment_df['type'] == 'Dribble') & (segment_df['dribble_outcome'] == 'Unsuccessful') |
                    (segment_df['type'] == 'Goalkeeper Distribution') & (segment_df['goalkeeper_outcome'] == 'Unsuccessful') |
                    segment_df['type'].isin(['Dispossessed', 'Turnover', 'Interception', 'Error']) |
                    (segment_df['type'] == 'Foul Committed')
                ].copy()

                segment_recovery_events = segment_df[
                    (segment_df['type'] == 'Duel') & (segment_df['duel_outcome'] == 'Won') |
                    (segment_df['type'] == 'Pass') & (segment_df['pass_outcome'] == 'Successful') |
                    (segment_df['type'] == 'Interception') & (segment_df['interception_outcome'] == 'Successful') |
                    (segment_df['type'] == 'Goalkeeper Action') & (segment_df['goalkeeper_outcome'].isin(['Saved', 'Caught'])) |
                    segment_df['type'].isin(['Ball Recovery', 'Interception', 'Tackle', 'Goalkeeper Possession']) |
                    (segment_df['type'] == 'Foul Won')
                ].copy()

                if segment_loss_events.empty or segment_recovery_events.empty:
                    start_time = goal_time  # Move to the next segment
                    continue

                # Store results
                time_to_recover = []

                latest_recovery_time = None  # Track the last recovery time

                for _, loss_event in segment_loss_events.iterrows():
                    team = loss_event['team']
                    loss_time = loss_event['timestamp']

                    # Skip new losses if waiting for recovery
                    if latest_recovery_time and loss_time < latest_recovery_time:
                        continue  # Ignore this loss because a recovery hasn't happened yet

                    # Find the next valid recovery
                    valid_recoveries = segment_recovery_events[
                        (segment_recovery_events['team'] == team) &
                        (segment_recovery_events['timestamp'] > loss_time)
                    ]

                    if not valid_recoveries.empty:
                        recovery_event = valid_recoveries.iloc[0]
                        recovery_time = recovery_event['timestamp']
                        time_diff = (recovery_time - loss_time).total_seconds()

                        if time_diff >= 1:
                            time_to_recover.append({
                                'team': team,
                                'period': period_val,
                                'loss_time': loss_time,
                                'recovery_time': recovery_time,
                                'time_to_recover': time_diff
                            })
                            latest_recovery_time = recovery_time  # Update latest recovery time

                period_results = pd.DataFrame(time_to_recover)
                if not period_results.empty:
                    period_results_list.append(period_results)

                start_time = goal_time  # Move to the next segment after a goal

        # ------------------------------------------------------------------------------
        # 6. Create a Global Variable for Each Match
        # ------------------------------------------------------------------------------
        if period_results_list:
            # Get both team names and format them for the variable name and filename
            team1 = details['team'].replace(' ', '_').lower()
            team2 = home_team.replace(' ', '_').lower()
            
            # Concatenate all period results
            final_df = pd.concat(period_results_list, ignore_index=True)
            
            # Create variable name with both teams
            df_var_name = f"df_time_to_recover_{team2}_vs_{team1}"
            
            # Create a global variable with the appropriate name
            globals()[df_var_name] = final_df
            
            # Save to CSV with both team names
            csv_filename = f"{output_dir}/time_to_recover_{team2}_vs_{team1}.csv"
            final_df.to_csv(csv_filename, index=False)

            print(f"** Global variable '{df_var_name}' created for match {match_id}: {team2} vs {team1} **")
            print(f"Data also saved to {csv_filename}")
        else:
            print(f"No time-to-recover results for Match {match_id} in any period.")

analyze_time_to_recover('Argentina', arg_matches)
analyze_time_to_recover('France', fr_matches)


Processing match with Match ID: 3857264 (Poland)...
** Global variable 'df_time_to_recover_argentina_vs_poland' created for match 3857264: argentina vs poland **
Data also saved to output/time_to_recover_argentina_vs_poland.csv

Processing match with Match ID: 3857289 (Mexico)...
** Global variable 'df_time_to_recover_argentina_vs_mexico' created for match 3857289: argentina vs mexico **
Data also saved to output/time_to_recover_argentina_vs_mexico.csv

Processing match with Match ID: 3857300 (Saudi Arabia)...
** Global variable 'df_time_to_recover_argentina_vs_saudi_arabia' created for match 3857300: argentina vs saudi_arabia **
Data also saved to output/time_to_recover_argentina_vs_saudi_arabia.csv

Processing match with Match ID: 3869151 (Australia)...
** Global variable 'df_time_to_recover_argentina_vs_australia' created for match 3869151: argentina vs australia **
Data also saved to output/time_to_recover_argentina_vs_australia.csv

Processing match with Match ID: 3869321 (Nether

### Visuals for coaches

In [19]:
def add_explanation_buttons_to_html(html_file_path):
    """
    Adds explanation buttons and modals to plot HTML files.
    
    Args:
        html_file_path (str): Path to the HTML file to modify
    
    Returns:
        None: Modifies the file in place
    """
    import os
    
    # Read the HTML file
    with open(html_file_path, 'r', encoding='utf-8') as f:
        html_content = f.read()
    
    # Detect plot types in this file
    filename = os.path.basename(html_file_path).lower()
    has_qq = 'qq' in filename or 'QQ Plot' in html_content
    has_kde = 'kde' in filename or 'Density Plot' in html_content
    has_match_kde = ('match' in filename and 'kde' in filename) or ('Recovery Time Distributions' in html_content)
    has_match_events = 'match_events' in filename or 'Match Events' in html_content or 'Game Time (minutes)' in html_content
    
    # Create the HTML for the explanation button container
    explanation_button_html = """
    <div class="plot-explanation-container" style="position: absolute; top: 10px; right: 10px; z-index: 100;">
    """
    
    # Add buttons based on detected plot types
    if has_match_events:
        explanation_button_html += """
        <button id="explain-plot" class="explanation-button" data-plot-type="match-events">
            <span class="explanation-icon">❓</span> Explain This Plot
        </button>
        """
    elif has_qq and has_kde and not has_match_kde:
        # Combined QQ and KDE plot page
        explanation_button_html += """
        <button id="explain-plot" class="explanation-button" data-plot-type="combined">
            <span class="explanation-icon">❓</span> Explain This Plot
        </button>
        """
    elif has_match_kde:
        explanation_button_html += """
        <button id="explain-plot" class="explanation-button" data-plot-type="match-kde">
            <span class="explanation-icon">❓</span> Explain This Plot
        </button>
        """
    elif has_kde:
        explanation_button_html += """
        <button id="explain-plot" class="explanation-button" data-plot-type="kde">
            <span class="explanation-icon">❓</span> Explain This Plot
        </button>
        """
    elif has_qq:
        explanation_button_html += """
        <button id="explain-plot" class="explanation-button" data-plot-type="qq">
            <span class="explanation-icon">❓</span> Explain This Plot
        </button>
        """
    
    explanation_button_html += """
    </div>
    """
    
    # Create the HTML for the explanation modal
    explanation_modal_html = """
    <div id="explanation-modal" class="modal">
      <div class="modal-content">
        <div class="modal-header">
          <h2 id="modal-title">Plot Explanation</h2>
          <span class="close-modal">&times;</span>
        </div>
        <div class="modal-body">
          <div id="combined-explanation" class="explanation-content">
            <h3>Understanding QQ and KDE Plots</h3>
            
            <div class="explanation-section">
              <h4>Left Side: QQ Plot</h4>
              <p>This plot helps us understand if recovery times follow a "normal" pattern or if they show unusual trends:</p>
              <ul>
                <li><strong>Red line:</strong> This is the "expected" pattern if all players were recovering at similar rates</li>
                <li><strong>Blue dots:</strong> Each dot represents actual player recovery times</li>
                <li><strong>What to look for:</strong> 
                  <ul>
                    <li>Dots following the line closely = recovery times are consistent across the team</li>
                    <li>Dots curving away from the line = some players recover much differently than others</li>
                    <li>Dots above the line at right side = some players taking much longer to recover</li>
                  </ul>
                </li>
                <li><strong>P-value in title:</strong> Below 0.05 means the team has inconsistent recovery patterns that may need attention</li>
              </ul>
              <p><strong>Football insights:</strong></p>
              <ul>
                <li>Identify players who don't fit the team's recovery profile</li>
                <li>Spot if your team has distinct "fast recoverers" and "slow recoverers" that need different training approaches</li>
                <li>Track if tactical changes impact recovery consistency</li>
              </ul>
            </div>
            
            <div class="explanation-section">
              <h4>Right Side: KDE Plot</h4>
              <p>This plot shows the distribution of how quickly players recover after losing possession:</p>
              <ul>
                <li><strong>Blue curve:</strong> Shows the frequency of different recovery times - peaks indicate most common recovery times</li>
                <li><strong>Blue histogram:</strong> Actual recovery time frequencies</li>
                <li><strong>Blue circles:</strong> Individual player recovery instances</li>
                <li><strong>Vertical lines:</strong> Median (blue) and average (green) recovery times</li>
              </ul>
              <p><strong>Football insights:</strong></p>
              <ul>
                <li>Lower peaks (shorter recovery times) indicate more effective pressing/counterpress</li>
                <li>Wide spread indicates inconsistent recovery approach across the team</li>
                <li>Long "tail" to the right suggests specific situations where recovery breaks down</li>
                <li>Big difference between average and median suggests a few problematic recovery scenarios affecting overall performance</li>
                <li>Use to compare recovery effectiveness across different formations, opponents, or pitch zones</li>
              </ul>
            </div>
          </div>
          
          <div id="kde-explanation" class="explanation-content">
            <h3>Understanding Recovery Time Distribution</h3>
            <p>This density plot (KDE) shows how quickly your team recovers possession after losing the ball:</p>
            <ul>
              <li><strong>Blue curve:</strong> Shows the frequency of different recovery times - the peak shows your team's most common recovery time</li>
              <li><strong>Blue histogram:</strong> The actual count of recovery instances at each time point</li>
              <li><strong>Blue circles:</strong> Individual recovery instances from the match data</li>
              <li><strong>Vertical lines:</strong> Show the median (blue) and average (green) recovery times</li>
            </ul>
            <p><strong>What to look for:</strong></p>
            <ul>
              <li><strong>Peak position:</strong> Lower values (left-side peaks) mean faster recoveries - better pressing efficiency</li>
              <li><strong>Curve width:</strong> Narrow peaks mean consistent recovery approach; wide spread suggests variable recovery tactics</li>
              <li><strong>Right-side tail:</strong> Long tail means some situations lead to much longer recovery times</li>
              <li><strong>Multiple peaks:</strong> Could indicate different recovery strategies in different game situations</li>
              <li><strong>Average vs Median gap:</strong> Large gap suggests a few problematic recovery situations affecting overall performance</li>
            </ul>
            <p><strong>Football applications:</strong></p>
            <ul>
              <li>Compare recovery performance across different matches, periods, or opponents</li>
              <li>Identify if tactical changes are improving recovery times</li>
              <li>Spot if fatigue is affecting recovery (later periods showing longer times)</li>
              <li>Evaluate effectiveness of pressing strategies</li>
              <li>Identify if specific player rotations improve recovery performance</li>
            </ul>
          </div>
          
          <div id="qq-explanation" class="explanation-content">
            <h3>Understanding Recovery Time Patterns</h3>
            <p>This QQ plot helps coaches identify if recovery times follow expected patterns or show unusual trends:</p>
            <ul>
              <li><strong>Red line:</strong> Represents "normal" distribution - what we'd expect if recovery was consistent</li>
              <li><strong>Blue dots:</strong> Each dot represents actual recovery time instances</li>
              <li><strong>P-value:</strong> Statistical measure of how "normal" your recovery patterns are (below 0.05 means unusual patterns)</li>
            </ul>
            <p><strong>What to look for:</strong></p>
            <ul>
              <li><strong>Dots following the line:</strong> Team recovers consistently across all situations</li>
              <li><strong>Dots curving above line (right side):</strong> Some recovery instances taking much longer than expected</li>
              <li><strong>Dots curving below line (left side):</strong> Some unusually quick recoveries</li>
              <li><strong>S-shaped curve:</strong> Team has mixed recovery performance - very quick in some situations, very slow in others</li>
            </ul>
            <p><strong>Football applications:</strong></p>
            <ul>
              <li>Identify if you have distinct "recovery profiles" within your team that need different training</li>
              <li>Spot if your recovery approach changes dramatically in different match situations</li>
              <li>Evaluate if your team's recovery times are predictable (following the line) or highly variable</li>
              <li>Determine if recovery performance is being affected by specific opposition tactics</li>
              <li>Use to decide if you need one unified pressing strategy or multiple approaches for different scenarios</li>
            </ul>
          </div>
          
          <div id="match-kde-explanation" class="explanation-content">
            <h3>Understanding Match Recovery Comparison</h3>
            <p>This plot compares ball recovery times between teams and across different match periods:</p>
            <ul>
              <li><strong>Different colored lines:</strong> Each color represents a different team</li>
              <li><strong>Line styles:</strong> Solid lines show all periods combined; dashed/dotted lines show individual periods</li>
              <li><strong>Peaks:</strong> The most common recovery time for each team</li>
              <li><strong>Vertical lines:</strong> Show mean recovery times for each team</li>
            </ul>
            <p><strong>What to look for:</strong></p>
            <ul>
              <li><strong>Left-shifted peaks:</strong> Teams with peaks further left recover possession faster</li>
              <li><strong>Peak height:</strong> Taller, narrower peaks indicate more consistent recovery times</li>
              <li><strong>Period differences:</strong> Compare how recovery times change across periods to spot fatigue or tactical shifts</li>
              <li><strong>Curve overlap:</strong> Where curves overlap, teams have similar recovery performance</li>
              <li><strong>Multiple peaks:</strong> Could indicate different recovery scenarios (e.g., high press vs. low block)</li>
            </ul>
            <p><strong>Football applications:</strong></p>
            <ul>
              <li>Directly compare your pressing efficiency against opponents</li>
              <li>Identify which periods your team had recovery advantage/disadvantage</li>
              <li>Spot if opponent's tactics affected your recovery performance</li>
              <li>See if substitutions improved recovery times in later periods</li>
              <li>Evaluate if recovery performance correlates with match outcome</li>
              <li>Determine when to increase/decrease pressing intensity based on recovery trends</li>
            </ul>
          </div>
          
          <div id="match-events-explanation" class="explanation-content">
            <h3>Understanding Match Events Plot</h3>
            <p>This timeline visualization shows recovery times alongside key match events, helping coaches connect tactical moments with recovery performance:</p>
            <ul>
              <li><strong>Recovery time lines:</strong> The colored lines show recovery times of each team throughout the match</li>
              <li><strong>Y-axis:</strong> Shows time taken to recover possession (in seconds) - lower values indicate faster recovery</li>
              <li><strong>X-axis:</strong> Match timeline in minutes</li>
              <li><strong>Event icons:</strong> Shows goals ⚽, injuries 🤕, substitutions 🔄, tactical shifts 🔄, and cards 🟨/🟥</li>
              <li><strong>Hover tooltips:</strong> Reveal detailed information about specific recovery instances or match events</li>
            </ul>
            
            <p><strong>What to look for:</strong></p>
            <ul>
              <li><strong>Recovery time patterns:</strong> Identify periods of effective and poor recovery performance</li>
              <li><strong>Event impact:</strong> See how events like goals, substitutions or tactical shifts affect recovery times</li>
              <li><strong>Pre/post injury:</strong> Compare recovery performance before and after player injuries</li>
              <li><strong>Formation influence:</strong> Notice how formation changes impact recovery efficiency</li>
              <li><strong>Period-to-period changes:</strong> Track how fatigue affects recovery across match periods</li>
            </ul>
            
            <p><strong>Football applications:</strong></p>
            <ul>
              <li>Identify which in-game events create recovery vulnerabilities</li>
              <li>Assess if substitutions improve or disrupt recovery patterns</li>
              <li>Determine optimal times for tactical adjustments based on recovery trends</li>
              <li>Analyze which formations deliver most consistent recovery performance</li>
              <li>Spot connections between recovery breakdown and conceded goals</li>
              <li>Understand how specific player injuries impact team-wide recovery efficiency</li>
            </ul>
            
            <p>The detailed match events table below the plot provides additional context about each event, and clicking on formation details reveals full team positioning information.</p>
          </div>
        </div>
      </div>
    </div>
    """
    
    # Create the CSS styles with BLUE button
    explanation_css = """
    <style>
    /* Plot Explanation Button */
    .explanation-button {
      background-color: #1e88e5; /* Changed to blue to match your dashboard */
      color: white;
      border: none;
      border-radius: 4px;
      padding: 8px 16px;
      display: flex;
      align-items: center;
      gap: 8px;
      cursor: pointer;
      font-size: 14px;
      font-weight: 500;
      transition: all 0.2s;
      box-shadow: 0 2px 4px rgba(0,0,0,0.1);
    }
    
    .explanation-button:hover {
      background-color: #1565C0; /* Darker blue on hover */
      transform: translateY(-2px);
      box-shadow: 0 4px 8px rgba(0,0,0,0.15);
    }
    
    .explanation-icon {
      font-size: 16px;
    }
    
    /* Modal Styles */
    .modal {
      display: none;
      position: fixed;
      z-index: 1000;
      left: 0;
      top: 0;
      width: 100%;
      height: 100%;
      overflow: auto;
      background-color: rgba(0,0,0,0.6);
    }
    
    .modal-content {
      background-color: white;
      margin: 5% auto;
      width: 80%;
      max-width: 900px;
      border-radius: 8px;
      box-shadow: 0 5px 30px rgba(0,0,0,0.3);
      animation: modalFadeIn 0.3s;
      max-height: 90vh;
      display: flex;
      flex-direction: column;
    }
    
    @keyframes modalFadeIn {
      from {opacity: 0; transform: translateY(-20px);}
      to {opacity: 1; transform: translateY(0);}
    }
    
    .modal-header {
      display: flex;
      justify-content: space-between;
      align-items: center;
      padding: 15px 20px;
      border-bottom: 1px solid #eee;
    }
    
    .modal-header h2 {
      margin: 0;
      color: #1e88e5;
      font-size: 22px;
    }
    
    .close-modal {
      font-size: 28px;
      font-weight: bold;
      color: #666;
      cursor: pointer;
    }
    
    .close-modal:hover {
      color: #000;
    }
    
    .modal-body {
      padding: 20px;
      overflow-y: auto;
    }
    
    .explanation-content {
      display: none;
      line-height: 1.6;
    }
    
    .explanation-content.active {
      display: block;
    }
    
    .explanation-section {
      margin-bottom: 30px;
      padding-bottom: 20px;
      border-bottom: 1px solid #eee;
    }
    
    .explanation-section:last-child {
      border-bottom: none;
      margin-bottom: 0;
      padding-bottom: 0;
    }
    
    .explanation-content h3 {
      margin-top: 0;
      color: #1e88e5;
      font-size: 20px;
      margin-bottom: 15px;
    }
    
    .explanation-content h4 {
      color: #1565C0;
      font-size: 18px;
      margin-top: 0;
      margin-bottom: 10px;
    }
    
    .explanation-content p {
      margin-bottom: 15px;
    }
    
    .explanation-content ul {
      padding-left: 20px;
      margin-bottom: 15px;
    }
    
    .explanation-content li {
      margin-bottom: 8px;
    }
    
    /* Media query for smaller screens */
    @media (max-width: 768px) {
      .modal-content {
        width: 95%;
        margin: 5% auto;
      }
    }
    </style>
    """
    
    # Create the JavaScript code
    explanation_js = """
    <script>
    // Wait for page to fully load before initializing
    window.addEventListener('load', function() {
        const modal = document.getElementById('explanation-modal');
        const modalTitle = document.getElementById('modal-title');
        const closeModal = document.querySelector('.close-modal');
        const combinedExplanation = document.getElementById('combined-explanation');
        const kdeExplanation = document.getElementById('kde-explanation');
        const qqExplanation = document.getElementById('qq-explanation');
        const matchKdeExplanation = document.getElementById('match-kde-explanation');
        const matchEventsExplanation = document.getElementById('match-events-explanation');
        const explainButton = document.getElementById('explain-plot');
        
        // Set the active explanation based on plot type
        function showExplanation(plotType) {
            // Hide all explanations first
            combinedExplanation.classList.remove('active');
            kdeExplanation.classList.remove('active');
            qqExplanation.classList.remove('active');
            matchKdeExplanation.classList.remove('active');
            matchEventsExplanation.classList.remove('active');
            
            // Show the appropriate explanation
            if (plotType === 'combined') {
                combinedExplanation.classList.add('active');
                modalTitle.textContent = 'Understanding QQ and KDE Plots';
            } else if (plotType === 'kde') {
                kdeExplanation.classList.add('active');
                modalTitle.textContent = 'Understanding KDE Plots';
            } else if (plotType === 'qq') {
                qqExplanation.classList.add('active');
                modalTitle.textContent = 'Understanding QQ Plots';
            } else if (plotType === 'match-kde') {
                matchKdeExplanation.classList.add('active');
                modalTitle.textContent = 'Understanding Match KDE Plots';
            } else if (plotType === 'match-events') {
                matchEventsExplanation.classList.add('active');
                modalTitle.textContent = 'Understanding Match Events Plot';
            }
            
            // Show the modal
            modal.style.display = 'block';
            document.body.style.overflow = 'hidden'; // Prevent scrolling behind modal
        }
        
        // Add click event to the explain button
        if (explainButton) {
            explainButton.addEventListener('click', function() {
                const plotType = this.getAttribute('data-plot-type');
                showExplanation(plotType);
            });
        }
        
        // Close modal when clicking the X
        if (closeModal) {
            closeModal.addEventListener('click', function() {
                modal.style.display = 'none';
                document.body.style.overflow = 'auto'; // Restore scrolling
            });
        }
        
        // Close modal when clicking outside of it
        window.addEventListener('click', function(event) {
            if (event.target === modal) {
                modal.style.display = 'none';
                document.body.style.overflow = 'auto';
            }
        });
        
        // Close modal with Escape key
        document.addEventListener('keydown', function(event) {
            if (event.key === 'Escape' && modal.style.display === 'block') {
                modal.style.display = 'none';
                document.body.style.overflow = 'auto';
            }
        });
    });
    </script>
    """
    
    # Add content before the closing body tag
    if '</body>' in html_content:
        modified_html = html_content.replace('</body>', 
                                            f'{explanation_button_html}\n{explanation_modal_html}\n{explanation_css}\n{explanation_js}\n</body>')
    else:
        # If there's no closing body tag, append at the end
        modified_html = html_content + f'\n{explanation_button_html}\n{explanation_modal_html}\n{explanation_css}\n{explanation_js}'
    
    # Write the modified content back to the file
    with open(html_file_path, 'w', encoding='utf-8') as f:
        f.write(modified_html)

def add_explanations_to_all_plots(directory):
    """
    Recursively adds explanation buttons to all plot HTML files in a directory.
    
    Args:
        directory (str): Path to the directory containing HTML plot files
        
    Returns:
        int: Number of files modified
    """
    import os
    
    count = 0
    
    # Walk through all files in the directory and subdirectories
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.html'):
                file_path = os.path.join(root, file)
                
                # Skip the dashboard file
                if file == 'dashboard.html':
                    continue
                
                # Check if file contains plot data (simple check)
                try:
                    with open(file_path, 'r', encoding='utf-8') as f:
                        content = f.read(2000)  # Read first 2000 chars to check
                        if ('plotly' in content or 'Plotly' in content) and ('plot' in content.lower()):
                            # Add explanation buttons to the file
                            add_explanation_buttons_to_html(file_path)
                            count += 1
                except Exception as e:
                    print(f"Error checking file {file_path}: {e}")
    
    return count

In [20]:
def preprocess_game_data(df, selected_team):
    """
    Preprocess game data for a selected team.
    Filters the DataFrame for the selected team, converts time-related columns to the proper
    data types, and computes the game time in minutes from the match start (defined as the first
    loss event). Returns the processed DataFrame or None if no valid data is available.
    """
    team_data = df[df['team'] == selected_team].copy()
    team_data['loss_time'] = pd.to_datetime(team_data['loss_time'], errors='coerce')
    team_data['time_to_recover'] = pd.to_numeric(team_data['time_to_recover'], errors='coerce')
    team_data = team_data.dropna(subset=['loss_time', 'time_to_recover'])
    
    if team_data.empty:
        return None  

    # Set match start time to the first ball loss event
    match_start_time = team_data['loss_time'].min() 
    print(match_start_time)
    # Ensure match_start_time is valid
    if pd.isna(match_start_time):
        return None  

    # Compute game time in minutes from match start
    team_data['game_time_minutes'] = (team_data['loss_time'] - match_start_time).dt.total_seconds() / 60

    return team_data

# Periods to process
periods = [1, 2, 3, 4]

def get_flag_emoji(country_name: str) -> str:
    name = country_name.strip().lower()
    if name in ("england", "scotland", "wales", "northern ireland"):
        # use the United Kingdom flag
        return "".join(chr(0x1F1E6 + (ord(c) - ord('A'))) for c in "GB")
    try:
        country = pycountry.countries.lookup(country_name)
        code = country.alpha_2.upper()
        return "".join(chr(0x1F1E6 + (ord(c) - ord('A'))) for c in code)
    except Exception:
        return ""

def generate_match_plots(*team_dicts, output_dir="visuals"):  
    """
    Generate interactive recovery plots for football matches with match event tables
    and an index HTML page linking to them.
    This function considers only periods 1, 2, 3, and 4, and will skip any period with no data.
    The dropdown menu now shows only individual periods, with Period 1 selected by default.
    Can accept multiple team dictionaries at once.
    
    Args:
        *team_dicts: Variable number of tuples, each containing (team_name, team_color, matches_dict)
    """
    # Define periods to consider
    periods = [1, 2, 3, 4]
    
    # Define emoji mapping for event types
    emoji_map = {
        "Goal": "⚽",
        "Own Goal": "🚫⚽",
        "Substitution": "🔄",
        "Tactical Shift": "🔀",
        "Injury": "🤕",
        "Yellow Card": "🟨",
        "Red Card": "🟥"
    }
    
    # Define emojis for player substitutions
    player_in_emoji = "⬆️"  # Player coming in
    player_out_emoji = "⬇️"  # Player going out
    
    def get_recovery_value(event_time, team_data):
        """
        Given an event time and the team's recovery data (with 'game_time_minutes' and 'time_to_recover'),
        return an interpolated recovery value at that event time.
        """
        if team_data.empty:
            return None
        return np.interp(event_time, team_data['game_time_minutes'], team_data['time_to_recover'])
    
    def extract_player_name(row):
        """
        Extract player name from event data in a consistent way.
        """
        if row.get('type') == 'Substitution':
            player_out = row.get('player', {}).get('name', 'Unknown') if isinstance(row.get('player'), dict) else row.get('player', 'Unknown')
            player_in = row.get('substitution_replacement', {}).get('name', 'Unknown') if isinstance(row.get('substitution_replacement'), dict) else row.get('substitution_replacement', 'Unknown')
            return f"{player_in_emoji} {player_in} {player_out_emoji} {player_out}"
        return row.get('player', {}).get('name', 'Unknown') if isinstance(row.get('player'), dict) else row.get('player', 'Unknown')
    
    def process_match(match_id, matches_dict):
        """
        Process match events to create a structured dataframe for display.
        Only includes events from periods 1, 2, 3, and 4.
        Tracks formation changes for tactical shifts.
        Ensures scores are calculated correctly in chronological order.
        
        Args:
            match_id: The ID of the match to process
            matches_dict: Dictionary containing the match details
            
        Returns:
            A dictionary containing match information and processed dataframe
        """
        try:
            # Get the match details
            details = matches_dict.get(match_id)
            if not details:
                print(f"No details found for Match ID {match_id}")
                return None
                
            # Fetch event data for the match
            events_df = sb.events(match_id=match_id)
            if events_df.empty:
                print(f"No event data found for Match ID {match_id}")
                return None
            
            # Get the team names
            primary_team = details['team']
            
            # Determine the opposing team from events
            all_teams = events_df['team'].dropna().unique()
            opponent_teams = [team for team in all_teams if team != primary_team]
            opponent_team = opponent_teams[0] if opponent_teams else "Unknown Team"
            team_flags[opponent_team] = get_flag_emoji(opponent_team)

            # Process events into a display-friendly format
            processed_events = []
            
            # Filter events to only include periods 1, 2, 3, and 4
            valid_periods = [1, 2, 3, 4]
            events_df = events_df[events_df['period'].isin(valid_periods)]
            
            # Sort events by period and then by timestamp within each period
            # This ensures proper chronological ordering for score calculation
            events_df = events_df.sort_values(by=['period', 'timestamp'])
            
            # Remove duplicate events if present
            if 'id' in events_df.columns:
                events_df = events_df.drop_duplicates(subset=['id'])
            
            # Initialize score counters
            primary_score, opponent_score = 0, 0
            
            # Create dictionaries to track formations by team and period
            team_formations = {
                primary_team: {period: [] for period in valid_periods},
                opponent_team: {period: [] for period in valid_periods}
            }
            
            # Extract starting formation data
            starting_xi_events = events_df[events_df['type'] == 'Starting XI']
            for _, event in starting_xi_events.iterrows():
                team = event.get('team', 'Unknown')
                tactics = event.get('tactics') or {}
                formation = tactics.get('formation', 'Unknown')
                lineup = tactics.get('lineup', [])
                
                # Store the initial formation in period 1
                if team in team_formations:
                    team_formations[team][1].append({
                        'timestamp': event.get('timestamp'),
                        'minute': event.get('minute', 0),
                        'formation': formation,
                        'lineup': lineup
                    })
            
            # Process events
            for _, event in events_df.iterrows():
                event_type = event['type']
                team = event.get('team', 'Unknown')
                period = event.get('period', '')
                minute = event.get('minute', '')
                second = event.get('second', 0)
                time_display = f"{minute}" if pd.notna(minute) else ""
                
                evt_id = event.get('id')

                # Extract details based on event type
                details = ""
                if event_type == 'Shot' and event.get('shot_outcome') == 'Goal':
                    # Update score for the correct team
                    if team == primary_team:
                        primary_score += 1
                    else:
                        opponent_score += 1
                    event_type = 'Goal'
                    player_name = extract_player_name(event)
                    details = f"{extract_player_name(event)}: <strong> scored </strong>"

                elif event_type == 'Own Goal Against':
                    # Get the original team (the one that actually scored the own goal)
                    own_goal_by_team = team
                    own_goal_by_player = extract_player_name(event)
                    
                    # For own goals, credit the opposing team
                    if team == primary_team:
                        opponent_score += 1  # Primary team scored an own goal, opponent gets the point
                        benefiting_team = opponent_team
                    else:
                        primary_score += 1  # Opponent scored an own goal, primary team gets the point
                        benefiting_team = primary_team
                    
                    # Set the display team to the team that gets the point
                    team = benefiting_team
                    event_type = 'Own Goal'
                    
                    # Create clearer details with info about who scored the own goal
                    details = f"{own_goal_by_player} ({own_goal_by_team}): <strong>OWN GOAL ⚽ (point for {benefiting_team})</strong>"
                    
                    # Store this information for hover text later
                    event['own_goal_by_team'] = own_goal_by_team
                    event['own_goal_by_player'] = own_goal_by_player
                    event['own_goal_benefiting_team'] = benefiting_team

                elif event_type == 'Substitution':
                    player_name = extract_player_name(event)
                    details = player_name
                elif event_type == 'Injury Stoppage':
                    event_type = 'Injury'
                    player_name = extract_player_name(event)
                    details = f"{player_name}: <strong> injured </strong>"
                elif event_type == 'Tactical Shift':
                    # Get the new formation details
                    new_formation = event.get('tactics', {}).get('formation', 'Unknown')
                    lineup_data = event.get('tactics', {}).get('lineup', [])
                    
                    # Store the new formation in our tracking dictionary for this period
                    if team in team_formations and period in team_formations[team]:
                        team_formations[team][period].append({
                            'timestamp': event.get('timestamp'),
                            'minute': minute,
                            'formation': new_formation,
                            'lineup': lineup_data
                        })
                    
                    # Get the previous formation for this team in this period
                    prev_formation = None
                    prev_lineup = None
                    
                    if team in team_formations and period in team_formations[team]:
                        period_formations = team_formations[team][period]
                        
                        if len(period_formations) > 1:
                            # Get the previous formation from the same period
                            prev_idx = len(period_formations) - 2
                            prev_formation = period_formations[prev_idx]['formation']
                            prev_lineup = period_formations[prev_idx]['lineup']
                        elif period > 1:
                            # If this is the first formation change in this period,
                            # try to get the last formation from the previous period
                            for prev_period in range(period-1, 0, -1):
                                prev_period_formations = team_formations[team][prev_period]
                                if prev_period_formations:
                                    prev_formation = prev_period_formations[-1]['formation']
                                    prev_lineup = prev_period_formations[-1]['lineup']
                                    break
                    
                    # Create the JSON data for the lineups
                    lineup_json = json.dumps(lineup_data).replace('"', '&quot;')
                    prev_lineup_json = json.dumps(prev_lineup).replace('"', '&quot;') if prev_lineup else ""
                    
                    # Get team styling information
                    team_color = team_colors.get(team.strip().title(), 'gray')
                    flag = team_flags.get(team, "")
                    team_label = f"{flag} {team}"
                    
                    # Create the clickable span with all necessary data attributes
                    if prev_formation:
                        details = (
                            f'<span class="formation-details" '
                            f'data-lineup="{lineup_json}" '
                            f'data-prev-lineup="{prev_lineup_json}" '
                            f'data-team-color="{team_color}" '
                            f'data-team-name="{team_label}" '
                            f'data-formation="{new_formation}" '
                            f'data-prev-formation="{prev_formation}" '
                            f'data-period="{period}">'
                            f'Formation change: {prev_formation} → {new_formation} (click for details)</span>'
                        )
                    else:
                        details = (
                            f'<span class="formation-details" '
                            f'data-lineup="{lineup_json}" '
                            f'data-team-color="{team_color}" '
                            f'data-team-name="{team_label}" '
                            f'data-formation="{new_formation}" '
                            f'data-period="{period}">'
                            f'New formation: {new_formation} (click for details)</span>'
                        )
                elif event_type == 'Foul Committed':
                    card = event.get('foul_committed_card', '')
                    if card == 'Yellow Card':
                        event_type = 'Yellow Card'
                        player_name = extract_player_name(event)
                        details = f"{player_name}: <strong> fouled </strong>"
                    elif card == 'Red Card':
                        event_type = 'Red Card'
                        player_name = extract_player_name(event)
                        details = f"{player_name}: <strong> fouled </strong>"
                    else:
                        continue  
                else:
                    continue  

                # Add the event with the current score
                processed_events.append({
                    'Period': period,
                    'Event': event_type,
                    'Team': team,
                    'Minute': time_display,
                    'Score': f"{opponent_score}–{primary_score}",  # Current score after this event
                    'Details': details
                })
                
            # Create DataFrame from processed events
            match_df = pd.DataFrame(processed_events)
            
            # Ensure events are in correct order for display
            if not match_df.empty:
                # Convert Period to numeric for sorting
                match_df['Period_num'] = pd.to_numeric(match_df['Period'], errors='coerce')
                # Convert Minute to numeric for sorting (handle format like "45:00" by extracting first part)
                match_df['Minute_num'] = match_df['Minute'].apply(
                    lambda x: float(x.split(':')[0]) if isinstance(x, str) and ':' in x else float(x) if pd.notna(x) else 0
                )
                # Sort by Period and then by Minute
                match_df = match_df.sort_values(by=['Period_num', 'Minute_num'])
                # Drop the helper columns used for sorting
                match_df = match_df.drop(columns=['Period_num', 'Minute_num'])
            
            # Return match info, processed data, and formation tracking
            return {
                'match_id': match_id,
                'primary_team': primary_team,
                'opponent_team': opponent_team,
                'match_df': match_df,
                'team_formations': team_formations
            }
            
        except Exception as e:
            print(f"Error processing match {match_id}: {e}")
            import traceback
            traceback.print_exc()
            return None


    def generate_match_table_html(match_info, team_colors):
        """
        Generate HTML for the match event table, including kickoff formations
        shown like tactical shifts (clickable), with consistent team-color based styling.
        Works with both hex color codes and color names.
        """
        if match_info is None or match_info['match_df'].empty:
            return "<p>No match data available to display.</p>"

        match_df = match_info['match_df']
        primary_team = match_info['primary_team']
        opponent_team = match_info['opponent_team']
        match_id = match_info['match_id']
        team_formations = match_info.get('team_formations', {})
        

        def ensure_hex_color(color):
            """
            Convert any CSS‐style color name or hex code into normalized #RRGGBB.
            Falls back to light gray (#888888) if parsing fails.
            """
            try:
                # If it's already a hex code, normalize #RGB → #RRGGBB
                if color.startswith('#'):
                    hex_code = color.lstrip('#')
                    if len(hex_code) == 3:
                        hex_code = ''.join(c*2 for c in hex_code)
                    return '#' + hex_code.upper()
                # Otherwise let matplotlib parse the name (supports all CSS4/X11 names)
                rgb = mcolors.to_rgb(color)
                return mcolors.to_hex(rgb).upper()
            except Exception:
                return "#888888"

        # --- pull Kick-off lineup & formation from sb.events ---
        try:
            all_events  = sb.events(match_id=match_id)
            starting_xi = all_events[all_events['type'] == 'Starting XI']
        except Exception:
            starting_xi = []

        initial_formations = {}
        initial_lineups    = {}
        for _, ev in starting_xi.iterrows():
            team    = ev.get('team', 'Unknown')
            tactics = ev.get('tactics') or {}
            fv      = tactics.get('formation')
            form_str = str(fv) if fv else 'Unknown'
            initial_formations[team] = form_str
            initial_lineups[team]    = tactics.get('lineup', [])

        p_form = initial_formations.get(primary_team,  'Unknown')
        o_form = initial_formations.get(opponent_team, 'Unknown')
        p_line = initial_lineups.get(primary_team,    [])
        o_line = initial_lineups.get(opponent_team,   [])

        flag_p = team_flags.get(primary_team, "")
        flag_o = team_flags.get(opponent_team, "")
        
        # --- Define the opacity level for team color backgrounds: 20% ---
        opacity = "33"  
        
        # --- build HTML header (6 columns) ---
        html = f"""
        <div class="match-table-container">
        <h2>Match Events: {flag_p}{primary_team} vs {flag_o}{opponent_team}</h2>
        <table class="match-table">
            <thead>
            <tr>
                <th>Period</th>
                <th>Event</th>
                <th>Team</th>
                <th>Minute</th>
                <th>Score</th>
                <th>Details</th>
            </tr>
            </thead>
            <tbody>

            <!-- Kick-off formations -->
            <tr class="period-header">
                <td colspan="6"><strong>Kick-off Formations</strong></td>
            </tr>
            
        """

        # -- Primary kickoff row --
        p_lineup_json = json.dumps(p_line).replace('"', '&quot;')
        p_team_color_raw = team_colors.get(primary_team, '#888888')
        p_team_color = ensure_hex_color(p_team_color_raw)
        p_bg_color = f"background-color: {p_team_color}{opacity};"
        
        span_p = (
            f'<span class="formation-details" '
            f'data-lineup="{p_lineup_json}" '
            f'data-team-color="{p_team_color}" '
            f'data-team-name="{flag_p} {primary_team}" '
            f'data-formation="{p_form}" '
            f'data-period="1">'
            f'<strong>Starting formation: {p_form} (click for details)</strong></span>'
        )
        html += f"""
            <tr style="{p_bg_color}">
                <td>1</td>
                <td>Formation</td>
                <td><span style="
                color:#fff;
                background-color:{p_team_color};
                padding:2px 6px;
                border-radius:4px;
                font-weight:bold;
                ">{flag_p} {primary_team}</span></td>
                <td>Kick-off</td>
                <td>0–0</td>
                <td class="tactical-shift">{span_p}</td>
            </tr>
        """

        # -- Opponent kickoff row --
        o_lineup_json = json.dumps(o_line).replace('"', '&quot;')
        o_team_color_raw = team_colors.get(opponent_team, '#888888')
        o_team_color = ensure_hex_color(o_team_color_raw)
        o_bg_color = f"background-color: {o_team_color}{opacity};"
        
        span_o = (
            f'<span class="formation-details" '
            f'data-lineup="{o_lineup_json}" '
            f'data-team-color="{o_team_color}" '
            f'data-team-name="{flag_o} {opponent_team}" '
            f'data-formation="{o_form}" '
            f'data-period="1">'
            f'<strong>Starting formation: {o_form} (click for details)</strong></span>'
        )
        html += f"""
            <tr style="{o_bg_color}">
                <td>1</td>
                <td>Formation</td>
                <td><span style="
                color:#fff;
                background-color:{o_team_color};
                padding:2px 6px;
                border-radius:4px;
                font-weight:bold;
                ">{flag_o} {opponent_team}</span></td>
                <td>Kick-off</td>
                <td>0–0</td>
                <td class="tactical-shift">{span_o}</td>
            </tr>
        """

        # --- Event rows with consistent team-colored backgrounds ---
        current_period = None
        for _, row in match_df.iterrows():
            period = row['Period']
            if period != current_period:
                current_period = period
                html += f"""
            <tr class="period-header">
                <td colspan="6"><strong>Period {period}</strong></td>
            </tr>
            """
            # Get team info and color
            team = row['Team']
            team_color_raw = team_colors.get(team.strip().title(), '#888888')
            team_color = ensure_hex_color(team_color_raw)
            # Create consistent background color with the specified opacity
            bg_color = f"background-color: {team_color}{opacity};"
            
            # Team label styling
            team_label_style = (
                "color:#fff;"
                f"background-color:{team_color};"
                "padding:2px 6px;"
                "border-radius:4px;"
                "font-weight:bold;"
            )

            # Special formatting for certain event types
            if row['Event'] == 'Tactical Shift':
                details_td = f'<td class="tactical-shift">{row["Details"]}</td>'
            else:
                details_td = f'<td>{row["Details"]}</td>'

            html += f"""
            <tr style="{bg_color}">
                <td><strong>{row['Period']}</strong></td>
                <td>{row['Event']}</td>
                <td>
                    <span style="{team_label_style}">
                        {team_flags.get(team, "")} {team}
                    </span>
                </td>
                <td>{row['Minute']}</td>
                <td>{row['Score']}</td>
                {details_td}
            </tr>
        """

        html += """
            </tbody>
        </table>
        </div>
        """
        
        return html

    # Dictionary to store match information for the index page
    match_info = {}
    match_tables = {}  # Store processed match tables
    team_colors = {}  # Store team colors for styling
    all_teams = []
    team_flags = {}  

    for team_name, team_color, matches_dict in team_dicts:
        all_teams.append(team_name)
        team_colors[team_name] = team_color
        team_flags[team_name]  = get_flag_emoji(team_name)

        for match_id, details in matches_dict.items():
            opp = details['team']
            team_flags[opp] = get_flag_emoji(opp)
        
            opp_color_data = details.get('color')
            if isinstance(opp_color_data, dict):
                team_colors[opp] = opp_color_data.get('opposing',
                                                     opp_color_data.get('main',
                                                                        team_color))
            else:
                team_colors[opp] = opp_color_data or team_color

        for match_id, details in matches_dict.items():
            try:
                print(f"\nProcessing Match ID: {match_id} ({details['team']})...")
                
                # Use the team_name parameter instead of a hard-coded team name
                team1 = details['team'].replace(' ', '_').lower()
                team2 = team_name.replace(' ', '_').lower()
                
                # Retrieve the main team data from a global DataFrame variable based on a naming convention
                df_main = globals()[f"df_time_to_recover_{team2}_vs_{team1}"]
                
                # Determine the opposing team from the main team's DataFrame
                opposing_team = df_main[df_main['team'] != details['team']]['team'].unique()[0]

                flag_main = team_flags.get(details['team'], '')
                flag_opp  = team_flags.get(opposing_team, '')

                # Process match events for table display
                match_data = process_match(match_id, matches_dict)
                if match_data:
                    match_tables[match_id] = match_data
                
                # Save match info for later
                match_info[match_id] = {
                    'main_team': details['team'],
                    'opposing_team': opposing_team,
                    'match_name': f"{details['team']} vs {opposing_team}"
                }
                
                # Extract colors for each team with special handling for the specified team
                if details['team'] == team_name:
                    main_color = team_color
                elif isinstance(details.get('color'), dict):
                    main_color = details['color'].get('main', '#0000FF')  # default blue if not provided
                else:
                    main_color = details.get('color', '#0000FF')
                
                if opposing_team == team_name:
                    opposing_color = team_color
                else:
                    # Look up color for opposing team across all provided dictionaries
                    opposing_color = 'gray'  # default
                    for _, _, match_dict in team_dicts:
                        for _, d in match_dict.items():
                            if d['team'] == opposing_team:
                                if isinstance(d.get('color'), dict):
                                    opposing_color = d['color'].get('opposing', 'gray')
                                else:
                                    opposing_color = d.get('color', 'gray')
                                break
                
                # Store team colors for HTML styling
                team_colors[details['team']] = main_color
                team_colors[opposing_team] = opposing_color
                
                # Process data for both teams using the helper function
                main_team_data = preprocess_game_data(df_main, details['team'])
                opposing_team_data = preprocess_game_data(df_main, opposing_team)
                
                if main_team_data is None or opposing_team_data is None:
                    print(f"Missing team data for Match ID {match_id}. Skipping...")
                    continue
                
                # Fetch event data for the match
                events = sb.events(match_id=match_id)
                if events.empty:
                    print(f"No event data found for Match ID {match_id}. Skipping...")
                    continue
                
                # Convert timestamps and filter events for valid periods (1,2,3,4)
                events['timestamp'] = pd.to_datetime(events['timestamp'], errors='coerce')
                events = events[events['period'].isin(periods)]
                match_start_time = events['timestamp'].min()
                events['event_time_minutes'] = (events['timestamp'] - match_start_time).dt.total_seconds() / 60
                mask = events['type'] == 'Own Goal Against'

                if mask.any():
                    # 2) Rename them to plain “Own Goal”
                    events.loc[mask, 'type'] = 'Own Goal'
                    
                    # 3) Make sure your team‐column is a clean string
                    events['team'] = events['team'].astype(str).str.strip()
                    
                    # 4) Vectorized swap: if original team == primary_team → opposing_team, 
                    #    if original team == opposing_team → primary_team, else leave as-is.
                    teams = events.loc[mask, 'team']
                    events.loc[mask, 'team'] = np.where(
                        teams == details['team'],
                        opposing_team,
                        np.where(
                            teams == opposing_team,
                            details['team'],
                            teams
                        )
                    )
                # Create a folder for match outputs
                match_folder = os.path.join(output_dir, f"match_{match_id}")
                os.makedirs(match_folder, exist_ok=True)
                
                # Determine which periods have data for either team or events
                available_periods = []
                for period in periods:
                    main_period_data = main_team_data[main_team_data['period'] == period]
                    opp_period_data = opposing_team_data[opposing_team_data['period'] == period]
                    period_events = events[events['period'] == period]
                    if (not main_period_data.empty or not opp_period_data.empty or not period_events.empty):
                        available_periods.append(period)
                
                if not available_periods:
                    print(f"No valid period data found for Match ID {match_id}. Skipping...")
                    continue
                
                print(f"Available periods for Match ID {match_id}: {available_periods}")
                match_info[match_id]['available_periods'] = available_periods
                match_info[match_id]['filename'] = f"match_{match_id}_all_periods.html"
                
                # Create one figure for the entire match with all available periods
                fig = go.Figure()
                
                # Dictionary to store trace indices for each period
                period_trace_indices = {period: {"main": None, "opposing": None, "events": []} for period in available_periods}
                
                # Add traces for the main team's data for each available period
                for period in available_periods:
                    period_data = main_team_data[main_team_data['period'] == period]
                    if not period_data.empty:
                        fig.add_trace(go.Scatter(
                            x=period_data['game_time_minutes'],
                            y=period_data['time_to_recover'],
                            mode='lines+markers',
                            name=f"{flag_main} {details['team']} - Period {period}",
                            line=dict(color=main_color, width=2.5),
                            marker=dict(symbol='circle', size=8),
                            hovertemplate='<b>%{fullData.name}</b><br>Time: %{x:.0f} min<br>Recovery: %{y:.2f} sec<extra></extra>',
                            legendgroup=f"{details['team']}_p{period}",
                            showlegend=True,
                            visible=(period == available_periods[0])
                        ))
                        # Store the trace index for this period's main team data
                        period_trace_indices[period]["main"] = len(fig.data) - 1
                
                # Add traces for the opposing team's data for each available period
                for period in available_periods:
                    period_data = opposing_team_data[opposing_team_data['period'] == period]
                    if not period_data.empty:
                        fig.add_trace(go.Scatter(
                            x=period_data['game_time_minutes'],
                            y=period_data['time_to_recover'],
                            mode='lines+markers',
                            name=f"{flag_opp} {opposing_team} - Period {period}",
                            line=dict(color=opposing_color, width=2.5),
                            marker=dict(symbol='square', size=8),
                            hovertemplate='<b>%{fullData.name}</b><br>Time: %{x:.0f} min<br>Recovery: %{y:.2f} sec<extra></extra>',
                            legendgroup=f"{opposing_team}_p{period}",
                            showlegend=True,
                            visible=(period == available_periods[0])
                        ))
                        # Store the trace index for this period's opposing team data
                        period_trace_indices[period]["opposing"] = len(fig.data) - 1
                
                # Add event markers for each available period and team
                for period in available_periods:
                    period_events = events[events['period'] == period]
                    
                    key_events = {
                        "Goal": period_events[(period_events['type'] == 'Shot') & (period_events['shot_outcome'] == 'Goal')],
                        "Own Goal": period_events[period_events['type'] == 'Own Goal'],
                        "Substitution": period_events[period_events['type'] == 'Substitution'],
                        "Tactical Shift": period_events[period_events['type'] == 'Tactical Shift'],
                        "Injury": period_events[period_events['type'] == 'Injury Stoppage'],
                        "Yellow Card": period_events[(period_events['type'] == 'Foul Committed') & (period_events['foul_committed_card'] == 'Yellow Card')],
                        "Red Card": period_events[(period_events['type'] == 'Foul Committed') & (period_events['foul_committed_card'] == 'Red Card')]
                    }
                    
                    main_period_data = main_team_data[main_team_data['period'] == period]
                    opp_period_data = opposing_team_data[opposing_team_data['period'] == period]
                    
                    for event_type, events_df in key_events.items():
                        if events_df.empty:
                            continue
                        
                        events_df = events_df.copy()
                        events_df['player_name'] = events_df.apply(extract_player_name, axis=1)
                        
                        # For tactical shifts, extract the new formation
                        if event_type == "Tactical Shift":
                            events_df['new_formation'] = events_df.apply(
                                lambda row: row.get('tactics', {}).get('formation', 'Unknown'), axis=1
                            )
                        
                        if "team" in events_df.columns:
                            main_events = events_df[events_df['team'].str.lower() == details['team'].lower()]
                            opp_events = events_df[events_df['team'].str.lower() == opposing_team.lower()]
                            other_events = events_df[~events_df['team'].str.lower().isin([details['team'].lower(), opposing_team.lower()])]
                        else:
                            main_events = events_df.copy()
                            opp_events = pd.DataFrame()
                            other_events = pd.DataFrame()
                        
                        emoji = emoji_map.get(event_type, "❓")
                        
                        def create_hover_text(row, base_team, period, team_label):
                            # Special handling for Own Goal events
                            if event_type == "Own Goal":
                                # Get the actual team that scored the own goal if available
                                if 'own_goal_by_team' in row:
                                    own_goal_by_team = row['own_goal_by_team']
                                    own_goal_by_player = row['own_goal_by_player']
                                    benefiting_team = row['own_goal_benefiting_team']
                                else:
                                    # Fallback if we don't have the special fields
                                    own_goal_by_team = "Unknown"
                                    own_goal_by_player = row['player_name']
                                    benefiting_team = team_label.split(' ')[-1]
                                
                                # Get flag emojis for the teams
                                own_goal_flag = team_flags.get(own_goal_by_team, "")
                                benefiting_flag = team_flags.get(benefiting_team, "")
                                
                                return (
                                    f"<b>OWN GOAL 🚫⚽</b><br>" +
                                    f"Time: {int(row['event_time_minutes'])} min<br>" +
                                    f"Own goal by: {own_goal_by_player}<br>" +
                                    f"Point awarded to: {benefiting_flag} {benefiting_team}<br>" +
                                    f"Period: {period}"
                                )
                            
                            # Regular handling for other event types
                            base_text = (
                                f"<b>{event_type}</b><br>" +
                                f"Time: {int(row['event_time_minutes'])} min<br>" +
                                f"Team: {team_label}<br>" +
                                f"Period: {period}"
                            )
                            
                            if event_type == "Tactical Shift":
                                formation_text = f"<br>New Formation: {row['new_formation']}"
                                return base_text + formation_text
                            else:
                                # For all other events, show the player name
                                player_text = f"<br>Player: {row['player_name']}"
                                return base_text + player_text
                        
                        # Plot events on the main team's curve
                        if not main_events.empty and not main_period_data.empty:
                            # Special handling for own goals to ensure we can access the additional fields
                            if event_type == "Own Goal":
                                hover_texts = []
                                for _, ev_row in main_events.iterrows():
                                    hover_texts.append(create_hover_text(ev_row, main_period_data, period, f"{flag_main} {details['team']}"))
                            else:
                                hover_texts = main_events.apply(
                                    lambda row: create_hover_text(row, main_period_data, period, f"{flag_main} {details['team']}"), 
                                    axis=1
                                )
                            
                            fig.add_trace(go.Scatter(
                                x=main_events['event_time_minutes'],
                                y=main_events['event_time_minutes'].apply(lambda t: get_recovery_value(t, main_period_data)),
                                mode='markers+text',
                                marker=dict(color=main_color, size=10, opacity=0.7, line=dict(color='black', width=1)),
                                text=[emoji] * len(main_events),
                                textposition='middle center',
                                name=f"{emoji} {event_type} ({flag_main} {details['team']})",
                                hovertemplate='%{hovertext}<extra></extra>',
                                hovertext=hover_texts,
                                legendgroup=f"{details['team']}_p{period}",
                                showlegend=True,
                                visible=(period == available_periods[0])
                            ))
                            # Store this event trace index for this period
                            period_trace_indices[period]["events"].append(len(fig.data) - 1)
                        
                        # Plot events on the opposing team's curve
                        if not opp_events.empty and not opp_period_data.empty:
                            # Special handling for own goals
                            if event_type == "Own Goal":
                                hover_texts = []
                                for _, ev_row in opp_events.iterrows():
                                    hover_texts.append(create_hover_text(ev_row, opp_period_data, period, f"{flag_opp} {opposing_team}"))
                            else:
                                hover_texts = opp_events.apply(
                                    lambda row: create_hover_text(row, opp_period_data, period, f"{flag_opp} {opposing_team}"), 
                                    axis=1
                                )
                            
                            fig.add_trace(go.Scatter(
                                x=opp_events['event_time_minutes'],
                                y=opp_events['event_time_minutes'].apply(lambda t: get_recovery_value(t, opp_period_data)),
                                mode='markers+text',
                                marker=dict(color=opposing_color, size=10, opacity=0.7, line=dict(color='black', width=1)),
                                text=[emoji] * len(opp_events),
                                textposition='middle center',
                                name=f"{emoji} {event_type} ({flag_opp} {opposing_team})",
                                hovertemplate='%{hovertext}<extra></extra>',
                                hovertext=hover_texts,
                                legendgroup=f"{opposing_team}_p{period}",
                                showlegend=True,
                                visible=(period == available_periods[0])
                            ))
                            # Store this event trace index for this period
                            period_trace_indices[period]["events"].append(len(fig.data) - 1)
                
                # Create dropdown buttons
                buttons = []
                
                # "All Periods" button: makes all traces visible.
                all_periods_visible = [True] * len(fig.data)
                buttons.append(
                    dict(
                        args=[
                            {'visible': all_periods_visible},
                            {'title': {
                                'text': f"Time to Recover Ball - {flag_main} {details['team']} vs {flag_opp} {opposing_team} (All Periods)",
                                'font': {'size': 20, 'family': 'Arial', 'color': 'black'},
                                'y': 0.95
                            }}
                        ],
                        label="All Periods",
                        method="update"
                    )
                )
                
                # Now add one button for each available period
                for period in available_periods:
                    # Set all traces to invisible first
                    period_visible = [False] * len(fig.data)
                    
                    # Make the main team trace for this period visible
                    if period_trace_indices[period]["main"] is not None:
                        period_visible[period_trace_indices[period]["main"]] = True
                    
                    # Make the opposing team trace for this period visible
                    if period_trace_indices[period]["opposing"] is not None:
                        period_visible[period_trace_indices[period]["opposing"]] = True
                    
                    # Make all event traces for this period visible
                    for event_idx in period_trace_indices[period]["events"]:
                        period_visible[event_idx] = True
                    
                    buttons.append(
                        dict(
                            args=[
                                {'visible': period_visible},
                                {'title': {
                                    'text': f"Time to Recover Ball - {flag_main} {details['team']} vs {flag_opp} {opposing_team} (Period {period})",
                                    'font': {'size': 20, 'family': 'Arial', 'color': 'black'},
                                    'y': 0.95
                                }}
                            ],
                            label=f"Period {period}",
                            method="update"
                        )
                    )
                
                # Set the active button index so that Period 1 is shown by default.
                # Since the "All Periods" button is at index 0, the Period 1 button is at index 1.
                active_button = 1 if len(buttons) > 1 else 0
                
                updatemenus = [
                    dict(
                        active=active_button,  # Default active button corresponds to Period 1.
                        buttons=buttons,
                        direction="down",
                        pad={"r": 10, "t": 10},
                        showactive=True,
                        x=0.1,
                        xanchor="left",
                        y=1.15,
                        yanchor="top",
                        name="period_selector"
                    )
                ]
                
                fig.update_layout(
                    title={
                        'text': f"Time to Recover Ball - {flag_main} {details['team']} vs {flag_opp}{opposing_team} (Period {available_periods[0]})",
                        'font': {'size': 20, 'family': 'Arial', 'color': 'black'},
                        'y': 0.95
                    },
                    xaxis_title={
                        'text': "Game Time (minutes)",
                        'font': {'size': 16, 'family': 'Arial'}
                    },
                    yaxis_title={
                        'text': "Time to Recover (seconds)",
                        'font': {'size': 16, 'family': 'Arial'}
                    },
                    updatemenus=updatemenus,
                    legend_title="Teams and Events",
                    hovermode="closest",
                    plot_bgcolor='white',
                    paper_bgcolor='white',
                    font=dict(family="Arial", size=14),
                    legend=dict(
                        bordercolor="Black",
                        borderwidth=1,
                        bgcolor="white",
                        font=dict(size=12),
                        groupclick="toggleitem",
                        traceorder="grouped"
                    ),
                    width=1000,
                    height=600,
                    margin=dict(l=80, r=80, t=100, b=80)
                )
                
                fig.update_xaxes(
                    showgrid=True,
                    gridwidth=1,
                    gridcolor='lightgray',
                    showline=True,
                    linewidth=2,
                    linecolor='black'
                )
                
                fig.update_yaxes(
                    showgrid=True,
                    gridwidth=1,
                    gridcolor='lightgray',
                    showline=True,
                    linewidth=2,
                    linecolor='black'
                )
                
                # Create match event table HTML if available
                match_table_html = ""
                if match_id in match_tables:
                    match_table_html = generate_match_table_html(match_tables[match_id], team_colors)
                
                # Add toggle for lines visualization and custom JavaScript for synchronized toggling
                custom_js = """
<script>
document.addEventListener('DOMContentLoaded', function() {
    // Function to wait for Plotly to be fully loaded
    function waitForPlotly() {
        if (typeof Plotly !== 'undefined' && document.querySelector('.plotly')) {
            setupSynchronizedToggle();
            setupLinesToggle();
            setupFormationComparison(); // Add our new function for formation comparison
        } else {
            setTimeout(waitForPlotly, 100);
        }
    }
    
    // Function to set up synchronized toggle for legend groups
    function setupSynchronizedToggle() {
        const gd = document.querySelector('.plotly');
        if (!gd || !gd.data) return;
        const origLegendClick = gd._context.legendClick;
        gd._context.legendClick = function(curveNumber) {
            const clickedTrace = gd.data[curveNumber];
            if (!clickedTrace || !clickedTrace.legendgroup) return origLegendClick.call(this, curveNumber);
            const legendGroup = clickedTrace.legendgroup;
            const isVisible = clickedTrace.visible === 'legendonly' ? true : 'legendonly';
            const tracesToUpdate = [];
            const visibilityValues = [];
            gd.data.forEach((trace, i) => {
                if (trace.legendgroup === legendGroup) {
                    tracesToUpdate.push(i);
                    visibilityValues.push(isVisible);
                }
            });
            if (tracesToUpdate.length > 0) {
                Plotly.restyle(gd, {'visible': visibilityValues}, tracesToUpdate);
                return false;
            }
            return origLegendClick.call(this, curveNumber);
        };
    }
    
    // Function to set up lines toggle button
    function setupLinesToggle() {
        const gd = document.querySelector('.plotly');
        if (!gd || !gd.data) return;
        
        // Create toggle button
        const toggleButton = document.createElement('button');
        toggleButton.innerText = 'Toggle Lines/Points';
        toggleButton.style.position = 'absolute';
        toggleButton.style.top = '20px';
        toggleButton.style.right = '20px';
        toggleButton.style.zIndex = '999';
        toggleButton.style.padding = '8px 16px';
        toggleButton.style.backgroundColor = '#f8f9fa';
        toggleButton.style.border = '1px solid #ddd';
        toggleButton.style.borderRadius = '4px';
        toggleButton.style.cursor = 'pointer';
        
        // Add button to the plot container
        const plotlyContainer = document.querySelector('.plotly-container');
        if (plotlyContainer) {
            plotlyContainer.style.position = 'relative';
            plotlyContainer.appendChild(toggleButton);
            
            // Toggle between lines+markers and markers only
            let showLines = true;
            toggleButton.addEventListener('click', function() {
                showLines = !showLines;
                
                const mainTraces = [];
                const mainModes = [];
                
                gd.data.forEach((trace, i) => {
                    if (trace.mode && trace.mode.includes('lines')) {
                        mainTraces.push(i);
                        mainModes.push(showLines ? 'lines+markers' : 'markers');
                    }
                });
                
                if (mainTraces.length > 0) {
                    Plotly.restyle(gd, {'mode': mainModes}, mainTraces);
                }
            });
        }
    }
    
    // Function to set up formation comparison functionality
    function setupFormationComparison() {
        document.querySelectorAll('.formation-details').forEach(function(el) {
            // Style the formation details elements
            el.style.cursor = 'pointer';
            el.style.color = '#0288d1';
            el.style.textDecoration = 'underline';
            
            // Add click event listener
            el.addEventListener('click', function() {
                try {
                    // Get data from attributes
                    const lineup = JSON.parse(this.getAttribute('data-lineup').replace(/&quot;/g, '"'));
                    const prevLineup = this.hasAttribute('data-prev-lineup') ? 
                      JSON.parse(this.getAttribute('data-prev-lineup').replace(/&quot;/g, '"')) : null;
                    const teamColor = this.getAttribute('data-team-color');
                    const teamName = this.getAttribute('data-team-name');
                    const formationName = this.getAttribute('data-formation') || 'Unknown';
                    const prevFormationName = this.getAttribute('data-prev-formation') || null;
                    
                    // Create modal HTML
                    let modalHTML = `
                      <div id="formationModal" style="position:fixed; top:0; left:0; width:100%; height:100%; background:rgba(0,0,0,0.5); z-index:1000; display:flex; justify-content:center; align-items:center;">
                        <div style="background:white; border-radius:8px; width:90%; max-width:900px; max-height:90vh; overflow:auto; box-shadow:0 4px 15px rgba(0,0,0,0.2); position:relative;">
                          <div style="display:flex; justify-content:space-between; align-items:center; padding:15px; border-bottom:2px solid ${teamColor};">
                            <h3 style="margin:0; color:${teamColor};">${teamName} Formation Details</h3>
                            <button id="closeFormationModal" style="background:none; border:none; font-size:24px; cursor:pointer;">×</button>
                          </div>
                          <div style="padding:20px;">`;
                    
                    // If we have a previous formation to compare with
                    if (prevLineup) {
                      modalHTML += `
                        <div style="text-align:center; margin-bottom:15px;">
                          <h4 style="margin:5px 0;">Formation Change</h4>
                          <div style="font-size:18px;">
                            <span style="color:#666;">${prevFormationName}</span>
                            <span style="margin:0 10px;">→</span>
                            <span style="font-weight:bold;">${formationName}</span>
                          </div>
                        </div>
                        
                        <div style="display:flex; gap:20px; margin-bottom:20px;">
                          <!-- Left side: Previous Formation - UPDATED TO LIGHT GRAY -->
                          <div style="flex:1; border:1px solid #eee; border-radius:8px; padding:10px; background-color:#f9f9f9;">
                            <h4 style="text-align:center; margin-top:0; color:#777;">Previous Formation</h4>
                            <div style="overflow-x:auto;">
                              <table style="width:100%; border-collapse:collapse;">
                                <thead>
                                  <tr style="background-color:#e0e0e0; color:#555;">
                                    <th style="text-align:left; padding:10px;">Position</th>
                                    <th style="text-align:center; padding:10px; width:80px;">#</th>
                                    <th style="text-align:left; padding:10px;">Player</th>
                                  </tr>
                                </thead>
                                <tbody>
                      `;
                      
                      // Group previous players by position type
                      const prevPlayers = groupPlayersByPosition(prevLineup);
                      
                      // Add previous players to the table - using gray color for previous formation
                      const prevColor = "#aaaaaa"; // Light gray for previous formation
                      modalHTML += renderPlayerGroup(prevPlayers.gk, 'Goalkeepers', prevColor, {}, true);
                      modalHTML += renderPlayerGroup(prevPlayers.defenders, 'Defenders', prevColor, {}, true);
                      modalHTML += renderPlayerGroup(prevPlayers.midfielders, 'Midfielders', prevColor, {}, true);
                      modalHTML += renderPlayerGroup(prevPlayers.forwards, 'Forwards', prevColor, {}, true);
                      modalHTML += renderPlayerGroup(prevPlayers.others, 'Others', prevColor, {}, true);
                      
                      modalHTML += `
                                </tbody>
                              </table>
                            </div>
                          </div>
                          
                          <!-- Right side: New Formation -->
                          <div style="flex:1; border:1px solid ${teamColor}; border-radius:8px; padding:10px;">
                            <h4 style="text-align:center; margin-top:0; color:${teamColor};">Current Formation</h4>
                            <div style="overflow-x:auto;">
                              <table style="width:100%; border-collapse:collapse;">
                                <thead>
                                  <tr style="background-color:${teamColor}; color:white;">
                                    <th style="text-align:left; padding:10px;">Position</th>
                                    <th style="text-align:center; padding:10px; width:80px;">#</th>
                                    <th style="text-align:left; padding:10px;">Player</th>
                                  </tr>
                                </thead>
                                <tbody>
                      `;
                      
                      // Find changes between lineups
                      const changes = identifyChanges(prevLineup, lineup);
                      
                      // Group current players by position type
                      const currentPlayers = groupPlayersByPosition(lineup);
                      
                      // Add current players to the table with changes highlighted
                      modalHTML += renderPlayerGroup(currentPlayers.gk, 'Goalkeepers', teamColor, changes, false);
                      modalHTML += renderPlayerGroup(currentPlayers.defenders, 'Defenders', teamColor, changes, false);
                      modalHTML += renderPlayerGroup(currentPlayers.midfielders, 'Midfielders', teamColor, changes, false);
                      modalHTML += renderPlayerGroup(currentPlayers.forwards, 'Forwards', teamColor, changes, false);
                      modalHTML += renderPlayerGroup(currentPlayers.others, 'Others', teamColor, changes, false);
                      
                      modalHTML += `
                                </tbody>
                              </table>
                            </div>
                          </div>
                        </div>
                        
                        <div style="margin-top:15px; background-color:#f8f9fa; padding:10px; border-radius:8px;">
                          <h4 style="margin-top:0;">Legend</h4>
                          <div style="display:flex; flex-wrap:wrap; gap:15px;">
                            <div style="display:flex; align-items:center;">
                              <span style="display:inline-block; width:16px; height:16px; background-color:#FFEB3B; margin-right:5px;"></span>
                              Position changed
                            </div>
                            <div style="display:flex; align-items:center;">
                              <span style="display:inline-block; width:16px; height:16px; background-color:#4CAF50; margin-right:5px;"></span>
                              New player
                            </div>
 
                          </div>
                        </div>
                      `;
                    } else {
                      // Single formation display (for kick-off formations)
                      modalHTML += `
                        <h4 style="text-align:center; margin-top:0;">Starting formation: ${formationName}</h4>
                        <table style="width:100%; border-collapse:collapse;">
                          <caption style="caption-side:top; font-size:18px; padding:8px 0;">
                            ${teamName}
                          </caption>
                          <thead>
                            <tr style="background-color:${teamColor}; color:white;">
                              <th style="text-align:left; padding:10px;">Position</th>
                              <th style="text-align:center; padding:10px; width:80px;">#</th>
                              <th style="text-align:left; padding:10px;">Player</th>
                            </tr>
                          </thead>
                          <tbody>
                      `;
                      
                      // Group players by position type
                      const players = groupPlayersByPosition(lineup);
                      
                      // Add players to the table
                      modalHTML += renderPlayerGroup(players.gk, 'Goalkeepers', teamColor, {}, false);
                      modalHTML += renderPlayerGroup(players.defenders, 'Defenders', teamColor, {}, false);
                      modalHTML += renderPlayerGroup(players.midfielders, 'Midfielders', teamColor, {}, false);
                      modalHTML += renderPlayerGroup(players.forwards, 'Forwards', teamColor, {}, false);
                      modalHTML += renderPlayerGroup(players.others, 'Others', teamColor, {}, false);
                      
                      modalHTML += `
                          </tbody>
                        </table>
                      `;
                    }
                    
                    // Close the HTML
                    modalHTML += `
                          </div>
                        </div>
                      </div>
                    `;
                    
                    // Add modal to the document
                    const modalContainer = document.createElement('div');
                    modalContainer.innerHTML = modalHTML;
                    document.body.appendChild(modalContainer);
                    
                    // Add close event
                    document.getElementById('closeFormationModal').addEventListener('click', function() {
                      document.body.removeChild(modalContainer);
                    });
                    
                    // Close on background click
                    document.getElementById('formationModal').addEventListener('click', function(e) {
                      if (e.target.id === 'formationModal') {
                        document.body.removeChild(modalContainer);
                      }
                    });
                    
                } catch (error) {
                    console.error('Error showing formation details:', error);
                    alert('There was an error displaying the formation details.');
                }
            });
        });
    }

    // Helper function to group players by position
    function groupPlayersByPosition(lineup) {
      let gk = [];
      let defenders = [];
      let midfielders = [];
      let forwards = [];
      let others = [];
      
      if (!lineup) return { gk, defenders, midfielders, forwards, others };
      
      lineup.forEach(player => {
        const pos = player.position?.name || '';
        if (pos.includes('Goalkeeper')) gk.push(player);
        else if (pos.includes('Back') || pos.includes('Center Back')) defenders.push(player);
        else if (pos.includes('Midfield') || pos.includes('Wing')) midfielders.push(player);
        else if (pos.includes('Forward') || pos.includes('Striker')) forwards.push(player);
        else others.push(player);
      });
      
      return { gk, defenders, midfielders, forwards, others };
    }

    // Helper function to identify changes between two lineups
    function identifyChanges(oldLineup, newLineup) {
      const changes = {
        positionChanged: {},  // Players who changed positions
        newPlayers: {},       // Players who are new
        removedPlayers: {}    // Players who were removed
      };
      
      if (!oldLineup || !newLineup) return changes;
      
      // Create maps for easy lookup
      const oldPlayerMap = {};
      const newPlayerMap = {};
      
      oldLineup.forEach(player => {
        const playerId = player.player?.id || player.player?.name;
        if (playerId) {
          oldPlayerMap[playerId] = player;
        }
      });
      
      newLineup.forEach(player => {
        const playerId = player.player?.id || player.player?.name;
        if (playerId) {
          newPlayerMap[playerId] = player;
          
          // Check if player existed in old lineup
          if (oldPlayerMap[playerId]) {
            // Check if position changed
            const oldPos = oldPlayerMap[playerId].position?.name;
            const newPos = player.position?.name;
            
            if (oldPos !== newPos) {
              changes.positionChanged[playerId] = {
                oldPosition: oldPos,
                newPosition: newPos
              };
            }
          } else {
            // This is a new player
            changes.newPlayers[playerId] = player;
          }
        }
      });
      
      // 
      
      return changes;
    }

    // Helper function to render a group of players in the table
    function renderPlayerGroup(players, sectionName, teamColor, changes, isPrevious) {
      if (players.length === 0) return '';
      
      // For previous formation, use a consistent light gray for section headers
      // For current formation, use team color with opacity
      const sectionBgColor = isPrevious ? "#f0f0f0" : `${teamColor}20`;
      
      let html = `
        <tr style="background-color:${sectionBgColor};">
          <td colspan="3" style="padding:8px; font-weight:bold; color:${isPrevious ? "#666666" : "#333333"};">${sectionName}</td>
        </tr>
      `;
      
      players.forEach((player, i) => {
        // Use different background colors for previous and current formation rows
        const bgColor = isPrevious ? (i % 2 === 0 ? '#f2f2f2' : '#e9e9e9') : (i % 2 === 0 ? '#f9f9f9' : 'white');
        const playerId = player.player?.id || player.player?.name;
        let highlightStyle = '';
        let statusIcon = '';
        
        if (!isPrevious && playerId) {
          // Highlight changes in the current formation
          if (changes.positionChanged && changes.positionChanged[playerId]) {
            highlightStyle = 'background-color:#FFEB3B50;';
            statusIcon = `<span title="Position changed: ${changes.positionChanged[playerId].oldPosition} → ${changes.positionChanged[playerId].newPosition}" style="margin-left:5px;">↔️</span>`;
          } else if (changes.newPlayers && changes.newPlayers[playerId]) {
            highlightStyle = 'background-color:#4CAF5050;';
            statusIcon = '<span title="New player" style="margin-left:5px;">➕</span>';
          }
        } else if (isPrevious && playerId && changes.removedPlayers && changes.removedPlayers[playerId]) {
          // Highlight removed players in the previous formation
          highlightStyle = 'background-color:#F4433650;';
          statusIcon = '<span title="Removed" style="margin-left:5px;">❌</span>';
        }
        
        // Use different text colors for previous and current formations
        const textColor = isPrevious ? "#666666" : "#333333";
        
        html += `
          <tr style="background-color:${bgColor}; ${highlightStyle}">
            <td style="padding:8px; border-bottom:1px solid #eee; color:${textColor};">${player.position?.name || 'Unknown'}</td>
            <td style="padding:8px; border-bottom:1px solid #eee; text-align:center; font-weight:bold; color:${textColor};">${player.jersey_number || '-'}</td>
            <td style="padding:8px; border-bottom:1px solid #eee; color:${textColor};">${player.player?.name || 'Unknown Player'} ${statusIcon}</td>
          </tr>
        `;
      });
      
      return html;
    }
    
    waitForPlotly();
});
</script>
"""
                
                # Create CSS for the match table
                table_css = """
                <style>
                /* Updated CSS for match table with team-colored backgrounds */
.match-table-container {
    margin-top: 30px;
    font-family: Arial, sans-serif;
    box-shadow: 0 4px 8px rgba(0,0,0,0.1);
    border-radius: 8px;
    overflow: hidden;
}

.match-table-container h2 {
    margin-bottom: 15px;
    font-size: 22px;
    text-align: center;
    padding: 15px;
    background-color: #f8f9fa;
    border-bottom: 1px solid #eee;
}

.match-table {
    width: 100%;
    border-collapse: collapse;
    margin-bottom: 0;
    border: none;
}

.match-table th, .match-table td {
    padding: 10px 12px;
    text-align: center;
    border: 1px solid #eee;
}

.match-table th {
    background-color: #2c3e50;
    color: white;
    font-weight: bold;
    text-transform: uppercase;
    font-size: 14px;
}

/* Ensure text is clearly visible on colored backgrounds */
.match-table td {
    color: #333;
    font-weight: normal;
}

/* Make headers stand out more */
.period-header {
    background-color: #34495e !important;
    color: white;
    font-size: 16px;
    text-align: left;
}

.period-header td {
    padding: 8px 12px;
    color: white !important;
}

/* Special event formatting */
.goal-event {
    font-weight: bold;
}

.tactical-shift {
    font-weight: bold;
    cursor: pointer;
}

/* Additional styling for the team pills */
.match-table td span {
    display: inline-block;
    margin: 0 auto;
    white-space: nowrap;
}

/* Make hover effect subtle to not hide team colors */
.match-table tr:hover {
    filter: brightness(1.05);
    transition: all 0.2s ease;
}

/* Adjust formation details styling */
.formation-details {
    cursor: pointer;
    text-decoration: underline;
    color: #2980b9;
}

.formation-details:hover {
    color: #3498db;
}
                </style>
                """
                
                html_file = os.path.join(match_folder, f"match_{match_id}_all_periods.html")
                html_content = pio.to_html(fig, full_html=True, include_plotlyjs=True)
                
                # Add our custom elements to the HTML
                # First, add the CSS, JS, and wrap the plot in a container
                html_content = html_content.replace('<body>', f'<body>\n{table_css}')
                html_content = html_content.replace('<div id="', '<div class="plotly-container"><div id="')
                # Use formatted string with actual content, not literal variable names
                html_content = html_content.replace('</div>\n</body>', f'</div></div>\n{match_table_html}\n{custom_js}\n</body>')
                
                with open(html_file, 'w', encoding='utf-8') as f:
                    f.write(html_content)
                
                print(f"Interactive plot with match table saved for Match ID {match_id} at {html_file}")
                
            except Exception as e:
                print(f"Error processing Match ID {match_id}: {e}")
                import traceback
                traceback.print_exc()
    
    # Create an index HTML page linking to all match plots if any were processed
    if match_info:
        # Create a unified index page for all teams
        index_html_title = "Football Match Recovery Analysis - " + ", ".join(all_teams)
        
        sorted_match_ids = sorted(match_info.keys(),
                                 key=lambda x: (match_info[x]['main_team'], match_info[x]['opposing_team']))
        
        index_html = f"""
        <!DOCTYPE html>
        <html>
        <head>
            <title>{index_html_title}</title>
            <style>
                body {{
                    font-family: Arial, sans-serif;
                    margin: 0;
                    padding: 20px;
                    background-color: #f8f9fa;
                }}
                .container {{
                    max-width: 1200px;
                    margin: 0 auto;
                    background-color: white;
                    padding: 20px;
                    box-shadow: 0 0 10px rgba(0,0,0,0.1);
                    border-radius: 5px;
                }}
                .header {{
                    text-align: center;
                    margin-bottom: 20px;
                }}
                .match-list {{
                    list-style-type: none;
                    padding: 0;
                }}
                .match-item {{
                    padding: 15px;
                    border-bottom: 1px solid #eee;
                    transition: background-color 0.2s;
                    cursor: pointer;
                }}
                .match-item:hover {{
                    background-color: #f5f5f5;
                }}
                .match-item.active {{
                    background-color: #e6f7ff;
                    border-left: 3px solid #1890ff;
                }}
                .match-link {{
                    display: block;
                    color: #0366d6;
                    text-decoration: none;
                    font-size: 18px;
                }}
                .match-info {{
                    margin-top: 5px;
                    font-size: 14px;
                    color: #666;
                }}
                .period-tag {{
                    display: inline-block;
                    background-color: #e1e4e8;
                    padding: 2px 8px;
                    border-radius: 10px;
                    margin-right: 5px;
                    font-size: 12px;
                }}
                iframe {{
                    width: 100%;
                    height: 800px; /* Increased height to show plot and table */
                    border: none;
                    margin-top: 20px;
                }}
            </style>
        </head>
        <body>
            <div class="container">
                <div class="header">
                    <h1>{index_html_title}</h1>
                    <p>Select a match to view detailed recovery analysis</p>
                </div> 
                <div class="match-selector">
                    <ul class="match-list">
        """
        
        for match_id in sorted_match_ids:
            m_info = match_info[match_id]
            p = m_info['main_team']
            o = m_info['opposing_team']
            flag_p = team_flags.get(p, "")
            flag_o = team_flags.get(o, "")
            match_name = m_info['match_name']
            main_team_str = m_info['main_team']
            opposing_team_str = m_info['opposing_team']
            periods_str = ' '.join([f'<span class="period-tag">Period {p}</span>' for p in m_info['available_periods']])
            
            index_html += f"""
                        <li class="match-item" onclick="loadMatch('{match_id}')">
            <a href="#" class="match-link">
                {flag_p} {p} vs {flag_o} {o}
            </a>
                            <div class="match-info">
                                {main_team_str} vs {opposing_team_str} | Available periods: {periods_str}
                            </div>
                        </li>
            """
        
        index_html += """
                    </ul>
                </div>
                <div id="match-viewer">
                    <iframe id="match-frame" src="" style="display:none;"></iframe>
                    <div id="initial-message" style="text-align:center; padding:50px;">
                        <h2>Click on a match from the list above to view it</h2>
                    </div>
                </div>
            </div>
            <script>
                function loadMatch(matchId) {
                    const matchItems = document.querySelectorAll('.match-item');
                    matchItems.forEach(item => {
                        item.classList.remove('active');
                    });
                    const clickedItem = document.querySelector(`.match-item[onclick="loadMatch('${matchId}')"]`);
                    if (clickedItem) {
                        clickedItem.classList.add('active');
                    }
                    document.getElementById('initial-message').style.display = 'none';
                    const iframe = document.getElementById('match-frame');
                    iframe.style.display = 'block';
                    iframe.src = `match_${matchId}/match_${matchId}_all_periods.html`;
                }
                document.addEventListener('DOMContentLoaded', function() {
                    const links = document.querySelectorAll('.match-link');
                    links.forEach(link => {
                        link.addEventListener('click', function(e) {
                            e.preventDefault();
                        });
                    });
                });
            </script>
        </body>
        </html>
        """
        
        # Create a unified index file for all teams
        # index_file = os.path.join(output_dir, f"match_index_all_teams.html")
    #     with open(index_file, 'w', encoding='utf-8') as f:
    #         f.write(index_html)
        
    #     print(f"\nUnified index page created at: {index_file}")
    # else:
    #     print("No match data was processed successfully. Index page was not created.")
    
    print("All match plots have been generated and saved!")

generate_match_plots(
    ("Argentina", "#89CFF0", arg_matches), 
    ("France", "blue", fr_matches),
    output_dir="visuals"
)




Processing Match ID: 3857264 (Poland)...
2025-06-07 00:04:07.033000
2025-06-07 00:00:19.698000
Available periods for Match ID 3857264: [1, 2]
Interactive plot with match table saved for Match ID 3857264 at visuals/match_3857264/match_3857264_all_periods.html

Processing Match ID: 3857289 (Mexico)...
2025-06-07 00:00:45.815000
2025-06-07 00:00:07.529000
Available periods for Match ID 3857289: [1, 2]
Interactive plot with match table saved for Match ID 3857289 at visuals/match_3857289/match_3857289_all_periods.html

Processing Match ID: 3857300 (Saudi Arabia)...
2025-06-07 00:00:11.377000
2025-06-07 00:00:41.083000
Available periods for Match ID 3857300: [1, 2]
Interactive plot with match table saved for Match ID 3857300 at visuals/match_3857300/match_3857300_all_periods.html

Processing Match ID: 3869151 (Australia)...
2025-06-07 00:00:09.615000
2025-06-07 00:00:16.562000
Available periods for Match ID 3869151: [1, 2]
Interactive plot with match table saved for Match ID 3869151 at visu

# Statistical Analysis

In [21]:
def get_cliffs_delta_explanation(cliffs_delta_value, ci_lower=None, ci_upper=None):
    """
    Get explanation for Cliff's Delta value with CI interpretation
    
    Args:
        cliffs_delta_value: The calculated Cliff's Delta value
        ci_lower: Lower bound of confidence interval
        ci_upper: Upper bound of confidence interval
        
    Returns:
        str: Explanation of what the value means
    """
    if pd.isna(cliffs_delta_value) or cliffs_delta_value == 'N/A':
        return "Not applicable"
    
    try:
        delta_float = float(cliffs_delta_value)
        
        # Basic interpretation
        if delta_float > 0:
            basic_interpretation = "Recovery is FASTER after events"
        elif delta_float < 0:
            basic_interpretation = "Recovery is SLOWER after events"
        else:
            basic_interpretation = "No difference in recovery times"
        
        # Add significance interpretation if CI is available
        if ci_lower is not None and ci_upper is not None:
            ci_lower_float = float(ci_lower) if not pd.isna(ci_lower) else None
            ci_upper_float = float(ci_upper) if not pd.isna(ci_upper) else None
            
            if ci_lower_float is not None and ci_upper_float is not None:
                if ci_lower_float > 0:
                    significance = " (Significantly faster)"
                elif ci_upper_float < 0:
                    significance = " (Significantly slower)"
                else:
                    significance = " (Not statistically significant)"
                
                return basic_interpretation + significance
        
        return basic_interpretation
        
    except:
        return "Not applicable"

def apply_benjamini_hochberg_correction(mannwhitney_results, alpha=0.05):
    """
    Apply Benjamini-Hochberg (FDR) correction to Mann-Whitney U test results
    
    Args:
        mannwhitney_results: Dictionary containing Mann-Whitney test results
        alpha: Significance level (default: 0.05)
    
    Returns:
        Updated results dictionary with corrected p-values and significance
    """
    from statsmodels.stats.multitest import multipletests
    import numpy as np
    
    # Extract p-values and test names
    p_values = []
    test_names = []
    
    for test_name, results in mannwhitney_results.items():
        if 'mw_p' in results and not pd.isna(results['mw_p']):
            p_values.append(results['mw_p'])
            test_names.append(test_name)
    
    if not p_values:
        print("No valid p-values found for correction.")
        return mannwhitney_results
    
    p_values = np.array(p_values)
    
    # Apply Benjamini-Hochberg correction using statsmodels
    rejected, corrected_p_values, alpha_sidak, alpha_bonf = multipletests(
        p_values, 
        alpha=alpha, 
        method='fdr_bh'  # Benjamini-Hochberg FDR correction
    )
    
    # Update results dictionary
    corrected_results = mannwhitney_results.copy()
    
    for i, test_name in enumerate(test_names):
        corrected_results[test_name]['corrected_p_value'] = corrected_p_values[i]
        corrected_results[test_name]['significant_corrected'] = rejected[i]
        corrected_results[test_name]['correction_method'] = "Benjamini-Hochberg (FDR)"
    
    # Print summary
    original_significant = sum(1 for p in p_values if p <= alpha)
    corrected_significant = sum(rejected)
    
    print(f"\n{'='*80}")
    print(f"BENJAMINI-HOCHBERG (FDR) CORRECTION APPLIED")
    print(f"{'='*80}")
    print(f"Total tests: {len(p_values)}")
    print(f"Significance level (α): {alpha}")
    print(f"Original significant tests: {original_significant}")
    print(f"Significant after BH correction: {corrected_significant}")
    print(f"Tests no longer significant: {original_significant - corrected_significant}")
    print(f"{'='*80}")
    
    return corrected_results
    
# def analyze_event_recovery_times(*team_dicts, output_dir="event_recovery_analysis", window_minutes=5, apply_multiple_correction=True, alpha=0.05):
#     """
#     Analyze recovery times around different event types in football matches.
    
#     Args:
#         *team_dicts: Variable number of tuples, each containing (team_name, matches_dict)
#         output_dir: Directory to save output CSVs (default: "event_recovery_analysis")
#         window_minutes: Time window in minutes before and after events to analyze (default: 5)
#         apply_multiple_correction: Whether to apply Benjamini-Hochberg correction (default: True)
#         alpha: Significance level for corrections (default: 0.05)
    
#     Returns:
#         Dictionary containing dataframes with recovery times for each team and event type
#     """

#     # Create output directory
#     os.makedirs(output_dir, exist_ok=True)
    
#     # Dictionary to store all dataframes
#     all_dataframes = {}
    
#     # Dictionary to store Shapiro-Wilk test results
#     normality_results = {}
    
#     # Dictionary to store Mann-Whitney test results
#     mannwhitney_results = {}
    
#     # Process all teams and their matches
#     processed_matches = set()
    
#     def cliffs_delta(group1, group2, n_bootstrap=1000, random_state=None):
#         """
#         Calculate Cliff's Delta effect size with bootstrap confidence intervals
        
#         Args:
#             group1: First group data (array-like)
#             group2: Second group data (array-like)
#             n_bootstrap: Number of bootstrap resamples (default: 1000)
#             random_state: Random seed for reproducibility
            
#         Returns:
#             delta: Cliff's Delta value
#             magnitude: Interpretation of effect size magnitude
#             ci_lower: Lower bound of bootstrap confidence interval
#             ci_upper: Upper bound of bootstrap confidence interval
#         """
#         import numpy as np
        
#         # Convert to numpy arrays
#         group1 = np.array(group1)
#         group2 = np.array(group2)
        
#         n1 = len(group1)
#         n2 = len(group2)
        
#         if n1 == 0 or n2 == 0:
#             return float('nan'), "Not applicable", float('nan'), float('nan')
        
#         # Set random seed if provided
#         if random_state is not None:
#             np.random.seed(random_state)
        
#         def calculate_cliffs_delta(g1, g2):
#             """Helper function to calculate Cliff's Delta for any two groups"""
#             dominance = 0
#             total_comparisons = len(g1) * len(g2)
            
#             for x in g1:
#                 for y in g2:
#                     if x > y:
#                         dominance += 1
#                     elif x < y:
#                         dominance -= 1
            
#             return dominance / total_comparisons
        
#         # Calculate original Cliff's Delta
#         delta = calculate_cliffs_delta(group1, group2)
        
#         # Bootstrap procedure
#         bootstrap_deltas = []
        
#         for i in range(n_bootstrap):
#             # Resample both groups with replacement
#             bootstrap_group1 = np.random.choice(group1, size=n1, replace=True)
#             bootstrap_group2 = np.random.choice(group2, size=n2, replace=True)
            
#             # Calculate Cliff's Delta for bootstrap sample
#             bootstrap_delta = calculate_cliffs_delta(bootstrap_group1, bootstrap_group2)
#             bootstrap_deltas.append(bootstrap_delta)
        
#         bootstrap_deltas = np.array(bootstrap_deltas)
        
#         # Calculate 95% confidence intervals using percentile method
#         ci_lower = np.percentile(bootstrap_deltas, 2.5)
#         ci_upper = np.percentile(bootstrap_deltas, 97.5)
        
#         # Ensure CI is within [-1, 1] bounds
#         ci_lower = max(-1, ci_lower)
#         ci_upper = min(1, ci_upper)
        
#         # Interpret magnitude according to Romano et al. (2006)
#         if abs(delta) < 0.147:
#             magnitude = "Negligible"
#         elif abs(delta) < 0.33:
#             magnitude = "Small"
#         elif abs(delta) < 0.474:
#             magnitude = "Medium"
#         else:
#             magnitude = "Large"
        
#         return delta, magnitude, ci_lower, ci_upper

        
#     for team_data in team_dicts:
#         # Unpack the team data (now expecting tuple of team_name and matches_dict)
#         team_name, matches_dict = team_data
        
#         # Create team-specific directory
#         team_output_dir = os.path.join(output_dir, team_name.replace(' ', '_').lower())
#         os.makedirs(team_output_dir, exist_ok=True)
        
#         # Create team-level subfolder
#         team_level_dir = os.path.join(team_output_dir, "team_level")
#         os.makedirs(team_level_dir, exist_ok=True)
        
#         # Create stats folder in team-level directory
#         team_stats_dir = os.path.join(team_level_dir, "stats")
#         os.makedirs(team_stats_dir, exist_ok=True)
        
#         print(f"\nProcessing matches for {team_name}...")
        
#         # Initialize dictionaries for this team
#         team_event_recoveries = {
#             'all_goals': {'times': [], 'timing': [], 'count': 0, 'team': []},
#             'substitutions': {'times': [], 'timing': [], 'count': 0, 'team': []},
#             'tactical_shifts': {'times': [], 'timing': [], 'count': 0, 'team': []},
#             'injuries': {'times': [], 'timing': [], 'count': 0, 'team': []},
#             'yellow_cards': {'times': [], 'timing': [], 'count': 0, 'team': []},
#             'red_cards': {'times': [], 'timing': [], 'count': 0, 'team': []}
#         }
        
#         # Team-specific dataframe dict
#         team_dfs = {}
        
#         # Team-specific normality results
#         team_normality_results = []
        
#         # Team-specific Mann-Whitney results
#         team_mannwhitney_results = []
        
#         # Process all matches for this team
#         for match_id, details in matches_dict.items():
#             # Skip if we've already processed this match
#             # if match_id in processed_matches:
#             #     print(f"Skipping already processed Match ID: {match_id}")
#             #     continue
                
#             # Mark this match as processed
#             processed_matches.add(match_id)
            
#             print(f"Processing match {match_id} ({details['team']})...")
            
#             # Create variable names based on team names
#             team1 = details['team'].replace(' ', '_').lower()  # opposing team
#             team2 = team_name.replace(' ', '_').lower()  # current team
            
#             # Store the opposing team's name for reference
#             opposing_team_name = details['team']
            
#             # Get the time-to-recover DataFrame
#             df_var_name = f"df_time_to_recover_{team2}_vs_{team1}"
#             if df_var_name not in globals():
#                 print(f"No time-to-recover data found for {df_var_name}")
#                 continue
                
#             time_to_recover_df = globals()[df_var_name]
            
#             # Ensure loss_time is datetime
#             time_to_recover_df['loss_time'] = pd.to_datetime(time_to_recover_df['loss_time'])
            
#             # Get events from the StatsBomb API
#             events = sb.events(match_id=match_id)
#             events['timestamp'] = pd.to_datetime(events['timestamp'], errors='coerce')
            
#             # Filter for periods 1-4
#             events = events[events['period'].isin([1, 2, 3, 4])]
            
#             if events.empty:
#                 print(f"No events found for periods 1-4 in Match {match_id}. Skipping...")
#                 continue
            
#             # ===================================================================================
#             # NEW CODE: Create match-specific directory and analysis structures
#             # ===================================================================================
            
#             # Create match-specific directory
#             match_output_dir = os.path.join(team_output_dir, f"match_{match_id}")
#             os.makedirs(match_output_dir, exist_ok=True)
            
#             # Create stats folder in match directory
#             match_stats_dir = os.path.join(match_output_dir, "stats")
#             os.makedirs(match_stats_dir, exist_ok=True)
            
#             # Initialize dictionaries for this match
#             match_event_recoveries = {
#                 'all_goals': {'times': [], 'timing': [], 'count': 0, 'team': []},
#                 'substitutions': {'times': [], 'timing': [], 'count': 0, 'team': []},
#                 'tactical_shifts': {'times': [], 'timing': [], 'count': 0, 'team': []},
#                 'injuries': {'times': [], 'timing': [], 'count': 0, 'team': []},
#                 'yellow_cards': {'times': [], 'timing': [], 'count': 0, 'team': []},
#                 'red_cards': {'times': [], 'timing': [], 'count': 0, 'team': []}
#             }
            
#             # Match-specific normality results
#             match_normality_results = []
            
#             # Match-specific Mann-Whitney results
#             match_mannwhitney_results = []
            
#             # Define event type filters
#             event_types = {
#                 'all_goals': events[((events['type'] == 'Shot') & (events['shot_outcome'] == 'Goal')) | 
#                                    (events['type'] == 'Own Goal For')].copy(),
#                 'substitutions': events[events['type'] == 'Substitution'].copy(),
#                 'tactical_shifts': events[events['type'] == 'Tactical Shift'].copy(),
#                 'injuries': events[events['type'] == 'Injury Stoppage'].copy(),
#                 'yellow_cards': events[(events['type'] == 'Foul Committed') & 
#                                       (events['foul_committed_card'] == 'Yellow Card')].copy(),
#                 'red_cards': events[(events['type'] == 'Foul Committed') & 
#                                    (events['foul_committed_card'] == 'Red Card')].copy()
#             }
            
#             # Process each type of event
#             for event_type, event_events in event_types.items():
#                 # Count the number of events of this type (team-specific)
#                 team_event_recoveries[event_type]['count'] += len(event_events)
#                 # Count the number of events of this type (match-specific)
#                 match_event_recoveries[event_type]['count'] += len(event_events)
                
#                 for _, event in event_events.iterrows():
#                     event_time = event['timestamp']
#                     event_period = event['period']
                    
#                     # Define time window around the event
#                     window_start = event_time - pd.Timedelta(minutes=window_minutes)
#                     window_end = event_time + pd.Timedelta(minutes=window_minutes)
                    
#                     # Get all ball loss and recovery events in the window
#                     window_events = time_to_recover_df[
#                         (time_to_recover_df['loss_time'] >= window_start) &
#                         (time_to_recover_df['loss_time'] <= window_end) &
#                         (time_to_recover_df['period'] == event_period) &
#                         (time_to_recover_df['time_to_recover'] >= 1)
#                     ].copy()
                    
#                     # Label events as "before event" or "after event"
#                     window_events['event_timing'] = window_events['loss_time'].apply(
#                         lambda x: "before event" if x < event_time else "after event"
#                     )
                    
#                     # Add to team-specific data with the timing label
#                     for _, row in window_events.iterrows():
#                         team_event_recoveries[event_type]['times'].append(row['time_to_recover'])
#                         team_event_recoveries[event_type]['timing'].append(row['event_timing'])
#                         team_event_recoveries[event_type]['team'].append(row['team'])
                        
#                         # Add to match-specific data with the timing label
#                         match_event_recoveries[event_type]['times'].append(row['time_to_recover'])
#                         match_event_recoveries[event_type]['timing'].append(row['event_timing'])
#                         match_event_recoveries[event_type]['team'].append(row['team'])
            
#             # Create and save match-specific dataframes for each event type
#             match_dfs = {}
#             for event_type, data in match_event_recoveries.items():
#                 if data['times']:
#                     # Create DataFrame for this event type in this match
#                     df_name = f"df_{team_name.replace(' ', '_').lower()}_match_{match_id}_{event_type}"
                    
#                     # Create DataFrame with team column
#                     df = pd.DataFrame({
#                         'time_to_recover': data['times'],
#                         'event_timing': data['timing'],
#                         'team': data['team']
#                     })
                    
#                     # Save the dataframe as a variable
#                     globals()[df_name] = df
                    
#                     # Add to match dataframes dictionary
#                     match_dfs[event_type] = df
                    
#                     # Add to all dataframes dictionary
#                     all_dataframes[df_name] = df
                    
#                     # Save to match-specific directory
#                     csv_path = f"{match_output_dir}/{event_type}_recovery_times.csv"
#                     df.to_csv(csv_path, index=False)
                    
#                     print(f"\n{team_name} - Match {match_id} - {event_type.replace('_', ' ').title()} data saved to {csv_path}")
#                     print(f"Number of {event_type.replace('_', ' ')} events: {data['count']}")
#                     print(f"Number of recovery times collected: {len(data['times'])}")
#                     print(f"DataFrame created as: {df_name}")
                    
#                     # Filter data to only include the specified team for statistical analysis
#                     team_data_df = df[df['team'] == team_name]
#                     team_recovery_times = team_data_df['time_to_recover'].tolist()
                    
#                     # Perform Shapiro-Wilk test if we have sufficient data for the team in this match
#                     if len(team_recovery_times) >= 8:  # Minimum sample size for meaningful normality test
#                         # Set up the normality test results dictionary for this dataframe
#                         normality_results[df_name] = {}
                        
#                         # Shapiro-Wilk test (best for sample sizes < 5000)
#                         if len(team_recovery_times) < 5000:  # Shapiro-Wilk works best for smaller sample sizes
#                             shapiro_stat, shapiro_p = stats.shapiro(team_recovery_times)
#                             normality_results[df_name]['shapiro_stat'] = shapiro_stat
#                             normality_results[df_name]['shapiro_p'] = shapiro_p
#                             normality_results[df_name]['shapiro_normal'] = shapiro_p > 0.05
                            
#                             # Save sample size for reporting
#                             normality_results[df_name]['sample_size'] = len(team_recovery_times)
                            
#                             # Create an entry for match-specific results
#                             match_result = {
#                                 'Event Type': event_type.replace('_', ' ').title(),
#                                 'Sample Size': len(team_recovery_times),
#                                 'Shapiro-Wilk Statistic': f"{shapiro_stat:.4f}",  # 4 decimal places
#                                 'p-value': f"{shapiro_p:.18f}",  # 18 decimal places
#                                 'Normal Distribution': "Yes" if shapiro_p > 0.05 else "No"
#                             }
#                             match_normality_results.append(match_result)
                            
#                             print("\n" + "="*80)
#                             print(f"SHAPIRO-WILK NORMALITY TEST: {team_name} - Match {match_id} - {event_type.replace('_', ' ').title()}")
#                             print("-"*80)
#                             print(f"Sample size:      {len(team_recovery_times)}")
#                             print(f"Test statistic:   {shapiro_stat:.4f}")  # 4 decimal places
#                             print(f"p-value:          {shapiro_p:.18f}")  # 18 decimal places
#                             print("-"*80)
#                             if shapiro_p > 0.05:
#                                 print("RESULT:           NORMALLY DISTRIBUTED ✓")
#                                 print("                  (p > 0.05, fail to reject null hypothesis of normality)")
#                             else:
#                                 print("RESULT:           NOT NORMALLY DISTRIBUTED ✗")
#                                 print("                  (p ≤ 0.05, reject null hypothesis of normality)")
#                             print("="*80)
#                         else:
#                             print("\n" + "="*80)
#                             print(f"SHAPIRO-WILK TEST SKIPPED: {team_name} - Match {match_id} - {event_type.replace('_', ' ').title()}")
#                             print("-"*80)
#                             print(f"Sample size too large: {len(team_recovery_times)} > 5000 (maximum for Shapiro-Wilk)")
#                             print("Consider using an alternative test for larger samples.")
#                             print("="*80)
#                     else:
#                         print("\n" + "="*80)
#                         print(f"SHAPIRO-WILK TEST SKIPPED: {team_name} - Match {match_id} - {event_type.replace('_', ' ').title()}")
#                         print("-"*80)
#                         print(f"Insufficient data: {len(team_recovery_times)} < 8 (minimum required samples)")
#                         print("="*80)
                    
#                     # Cliff's Delta and Mann-Whitney U test calculations for match-specific analysis
                    
#                     # Split data into before and after events - only for the specified team
#                     before_times = team_data_df[team_data_df['event_timing'] == 'before event']['time_to_recover'].tolist()
#                     after_times = team_data_df[team_data_df['event_timing'] == 'after event']['time_to_recover'].tolist()
                    
#                     # Calculate Cliff's Delta if we have any data in both groups
#                     cliff_d = float('nan')
#                     cliff_magnitude = "Not applicable"
                    
#                     if len(before_times) > 0 and len(after_times) > 0:
#                         cliff_d, cliff_magnitude, ci_lower, ci_upper = cliffs_delta(before_times, after_times, random_state=8)
                        
#                         print("\n" + "="*80)
#                         print(f"CLIFF'S DELTA EFFECT SIZE: {team_name} - {event_type.replace('_', ' ').title()}")
#                         print(f"Comparing recovery times BEFORE vs. AFTER events")
#                         print("-"*80)
#                         print(f"Before event sample size: {len(before_times)}")
#                         print(f"After event sample size:  {len(after_times)}")
#                         print(f"Cliff's Delta:           {cliff_d:.4f}")
#                         print(f"95% Bootstrap CI:        [{ci_lower:.4f}, {ci_upper:.4f}]")
#                         print(f"Effect Size:             {cliff_magnitude}")

#                         # Interpretation
#                         if ci_lower > 0:
#                             print(f"Interpretation:          Significantly FASTER recovery after events")
#                         elif ci_upper < 0:
#                             print(f"Interpretation:          Significantly SLOWER recovery after events")
#                         else:
#                             print(f"Interpretation:          No significant difference (CI includes 0)")
#                         print("="*80)
                    
#                     # Perform Mann-Whitney U test if there's enough data in both groups
#                     if len(before_times) >= 5 and len(after_times) >= 5:
#                         mw_stat, mw_p = stats.mannwhitneyu(before_times, after_times, alternative='two-sided')
                        
#                         # Create an entry for match-specific results
#                         mw_result = {
#                             'Event Type': event_type.replace('_', ' ').title(),
#                             'Before Event Sample Size': len(before_times),
#                             'After Event Sample Size': len(after_times),
#                             'Mann-Whitney U Statistic': f"{mw_stat:.4f}",
#                             'p-value': f"{mw_p:.18f}",
#                             'Significant Difference': "Yes" if mw_p <= 0.05 else "No",
#                             'Cliffs Delta': f"{cliff_d:.4f}",
#                             'Cliffs Delta CI Lower': f"{ci_lower:.4f}",
#                             'Cliffs Delta CI Upper': f"{ci_upper:.4f}",
#                             'Effect Size': cliff_magnitude,
#                             'Cliffs Delta Explanation': get_cliffs_delta_explanation(cliff_d, ci_lower, ci_upper)
#                         }
#                         match_mannwhitney_results.append(mw_result)

#                         # Store Mann-Whitney results in the main dictionary
#                         mannwhitney_results[df_name] = {
#                             'mw_stat': mw_stat,
#                             'mw_p': mw_p,
#                             'before_sample_size': len(before_times),
#                             'after_sample_size': len(after_times),
#                             'significant': mw_p <= 0.05,
#                             'cliffs_delta': cliff_d,
#                             'ci_lower': ci_lower,
#                             'ci_upper': ci_upper,
#                             'effect_size': cliff_magnitude
#                         }
                        
#                         print("\n" + "="*80)
#                         print(f"MANN-WHITNEY U TEST: {team_name} - Match {match_id} - {event_type.replace('_', ' ').title()}")
#                         print(f"Comparing recovery times BEFORE vs. AFTER events")
#                         print("-"*80)
#                         print(f"Before event sample size: {len(before_times)}")
#                         print(f"After event sample size:  {len(after_times)}")
#                         print(f"Test statistic:          {mw_stat:.4f}")  # 4 decimal places
#                         print(f"p-value:                 {mw_p:.18f}")  # 18 decimal places
#                         print("-"*80)
#                         if mw_p <= 0.05:
#                             print("RESULT:                  SIGNIFICANT DIFFERENCE ✓")
#                             print("                         (p ≤ 0.05, reject null hypothesis of equal distributions)")
#                         else:
#                             print("RESULT:                  NO SIGNIFICANT DIFFERENCE ✗")
#                             print("                         (p > 0.05, fail to reject null hypothesis of equal distributions)")
#                         print("="*80)
#                     else:
#                         print("\n" + "="*80)
#                         print(f"MANN-WHITNEY U TEST SKIPPED: {team_name} - Match {match_id} - {event_type.replace('_', ' ').title()}")
#                         print("-"*80)
#                         print(f"Insufficient data for before events: {len(before_times)} < 5 OR after events: {len(after_times)} < 5")
#                         print("Minimum 5 samples required in each group for Mann-Whitney U test.")
#                         print("="*80)
            
#             # Save match-specific normality test results
#             if match_normality_results:
#                 match_results_df = pd.DataFrame(match_normality_results)
#                 match_normality_csv_path = f"{match_stats_dir}/shapiro_wilk_test_results.csv"
#                 match_results_df.to_csv(match_normality_csv_path, index=False)
#                 print(f"\n{team_name} - Match {match_id} - Shapiro-Wilk test results saved to {match_normality_csv_path}")
            
#             # Apply Benjamini-Hochberg correction to match-specific Mann-Whitney results
#             if match_mannwhitney_results and apply_multiple_correction:
#                 print(f"\n{'='*60}")
#                 print(f"APPLYING BH CORRECTION TO {team_name.upper()} - MATCH {match_id} TESTS")
#                 print(f"{'='*60}")
                
#                 # Extract p-values from match results for correction
#                 match_p_values = []
#                 for result in match_mannwhitney_results:
#                     try:
#                         p_val = float(result['p-value'])
#                         match_p_values.append(p_val)
#                     except (ValueError, TypeError):
#                         match_p_values.append(float('nan'))
                
#                 # Apply BH correction if we have valid p-values
#                 valid_p_values = [p for p in match_p_values if not pd.isna(p)]
#                 if valid_p_values:
#                     from statsmodels.stats.multitest import multipletests
#                     import numpy as np
                    
#                     # Apply correction
#                     rejected, corrected_p_values, _, _ = multipletests(
#                         valid_p_values, 
#                         alpha=alpha, 
#                         method='fdr_bh'
#                     )
                    
#                     # Update match results with corrected values
#                     corrected_idx = 0
#                     for i, result in enumerate(match_mannwhitney_results):
#                         if not pd.isna(match_p_values[i]):
#                             result['BH Corrected p-value'] = f"{corrected_p_values[corrected_idx]:.6f}"
#                             result['Significant After BH Correction'] = "Yes" if rejected[corrected_idx] else "No"
#                             result['Correction Method'] = "Benjamini-Hochberg (FDR) - Match Level"
#                             corrected_idx += 1
#                         else:
#                             result['BH Corrected p-value'] = "N/A"
#                             result['Significant After BH Correction'] = "N/A"
#                             result['Correction Method'] = "N/A"
                    
#                     # Print match-level correction summary
#                     original_significant = sum(1 for p in valid_p_values if p <= alpha)
#                     corrected_significant = sum(rejected)
                    
#                     print(f"Match: {team_name} vs {opposing_team_name} (ID: {match_id})")
#                     print(f"Total tests: {len(valid_p_values)}")
#                     print(f"Originally significant: {original_significant}")
#                     print(f"Significant after BH correction: {corrected_significant}")
#                     print(f"Tests no longer significant: {original_significant - corrected_significant}")
#                     print(f"{'='*60}")
            
#             # Save match-specific Mann-Whitney test results (with BH correction if applied)
#             if match_mannwhitney_results:
#                 match_mw_df = pd.DataFrame(match_mannwhitney_results)
#                 match_mw_csv_path = f"{match_stats_dir}/mann_whitney_test_results.csv"
#                 match_mw_df.to_csv(match_mw_csv_path, index=False)
#                 print(f"\n{team_name} - Match {match_id} - Mann-Whitney U test results saved to {match_mw_csv_path}")
            
#             # Create and save the combined match dataframe
#             match_combined_df_name = f"df_{team_name.replace(' ', '_').lower()}_match_{match_id}_all_events"
#             match_combined_df = pd.DataFrame()
            
#             for event_type, data in match_event_recoveries.items():
#                 if data['times']:
#                     # Create a comprehensive dataframe with team data
#                     temp_df = pd.DataFrame({
#                         'event_type': [event_type] * len(data['times']),
#                         'time_to_recover': data['times'],
#                         'event_timing': data['timing'],
#                         'team': data['team']
#                     })
#                     match_combined_df = pd.concat([match_combined_df, temp_df], ignore_index=True)
            
#             if not match_combined_df.empty:
#                 # Save the combined dataframe as a variable
#                 globals()[match_combined_df_name] = match_combined_df
                
#                 # Add to all dataframes dictionary
#                 all_dataframes[match_combined_df_name] = match_combined_df
                
#                 # Save to match-specific directory
#                 match_summary_path = f"{match_output_dir}/all_events_summary.csv"
#                 match_combined_df.to_csv(match_summary_path, index=False)
#                 print(f"\n{team_name} - Match {match_id} - Complete event summary saved to {match_summary_path}")
#                 print(f"DataFrame created as: {match_combined_df_name}")
                
#                 # Skip tests on combined data as requested
#                 print("\n" + "="*80)
#                 print(f"SHAPIRO-WILK TEST SKIPPED: {team_name} - Match {match_id} - ALL EVENTS COMBINED")
#                 print("-"*80)
#                 print(f"Tests on combined data skipped as requested")
#                 print("="*80)
            
#             # ===================================================================================
#             # End of match-specific analysis
#             # ===================================================================================
    
#         # Create and save team-specific dataframes for each event type
#         for event_type, data in team_event_recoveries.items():
#             if data['times']:
#                 # Create DataFrame for this event type
#                 df_name = f"df_{team_name.replace(' ', '_').lower()}_{event_type}"
                
#                 # Create DataFrame with team column
#                 df = pd.DataFrame({
#                     'time_to_recover': data['times'],
#                     'event_timing': data['timing'],
#                     'team': data['team']
#                 })
                
#                 # Save the dataframe as a variable
#                 globals()[df_name] = df
                
#                 # Add to team dataframes dictionary
#                 team_dfs[event_type] = df
                
#                 # Add to all dataframes dictionary
#                 all_dataframes[df_name] = df
                
#                 # Save to team-level directory
#                 csv_path = f"{team_level_dir}/{event_type}_recovery_times.csv"
#                 df.to_csv(csv_path, index=False)
#                 print(f"\n{team_name} - {event_type.replace('_', ' ').title()} data saved to {csv_path}")
#                 print(f"Number of {event_type.replace('_', ' ')} events: {data['count']}")
#                 print(f"Number of recovery times collected: {len(data['times'])}")
#                 print(f"DataFrame created as: {df_name}")
                
#                 # Filter data to only include the specified team for statistical analysis
#                 team_data_df = df[df['team'] == team_name]
#                 team_recovery_times = team_data_df['time_to_recover'].tolist()
                
#                 # Perform Shapiro-Wilk test if we have sufficient data for the team
#                 if len(team_recovery_times) >= 8:  # Minimum sample size for meaningful normality test
#                     # Set up the normality test results dictionary for this dataframe
#                     normality_results[df_name] = {}
                    
#                     # Shapiro-Wilk test (best for sample sizes < 5000)
#                     if len(team_recovery_times) < 5000:  # Shapiro-Wilk works best for smaller sample sizes
#                         shapiro_stat, shapiro_p = stats.shapiro(team_recovery_times)
#                         normality_results[df_name]['shapiro_stat'] = shapiro_stat
#                         normality_results[df_name]['shapiro_p'] = shapiro_p
#                         normality_results[df_name]['shapiro_normal'] = shapiro_p > 0.05
                        
#                         # Save sample size for reporting
#                         normality_results[df_name]['sample_size'] = len(team_recovery_times)
                        
#                         # Create an entry for team-specific results
#                         team_result = {
#                             'Event Type': event_type.replace('_', ' ').title(),
#                             'Sample Size': len(team_recovery_times),
#                             'Shapiro-Wilk Statistic': f"{shapiro_stat:.4f}",  # 4 decimal places
#                             'p-value': f"{shapiro_p:.18f}",  # 18 decimal places
#                             'Normal Distribution': "Yes" if shapiro_p > 0.05 else "No"
#                         }
#                         team_normality_results.append(team_result)
                        
#                         print("\n" + "="*80)
#                         print(f"SHAPIRO-WILK NORMALITY TEST: {team_name} - {event_type.replace('_', ' ').title()}")
#                         print("-"*80)
#                         print(f"Sample size:      {len(team_recovery_times)}")
#                         print(f"Test statistic:   {shapiro_stat:.4f}")  # 4 decimal places
#                         print(f"p-value:          {shapiro_p:.18f}")  # 18 decimal places
#                         print("-"*80)
#                         if shapiro_p > 0.05:
#                             print("RESULT:           NORMALLY DISTRIBUTED ✓")
#                             print("                  (p > 0.05, fail to reject null hypothesis of normality)")
#                         else:
#                             print("RESULT:           NOT NORMALLY DISTRIBUTED ✗")
#                             print("                  (p ≤ 0.05, reject null hypothesis of normality)")
#                         print("="*80)
#                     else:
#                         print("\n" + "="*80)
#                         print(f"SHAPIRO-WILK TEST SKIPPED: {team_name} - {event_type.replace('_', ' ').title()}")
#                         print("-"*80)
#                         print(f"Sample size too large: {len(team_recovery_times)} > 5000 (maximum for Shapiro-Wilk)")
#                         print("Consider using an alternative test for larger samples.")
#                         print("="*80)
#                 else:
#                     print("\n" + "="*80)
#                     print(f"SHAPIRO-WILK TEST SKIPPED: {team_name} - {event_type.replace('_', ' ').title()}")
#                     print("-"*80)
#                     print(f"Insufficient data: {len(team_recovery_times)} < 8 (minimum required samples)")
#                     print("="*80)
                
#                 # Cliff's Delta and Mann-Whitney U test calculations
                
#                 # Split data into before and after events - only for the specified team
#                 before_times = team_data_df[team_data_df['event_timing'] == 'before event']['time_to_recover'].tolist()
#                 after_times = team_data_df[team_data_df['event_timing'] == 'after event']['time_to_recover'].tolist()
                
#                 # Calculate Cliff's Delta if we have any data in both groups
#                 cliff_d = float('nan')
#                 cliff_magnitude = "Not applicable"
                
#                 if len(before_times) > 0 and len(after_times) > 0:
#                     cliff_d, cliff_magnitude, ci_lower, ci_upper = cliffs_delta(before_times, after_times, random_state=8)
                    
#                     print("\n" + "="*80)
#                     print(f"CLIFF'S DELTA EFFECT SIZE: {team_name} - {event_type.replace('_', ' ').title()}")
#                     print(f"Comparing recovery times BEFORE vs. AFTER events")
#                     print("-"*80)
#                     print(f"Before event sample size: {len(before_times)}")
#                     print(f"After event sample size:  {len(after_times)}")
#                     print(f"Cliff's Delta:           {cliff_d:.4f}")
#                     print(f"95% Bootstrap CI:        [{ci_lower:.4f}, {ci_upper:.4f}]")
#                     print(f"Effect Size:             {cliff_magnitude}")

#                     # Interpretation
#                     if ci_lower > 0:
#                         print(f"Interpretation:          Significantly FASTER recovery after events")
#                     elif ci_upper < 0:
#                         print(f"Interpretation:          Significantly SLOWER recovery after events")
#                     else:
#                         print(f"Interpretation:          No significant difference (CI includes 0)")
#                     print("="*80)
                
#                 # Perform Mann-Whitney U test if there's enough data in both groups
#                 if len(before_times) >= 5 and len(after_times) >= 5:
#                     mw_stat, mw_p = stats.mannwhitneyu(before_times, after_times, alternative='two-sided')
                    
#                     # Create an entry for team-specific results
#                     mw_result = {
#                         'Event Type': event_type.replace('_', ' ').title(),
#                         'Before Event Sample Size': len(before_times),
#                         'After Event Sample Size': len(after_times),
#                         'Mann-Whitney U Statistic': f"{mw_stat:.4f}",
#                         'p-value': f"{mw_p:.18f}",
#                         'Significant Difference': "Yes" if mw_p <= 0.05 else "No",
#                         'Cliffs Delta': f"{cliff_d:.4f}",
#                         'Cliffs Delta CI Lower': f"{ci_lower:.4f}",
#                         'Cliffs Delta CI Upper': f"{ci_upper:.4f}",
#                         'Effect Size': cliff_magnitude,
#                         'Cliffs Delta Explanation': get_cliffs_delta_explanation(cliff_d, ci_lower, ci_upper)
#                     }
#                     team_mannwhitney_results.append(mw_result)

#                     # Store Mann-Whitney results in the main dictionary
#                     mannwhitney_results[df_name] = {
#                         'mw_stat': mw_stat,
#                         'mw_p': mw_p,
#                         'before_sample_size': len(before_times),
#                         'after_sample_size': len(after_times),
#                         'significant': mw_p <= 0.05,
#                         'cliffs_delta': cliff_d,
#                         'ci_lower': ci_lower,
#                         'ci_upper': ci_upper,
#                         'effect_size': cliff_magnitude
#                     }
                    
#                     print("\n" + "="*80)
#                     print(f"MANN-WHITNEY U TEST: {team_name} - {event_type.replace('_', ' ').title()}")
#                     print(f"Comparing recovery times BEFORE vs. AFTER events")
#                     print("-"*80)
#                     print(f"Before event sample size: {len(before_times)}")
#                     print(f"After event sample size:  {len(after_times)}")
#                     print(f"Test statistic:          {mw_stat:.4f}")  # 4 decimal places
#                     print(f"p-value:                 {mw_p:.18f}")  # 18 decimal places
#                     print("-"*80)
#                     if mw_p <= 0.05:
#                         print("RESULT:                  SIGNIFICANT DIFFERENCE ✓")
#                         print("                         (p ≤ 0.05, reject null hypothesis of equal distributions)")
#                     else:
#                         print("RESULT:                  NO SIGNIFICANT DIFFERENCE ✗")
#                         print("                         (p > 0.05, fail to reject null hypothesis of equal distributions)")
#                     print("="*80)
#                 else:
#                     print("\n" + "="*80)
#                     print(f"MANN-WHITNEY U TEST SKIPPED: {team_name} - {event_type.replace('_', ' ').title()}")
#                     print("-"*80)
#                     print(f"Insufficient data for before events: {len(before_times)} < 5 OR after events: {len(after_times)} < 5")
#                     print("Minimum 5 samples required in each group for Mann-Whitney U test.")
#                     print("="*80)
        
#         # Save team-specific normality test results
#         if team_normality_results:
#             team_results_df = pd.DataFrame(team_normality_results)
#             team_normality_csv_path = f"{team_stats_dir}/shapiro_wilk_test_results.csv"
#             team_results_df.to_csv(team_normality_csv_path, index=False)
#             print(f"\n{team_name} - Shapiro-Wilk test results saved to {team_normality_csv_path}")
        
#         # Apply Benjamini-Hochberg correction to team-specific Mann-Whitney results
#         if team_mannwhitney_results and apply_multiple_correction:
#             print(f"\n{'='*80}")
#             print(f"APPLYING BENJAMINI-HOCHBERG CORRECTION TO {team_name.upper()} TEAM-LEVEL TESTS")
#             print(f"{'='*80}")
            
#             # Extract p-values from team results for correction
#             team_p_values = []
#             for result in team_mannwhitney_results:
#                 try:
#                     p_val = float(result['p-value'])
#                     team_p_values.append(p_val)
#                 except (ValueError, TypeError):
#                     team_p_values.append(float('nan'))
            
#             # Apply BH correction if we have valid p-values
#             valid_p_values = [p for p in team_p_values if not pd.isna(p)]
#             if valid_p_values:
#                 from statsmodels.stats.multitest import multipletests
#                 import numpy as np
                
#                 # Apply correction
#                 rejected, corrected_p_values, _, _ = multipletests(
#                     valid_p_values, 
#                     alpha=alpha, 
#                     method='fdr_bh'
#                 )
                
#                 # Update team results with corrected values
#                 corrected_idx = 0
#                 for i, result in enumerate(team_mannwhitney_results):
#                     if not pd.isna(team_p_values[i]):
#                         result['BH Corrected p-value'] = f"{corrected_p_values[corrected_idx]:.6f}"
#                         result['Significant After BH Correction'] = "Yes" if rejected[corrected_idx] else "No"
#                         result['Correction Method'] = "Benjamini-Hochberg (FDR) - Team Level"
#                         corrected_idx += 1
#                     else:
#                         result['BH Corrected p-value'] = "N/A"
#                         result['Significant After BH Correction'] = "N/A"
#                         result['Correction Method'] = "N/A"
                
#                 # Print team-level correction summary
#                 original_significant = sum(1 for p in valid_p_values if p <= alpha)
#                 corrected_significant = sum(rejected)
                
#                 print(f"Team: {team_name}")
#                 print(f"Total tests: {len(valid_p_values)}")
#                 print(f"Originally significant: {original_significant}")
#                 print(f"Significant after BH correction: {corrected_significant}")
#                 print(f"Tests no longer significant: {original_significant - corrected_significant}")
#                 print(f"{'='*80}")
        
#         # Save team-specific Mann-Whitney test results (with BH correction if applied)
#         if team_mannwhitney_results:
#             team_mw_df = pd.DataFrame(team_mannwhitney_results)
#             team_mw_csv_path = f"{team_stats_dir}/mann_whitney_test_results.csv"
#             team_mw_df.to_csv(team_mw_csv_path, index=False)
#             print(f"\n{team_name} - Mann-Whitney U test results saved to {team_mw_csv_path}")
        
#         # Create and save the combined team dataframe
#         team_combined_df_name = f"df_{team_name.replace(' ', '_').lower()}_all_events"
#         team_combined_df = pd.DataFrame()
        
#         for event_type, data in team_event_recoveries.items():
#             if data['times']:
#                 # Create a comprehensive dataframe with team data
#                 temp_df = pd.DataFrame({
#                     'event_type': [event_type] * len(data['times']),
#                     'time_to_recover': data['times'],
#                     'event_timing': data['timing'],
#                     'team': data['team']
#                 })
#                 team_combined_df = pd.concat([team_combined_df, temp_df], ignore_index=True)
        
#         if not team_combined_df.empty:
#             # Save the combined dataframe as a variable
#             globals()[team_combined_df_name] = team_combined_df
            
#             # Add to all dataframes dictionary
#             all_dataframes[team_combined_df_name] = team_combined_df
            
#             # Save to team-level directory
#             team_summary_path = f"{team_level_dir}/all_events_summary.csv"
#             team_combined_df.to_csv(team_summary_path, index=False)
#             print(f"\n{team_name} - Complete event summary saved to {team_summary_path}")
#             print(f"DataFrame created as: {team_combined_df_name}")
            
#             # Skip tests on combined data as requested
#             print("\n" + "="*80)
#             print(f"SHAPIRO-WILK TEST SKIPPED: {team_name} - ALL EVENTS COMBINED")
#             print("-"*80)
#             print(f"Tests on combined data skipped as requested")
#             print("="*80)
    
#     # ===========================================================================================
#     # APPLY BENJAMINI-HOCHBERG CORRECTION TO ALL MANN-WHITNEY TESTS
#     # ===========================================================================================
    
#     if apply_multiple_correction and mannwhitney_results:
#         print("\n" + "="*80)
#         print("APPLYING BENJAMINI-HOCHBERG (FDR) CORRECTION")
#         print("="*80)
        
#         # Apply correction
#         mannwhitney_results = apply_benjamini_hochberg_correction(mannwhitney_results, alpha=alpha)
    
#     # Save the normality test results to a CSV with clear formatting
#     if normality_results:
#         # Convert dictionary to DataFrame with more readable format
#         results_data = []
#         for df_name, results in normality_results.items():
#             team_name = df_name.split('_')[1] if len(df_name.split('_')) > 1 else "Unknown"
            
#             # Check if this is a match-specific entry
#             if "match_" in df_name:
#                 # Extract match ID
#                 match_id = df_name.split('_match_')[1].split('_')[0]
#                 event_type = '_'.join(df_name.split(f'_match_{match_id}_')[1:]) if f'_match_{match_id}_' in df_name else "Unknown"

                
#                 # Skip combined data entries
#                 if event_type == "all_events":
#                     continue
                    
#                 results_data.append({
#                     'Team': team_name.replace('_', ' ').title(),
#                     'Match ID': match_id,
#                     'Event Type': event_type.replace('_', ' ').title(),
#                     'Sample Size': results.get('sample_size', "N/A"),
#                     'Shapiro-Wilk Statistic': f"{results.get('shapiro_stat', 'N/A'):.4f}" if isinstance(results.get('shapiro_stat'), (float, int)) else "N/A",  # 4 decimal places
#                     'p-value': f"{results.get('shapiro_p', 'N/A'):.18f}" if isinstance(results.get('shapiro_p'), (float, int)) else "N/A",  # 18 decimal places
#                     'Normal Distribution': "Yes" if results.get('shapiro_normal', False) else "No"
#                 })
#             else:
#                 # Regular team entry
#                 event_type = '_'.join(df_name.split('_')[2:]) if len(df_name.split('_')) > 2 else "Unknown"
                
#                 # Skip combined data entries
#                 if event_type == "all_events":
#                     continue
                    
#                 results_data.append({
#                     'Team': team_name.replace('_', ' ').title(),
#                     'Match ID': "All Matches",
#                     'Event Type': event_type.replace('_', ' ').title(),
#                     'Sample Size': results.get('sample_size', "N/A"),
#                     'Shapiro-Wilk Statistic': f"{results.get('shapiro_stat', 'N/A'):.4f}" if isinstance(results.get('shapiro_stat'), (float, int)) else "N/A",  # 4 decimal places
#                     'p-value': f"{results.get('shapiro_p', 'N/A'):.18f}" if isinstance(results.get('shapiro_p'), (float, int)) else "N/A",  # 18 decimal places
#                     'Normal Distribution': "Yes" if results.get('shapiro_normal', False) else "No"
#                 })
        
#         # Create DataFrame and save
#         results_df = pd.DataFrame(results_data)
#         normality_csv_path = f"{output_dir}/shapiro_wilk_test_results.csv"
#         results_df.to_csv(normality_csv_path, index=False)
        
#         print("\n" + "="*80)
#         print("SHAPIRO-WILK TEST SUMMARY")
#         print("-"*80)
#         print(f"Total datasets tested: {len(results_data)}")
#         print(f"Normally distributed datasets: {sum(1 for r in results_data if r['Normal Distribution'] == 'Yes')}")
#         print(f"Non-normally distributed datasets: {sum(1 for r in results_data if r['Normal Distribution'] == 'No')}")
#         print(f"Results saved to: {normality_csv_path}")
#         print("="*80)
    
#     # Save the Mann-Whitney test results to a CSV with clear formatting (including corrected results)
#     if mannwhitney_results:
#         # Convert dictionary to DataFrame with more readable format
#         mw_results_data = []
#         for df_name, results in mannwhitney_results.items():
#             team_name = df_name.split('_')[1] if len(df_name.split('_')) > 1 else "Unknown"
            
#             # Check if this is a match-specific entry
#             if "match_" in df_name:
#                 # Extract match ID
#                 match_id = df_name.split('_match_')[1].split('_')[0]
#                 event_type = '_'.join(df_name.split(f'_match_{match_id}_')[1:]) if f'_match_{match_id}_' in df_name else "Unknown"

                
#                 # Skip combined data entries
#                 if event_type == "all_events":
#                     continue
                    
#                 mw_results_data.append({
#                     'Team': team_name.replace('_', ' ').title(),
#                     'Match ID': match_id,
#                     'Event Type': event_type.replace('_', ' ').title(),
#                     'Before Event Sample Size': results.get('before_sample_size', "N/A"),
#                     'After Event Sample Size': results.get('after_sample_size', "N/A"),
#                     'Mann-Whitney U Statistic': f"{results.get('mw_stat', 'N/A'):.4f}" if isinstance(results.get('mw_stat'), (float, int)) else "N/A",
#                     'Original p-value': f"{results.get('mw_p', 'N/A'):.6f}" if isinstance(results.get('mw_p'), (float, int)) else "N/A",
#                     'BH Corrected p-value': f"{results.get('corrected_p_value', 'N/A'):.6f}" if isinstance(results.get('corrected_p_value'), (float, int)) else "N/A",
#                     'Original Significant': "Yes" if results.get('significant', False) else "No",
#                     'Significant After BH Correction': "Yes" if results.get('significant_corrected', False) else "No",
#                     'Correction Method': results.get('correction_method', "N/A"),
#                     'Cliffs Delta': f"{results.get('cliffs_delta', 'N/A'):.4f}" if isinstance(results.get('cliffs_delta'), (float, int)) else "N/A",
#                     'Cliffs Delta CI Lower': f"{results.get('ci_lower', 'N/A'):.4f}" if isinstance(results.get('ci_lower'), (float, int)) else "N/A",
#                     'Cliffs Delta CI Upper': f"{results.get('ci_upper', 'N/A'):.4f}" if isinstance(results.get('ci_upper'), (float, int)) else "N/A",
#                     'Effect Size': results.get('effect_size', "N/A"),
#                     'Cliffs Delta Explanation': get_cliffs_delta_explanation(results.get('cliffs_delta', 'N/A'), results.get('ci_lower', 'N/A'), results.get('ci_upper', 'N/A'))
#                 })
#             else:
#                 # Regular team entry
#                 event_type = '_'.join(df_name.split('_')[2:]) if len(df_name.split('_')) > 2 else "Unknown"
                
#                 # Skip combined data entries
#                 if event_type == "all_events":
#                     continue
                    
#                 mw_results_data.append({
#                     'Team': team_name.replace('_', ' ').title(),
#                     'Match ID': "All Matches",
#                     'Event Type': event_type.replace('_', ' ').title(),
#                     'Before Event Sample Size': results.get('before_sample_size', "N/A"),
#                     'After Event Sample Size': results.get('after_sample_size', "N/A"),
#                     'Mann-Whitney U Statistic': f"{results.get('mw_stat', 'N/A'):.4f}" if isinstance(results.get('mw_stat'), (float, int)) else "N/A",
#                     'Original p-value': f"{results.get('mw_p', 'N/A'):.6f}" if isinstance(results.get('mw_p'), (float, int)) else "N/A",
#                     'BH Corrected p-value': f"{results.get('corrected_p_value', 'N/A'):.6f}" if isinstance(results.get('corrected_p_value'), (float, int)) else "N/A",
#                     'Original Significant': "Yes" if results.get('significant', False) else "No",
#                     'Significant After BH Correction': "Yes" if results.get('significant_corrected', False) else "No",
#                     'Correction Method': results.get('correction_method', "N/A"),
#                     'Cliffs Delta': f"{results.get('cliffs_delta', 'N/A'):.4f}" if isinstance(results.get('cliffs_delta'), (float, int)) else "N/A",
#                     'Cliffs Delta CI Lower': f"{results.get('ci_lower', 'N/A'):.4f}" if isinstance(results.get('ci_lower'), (float, int)) else "N/A",
#                     'Cliffs Delta CI Upper': f"{results.get('ci_upper', 'N/A'):.4f}" if isinstance(results.get('ci_upper'), (float, int)) else "N/A",
#                     'Effect Size': results.get('effect_size', "N/A"),
#                     'Cliffs Delta Explanation': get_cliffs_delta_explanation(results.get('cliffs_delta', 'N/A'), results.get('ci_lower', 'N/A'), results.get('ci_upper', 'N/A'))
#                 })
        
#         # Create DataFrame and save
#         mw_results_df = pd.DataFrame(mw_results_data)
#         mw_csv_path = f"{output_dir}/mann_whitney_test_results_with_BH_correction.csv"
#         mw_results_df.to_csv(mw_csv_path, index=False)
        
#         # Count results for summary
#         original_significant = sum(1 for r in mw_results_data if r['Original Significant'] == 'Yes')
#         corrected_significant = sum(1 for r in mw_results_data if r['Significant After BH Correction'] == 'Yes')
        
#         print("\n" + "="*80)
#         print("MANN-WHITNEY U TEST SUMMARY (WITH BH CORRECTION)")
#         print("-"*80)
#         print(f"Total comparisons tested: {len(mw_results_data)}")
#         print(f"Originally significant: {original_significant}")
#         print(f"Significant after BH correction: {corrected_significant}")
#         print(f"Tests no longer significant after correction: {original_significant - corrected_significant}")
#         print(f"Results saved to: {mw_csv_path}")
#         print("="*80)
    
#     if not any(df.empty for df in all_dataframes.values()):
#         print("No data was collected for any matches.")
    
#     # Return dictionary with all created dataframes and test results
#     return all_dataframes, normality_results, mannwhitney_results

# # Example usage with Benjamini-Hochberg correction enabled (default)
# dataframes, normality_results, mannwhitney_results = analyze_event_recovery_times(
#     ("Argentina", arg_matches), ("France", fr_matches), 
#     apply_multiple_correction=True,  # Enable BH correction (default)
#     alpha=0.05  # Significance level
# )

# # If you want to run without correction:
# # dataframes, normality_results, mannwhitney_results = analyze_event_recovery_times(
# #     ("Argentina", arg_matches), ("France", fr_matches), 
# #     apply_multiple_correction=False
# # )

In [22]:
# def apply_benjamini_hochberg_correction_to_impacts(impact_results, alpha=0.05, level_name=""):
#     """
#     Apply Benjamini-Hochberg (FDR) correction to recovery impact results
    
#     Args:
#         impact_results: List of result dictionaries
#         alpha: Significance level (default: 0.05)
#         level_name: Name for the analysis level (for printing)
    
#     Returns:
#         Updated results list with corrected p-values and significance
#     """
#     if not impact_results:
#         return impact_results
    
#     from statsmodels.stats.multitest import multipletests
#     import numpy as np
    
#     # Extract p-values
#     p_values = []
#     for result in impact_results:
#         try:
#             p_val = float(result['p-value'])
#             p_values.append(p_val)
#         except (ValueError, TypeError):
#             p_values.append(float('nan'))
    
#     # Apply BH correction if we have valid p-values
#     valid_p_values = [p for p in p_values if not pd.isna(p)]
#     if valid_p_values:
#         # Apply correction
#         rejected, corrected_p_values, _, _ = multipletests(
#             valid_p_values, 
#             alpha=alpha, 
#             method='fdr_bh'
#         )
        
#         # Update results with corrected values
#         corrected_idx = 0
#         for i, result in enumerate(impact_results):
#             if not pd.isna(p_values[i]):
#                 result['BH Corrected p-value'] = f"{corrected_p_values[corrected_idx]:.6f}"
#                 result['Significant After BH Correction'] = "Yes" if rejected[corrected_idx] else "No"
#                 result['Correction Method'] = f"Benjamini-Hochberg (FDR) - {level_name}"
#                 corrected_idx += 1
#             else:
#                 result['BH Corrected p-value'] = "N/A"
#                 result['Significant After BH Correction'] = "N/A"
#                 result['Correction Method'] = "N/A"
        
#         # Print correction summary
#         original_significant = sum(1 for p in valid_p_values if p <= alpha)
#         corrected_significant = sum(rejected)
        
#         print(f"\n{'='*80}")
#         print(f"BENJAMINI-HOCHBERG CORRECTION APPLIED - {level_name.upper()}")
#         print(f"{'='*80}")
#         print(f"Total tests: {len(valid_p_values)}")
#         print(f"Originally significant: {original_significant}")
#         print(f"Significant after BH correction: {corrected_significant}")
#         print(f"Tests no longer significant: {original_significant - corrected_significant}")
#         print(f"{'='*80}")
    
#     return impact_results

# def analyze_recovery_impact(*team_dicts, output_dir="event_recovery_analysis", apply_multiple_correction=True, alpha=0.05):
#     """
#     Analyze if main teams had faster ball recovery times after key events.
#     For each team_name, treats all opposing teams as a single entity.
#     Also analyzes each match individually.
    
#     Args:
#         *team_dicts: Variable number of tuples, each containing (team_name, matches_dict)
#         output_dir: Directory to save output CSVs
#         apply_multiple_correction: Whether to apply Benjamini-Hochberg correction (default: True)
#         alpha: Significance level for corrections (default: 0.05)
    
#     Returns:
#         DataFrame with impact analysis results
#     """
    
#     # Create output directory
#     os.makedirs(output_dir, exist_ok=True)
    
#     # Store results for final report
#     impact_results = []
    
#     # Store match-specific results
#     match_impact_results = []
    
#     # Process each main team
#     for team_data in team_dicts:
#         team_name, matches_dict = team_data
#         team_formatted = team_name.replace(' ', '_').lower()
        
#         # Create team-level directory
#         team_output_dir = os.path.join(output_dir, team_formatted)
#         os.makedirs(team_output_dir, exist_ok=True)
        
#         # Create team-level subfolder for aggregate analysis
#         team_level_dir = os.path.join(team_output_dir, "team_level")
#         os.makedirs(team_level_dir, exist_ok=True)
        
#         print(f"\nAnalyzing recovery impact for {team_name}...")
        
#         # Store team-specific results for BH correction
#         team_specific_results = []
        
#         # Process each event type at team level
#         for event_type in ['all_goals', 'substitutions', 'tactical_shifts', 'injuries', 'yellow_cards', 'red_cards']:
#             df_name = f"df_{team_formatted}_{event_type}"
            
#             # Skip if this dataframe doesn't exist in globals
#             if df_name not in globals():
#                 print(f"  Skipping {event_type} - No data available")
#                 continue
                
#             # Get the dataframe
#             df = globals()[df_name]
            
#             # Skip if empty
#             if df.empty:
#                 print(f"  Skipping {event_type} - Empty dataset")
#                 continue
            
#             # Separate main team data and opposition data
#             main_team_df = df[df['team'] == team_name]
#             opposition_df = df[df['team'] != team_name]
            
#             # Skip if we don't have data for both
#             if main_team_df.empty or opposition_df.empty:
#                 print(f"  Skipping {event_type} - No data for either {team_name} or opposition")
#                 continue
            
#             # Compare before/after for main team
#             main_before = main_team_df[main_team_df['event_timing'] == 'before event']['time_to_recover']
#             main_after = main_team_df[main_team_df['event_timing'] == 'after event']['time_to_recover']
            
#             # Skip if we don't have enough data for the main team
#             if len(main_before) < 5 or len(main_after) < 5:
#                 print(f"  Skipping {event_type} - Insufficient before/after data for {team_name} (n<5)")
#                 continue
            
#             # Mann-Whitney U test for main team
#             try:
#                 main_mw, main_p = stats.mannwhitneyu(main_before, main_after, alternative='two-sided')
#                 main_significant = main_p <= 0.05
#                 main_avg_before = main_before.mean()
#                 main_avg_after = main_after.mean()
#                 main_faster = main_avg_after < main_avg_before
#             except Exception as e:
#                 print(f"  Error analyzing {team_name} for {event_type}: {e}")
#                 continue
            
#             # Create result dictionary
#             result = {
#                 'Team': team_name,
#                 'Match ID': 'All Matches',
#                 'Event Type': event_type.replace('_', ' ').title(),
#                 'Significant Impact': 'Yes' if main_significant else 'No',
#                 'Avg TRB Before (s)': f"{main_avg_before:.2f}",
#                 'Avg TRB After (s)': f"{main_avg_after:.2f}",
#                 'Faster After Event': 'Yes' if main_faster else 'No',
#                 'p-value': f"{main_p:.4f}"
#             }
            
#             impact_results.append(result)
#             team_specific_results.append(result)
            
#             # Only print if significant
#             if main_significant:
#                 print(f"\n  {event_type.replace('_', ' ').title()} Events:")
#                 print(f"  {'-'*50}")
#                 print(f"  {team_name}:")
#                 print(f"    Avg TRB Before: {main_avg_before:.2f}s, After: {main_avg_after:.2f}s")
#                 print(f"    p-value: {main_p:.4f}")
                
#                 if main_faster:
#                     print(f"  RESULT: ✓ SIGNIFICANT IMPACT - {team_name} recovered the ball FASTER after {event_type.replace('_', ' ')}!")
#                 else:
#                     print(f"  RESULT: ✓ SIGNIFICANT IMPACT - {team_name} recovered the ball SLOWER after {event_type.replace('_', ' ')}!")
#                 print(f"  {'-'*50}")
        
#         # Apply BH correction to team-specific results
#         if team_specific_results and apply_multiple_correction:
#             team_specific_results = apply_benjamini_hochberg_correction_to_impacts(
#                 team_specific_results, alpha=alpha, level_name=f"{team_name} Team Level"
#             )
        
#         # ===================================================================
#         # NEW CODE: Process each match individually
#         # ===================================================================
        
#         print(f"\nAnalyzing match-specific recovery impact for {team_name}...")
        
#         # Process each match for this team
#         for match_id, details in matches_dict.items():
#             print(f"  Analyzing Match {match_id} ({details['team']})...")
            
#             # Create match-specific directory if it doesn't exist
#             match_output_dir = os.path.join(team_output_dir, f"match_{match_id}")
#             os.makedirs(match_output_dir, exist_ok=True)
            
#             # Create match-specific stats directory
#             match_stats_dir = os.path.join(match_output_dir, "stats")
#             os.makedirs(match_stats_dir, exist_ok=True)
            
#             # Store match-specific impacts for this match
#             match_specific_impacts = []
            
#             # Process each event type for this match
#             for event_type in ['all_goals', 'substitutions', 'tactical_shifts', 'injuries', 'yellow_cards', 'red_cards']:
#                 # Get the match-specific dataframe name
#                 match_df_name = f"df_{team_formatted}_match_{match_id}_{event_type}"
                
#                 # Skip if this dataframe doesn't exist in globals
#                 if match_df_name not in globals():
#                     print(f"    Skipping {event_type} - No data available for Match {match_id}")
#                     continue
                    
#                 # Get the dataframe
#                 match_df = globals()[match_df_name]

#                 # Skip if empty
#                 if match_df.empty:
#                     print(f"    Skipping {event_type} - Empty dataset for Match {match_id}")
#                     continue

#                 # Separate main team data
#                 match_main_team_df = match_df[match_df['team'] == team_name]

#                 # Skip if we don't have data for the main team
#                 if match_main_team_df.empty:
#                     print(f"    Skipping {event_type} - No data for {team_name} in Match {match_id}")
#                     continue

#                 # Compare before/after for main team
#                 match_main_before = match_main_team_df[match_main_team_df['event_timing'] == 'before event']['time_to_recover']
#                 match_main_after = match_main_team_df[match_main_team_df['event_timing'] == 'after event']['time_to_recover']

#                 # Skip if we don't have enough data for the main team
#                 if len(match_main_before) < 5 or len(match_main_after) < 5:
#                     print(f"    Skipping {event_type} - Insufficient before/after data for {team_name} in Match {match_id} (n<5)")
#                     continue

#                 # Mann-Whitney U test for main team in this match
#                 try:
#                     match_main_mw, match_main_p = stats.mannwhitneyu(match_main_before, match_main_after, alternative='two-sided')
#                     match_main_significant = match_main_p <= 0.05
#                     match_main_avg_before = match_main_before.mean()
#                     match_main_avg_after = match_main_after.mean()
#                     match_main_faster = match_main_avg_after < match_main_avg_before
#                 except Exception as e:
#                     print(f"    Error analyzing {team_name} for {event_type} in Match {match_id}: {e}")
#                     continue
                
#                 # Create match-specific result dictionary
#                 match_result = {
#                     'Team': team_name,
#                     'Match ID': match_id,
#                     'Opposing Team': details['team'],
#                     'Event Type': event_type.replace('_', ' ').title(),
#                     'Significant Impact': 'Yes' if match_main_significant else 'No',
#                     'Avg TRB Before (s)': f"{match_main_avg_before:.2f}",
#                     'Avg TRB After (s)': f"{match_main_avg_after:.2f}",
#                     'Faster After Event': 'Yes' if match_main_faster else 'No',
#                     'p-value': f"{match_main_p:.4f}"
#                 }
                
#                 # Add to match-specific results
#                 match_specific_impacts.append(match_result)
                
#                 # Add to overall match results
#                 match_impact_results.append(match_result)
                
#                 # Only print if significant
#                 if match_main_significant:
#                     print(f"\n    {event_type.replace('_', ' ').title()} Events in Match {match_id}:")
#                     print(f"    {'-'*50}")
#                     print(f"    {team_name}:")
#                     print(f"      Avg TRB Before: {match_main_avg_before:.2f}s, After: {match_main_avg_after:.2f}s")
#                     print(f"      p-value: {match_main_p:.4f}")
                    
#                     if match_main_faster:
#                         print(f"    RESULT: ✓ SIGNIFICANT IMPACT - {team_name} recovered the ball FASTER after {event_type.replace('_', ' ')} in Match {match_id}!")
#                     else:
#                         print(f"    RESULT: ✓ SIGNIFICANT IMPACT - {team_name} recovered the ball SLOWER after {event_type.replace('_', ' ')} in Match {match_id}!")
#                     print(f"    {'-'*50}")
            
#             # Apply BH correction to match-specific results
#             if match_specific_impacts and apply_multiple_correction:
#                 print(f"\n{'='*60}")
#                 print(f"APPLYING BH CORRECTION TO {team_name.upper()} - MATCH {match_id} IMPACTS")
#                 print(f"{'='*60}")
                
#                 match_specific_impacts = apply_benjamini_hochberg_correction_to_impacts(
#                     match_specific_impacts, alpha=alpha, level_name=f"Match {match_id}"
#                 )
            
#             # Save match-specific impact results to CSV if we have any
#             if match_specific_impacts:
#                 # Create dataframe for this match's results
#                 match_impacts_df = pd.DataFrame(match_specific_impacts)
#                 match_impacts_csv_path = f"{match_output_dir}/recovery_impacts.csv"
#                 match_impacts_df.to_csv(match_impacts_csv_path, index=False)
#                 print(f"\n  Match {match_id} recovery impacts saved to {match_impacts_csv_path}")
                
#                 # Create a filtered version with only significant impacts (using corrected significance if BH was applied)
#                 significance_column = 'Significant After BH Correction' if apply_multiple_correction and 'Significant After BH Correction' in match_impacts_df.columns else 'Significant Impact'
#                 match_sig_impacts = match_impacts_df[match_impacts_df[significance_column] == 'Yes']
                
#                 if not match_sig_impacts.empty:
#                     match_sig_csv_path = f"{match_output_dir}/significant_recovery_impacts.csv"
#                     match_sig_impacts.to_csv(match_sig_csv_path, index=False)
#                     print(f"  Match {match_id} significant impacts saved to {match_sig_csv_path}")
                    
#                     # Print summary of significant findings for this match
#                     print(f"\n  {'-'*60}")
#                     correction_note = " (After BH Correction)" if apply_multiple_correction else ""
#                     print(f"  SUMMARY OF SIGNIFICANT FINDINGS FOR MATCH {match_id}{correction_note}")
#                     print(f"  {'-'*60}")
#                     for _, row in match_sig_impacts.iterrows():
#                         team = row['Team']
#                         event = row['Event Type']
#                         faster = row['Faster After Event']
#                         result = "FASTER" if faster == "Yes" else "SLOWER"
#                         print(f"  • {team} recovered the ball {result} after {event} in Match {match_id}")
#                     print(f"  {'-'*60}")
#                 else:
#                     correction_note = " after BH correction" if apply_multiple_correction else ""
#                     print(f"  No significant impacts were found for Match {match_id}{correction_note}.")
    
#     # ===========================================================================================
#     # APPLY BENJAMINI-HOCHBERG CORRECTION TO ALL IMPACT RESULTS (GLOBAL LEVEL)
#     # ===========================================================================================
    
#     # Combine all results for global correction
#     all_results_for_correction = impact_results + match_impact_results
    
#     if apply_multiple_correction and all_results_for_correction:
#         print(f"\n{'='*80}")
#         print(f"APPLYING BENJAMINI-HOCHBERG CORRECTION TO ALL RECOVERY IMPACT TESTS (GLOBAL)")
#         print(f"{'='*80}")
        
#         all_results_for_correction = apply_benjamini_hochberg_correction_to_impacts(
#             all_results_for_correction, alpha=alpha, level_name="Global Level"
#         )
        
#         # Split back into team and match results
#         team_corrected_results = [r for r in all_results_for_correction if r['Match ID'] == 'All Matches']
#         match_corrected_results = [r for r in all_results_for_correction if r['Match ID'] != 'All Matches']
        
#         # Update the original lists
#         impact_results = team_corrected_results
#         match_impact_results = match_corrected_results
    
#     # Combine all results (team-level and match-level)
#     all_impacts_df = pd.DataFrame(impact_results + match_impact_results)
    
#     # Save all impact results to CSV if we have any
#     if not all_impacts_df.empty:
#         all_impacts_csv_path = f"{output_dir}/all_recovery_impacts.csv"
#         all_impacts_df.to_csv(all_impacts_csv_path, index=False)
#         print(f"\nAll impacts saved to {all_impacts_csv_path}")
        
#         # Save team-level impact results
#         team_impacts_df = pd.DataFrame(impact_results)
#         team_impacts_csv_path = f"{output_dir}/team_level_recovery_impacts.csv"
#         team_impacts_df.to_csv(team_impacts_csv_path, index=False)
#         print(f"Team-level impacts saved to {team_impacts_csv_path}")
        
#         # Save match-level impact results
#         match_impacts_df = pd.DataFrame(match_impact_results)
#         match_impacts_csv_path = f"{output_dir}/match_level_recovery_impacts.csv"
#         match_impacts_df.to_csv(match_impacts_csv_path, index=False)
#         print(f"Match-level impacts saved to {match_impacts_csv_path}")
        
#         # Create filtered versions with only significant impacts
#         # Use corrected significance if BH correction was applied
#         significance_column = 'Significant After BH Correction' if apply_multiple_correction and 'Significant After BH Correction' in all_impacts_df.columns else 'Significant Impact'
        
#         sig_all_impacts = all_impacts_df[all_impacts_df[significance_column] == 'Yes']
#         sig_team_impacts = team_impacts_df[team_impacts_df[significance_column] == 'Yes']
#         sig_match_impacts = match_impacts_df[match_impacts_df[significance_column] == 'Yes']
        
#         correction_note = "_with_BH_correction" if apply_multiple_correction else ""
        
#         if not sig_all_impacts.empty:
#             sig_all_csv_path = f"{output_dir}/significant_all_recovery_impacts{correction_note}.csv"
#             sig_all_impacts.to_csv(sig_all_csv_path, index=False)
#             print(f"Significant all impacts saved to {sig_all_csv_path}")
        
#         if not sig_team_impacts.empty:
#             sig_team_csv_path = f"{output_dir}/significant_team_recovery_impacts{correction_note}.csv"
#             sig_team_impacts.to_csv(sig_team_csv_path, index=False)
#             print(f"Significant team-level impacts saved to {sig_team_csv_path}")
            
#             # Print summary of significant team-level findings
#             correction_summary = " (After BH Correction)" if apply_multiple_correction else ""
#             print("\n" + "="*80)
#             print(f"SUMMARY OF SIGNIFICANT TEAM-LEVEL FINDINGS{correction_summary}")
#             print("="*80)
#             for _, row in sig_team_impacts.iterrows():
#                 team = row['Team']
#                 event = row['Event Type']
#                 faster = row['Faster After Event']
#                 result = "FASTER" if faster == "Yes" else "SLOWER"
#                 print(f"• {team} recovered the ball {result} after {event} (across all matches)")
#             print("="*80)
#         else:
#             correction_note_text = " after BH correction" if apply_multiple_correction else ""
#             print(f"\nNo significant team-level impacts were found for any team/event combination{correction_note_text}.")
        
#         if not sig_match_impacts.empty:
#             sig_match_csv_path = f"{output_dir}/significant_match_recovery_impacts{correction_note}.csv"
#             sig_match_impacts.to_csv(sig_match_csv_path, index=False)
#             print(f"Significant match-level impacts saved to {sig_match_csv_path}")
            
#             # Print summary of significant match-level findings
#             correction_summary = " (After BH Correction)" if apply_multiple_correction else ""
#             print("\n" + "="*80)
#             print(f"SUMMARY OF SIGNIFICANT MATCH-LEVEL FINDINGS{correction_summary}")
#             print("="*80)
#             for _, row in sig_match_impacts.iterrows():
#                 team = row['Team']
#                 match = row['Match ID']
#                 opponent = row['Opposing Team']
#                 event = row['Event Type']
#                 faster = row['Faster After Event']
#                 result = "FASTER" if faster == "Yes" else "SLOWER"
#                 print(f"• {team} recovered the ball {result} after {event} in Match {match} against {opponent}")
#             print("="*80)
#         else:
#             correction_note_text = " after BH correction" if apply_multiple_correction else ""
#             print(f"\nNo significant match-level impacts were found for any match{correction_note_text}.")
    
#     return sig_all_impacts if 'sig_all_impacts' in locals() and not sig_all_impacts.empty else pd.DataFrame()

# # Example usage with Benjamini-Hochberg correction enabled (default)
# impact_df = analyze_recovery_impact(
#     ("Argentina", arg_matches), ("France", fr_matches),
#     apply_multiple_correction=True,  # Enable BH correction (default)
#     alpha=0.05  # Significance level
# )

# # If you want to run without correction:
# # impact_df = analyze_recovery_impact(
# #     ("Argentina", arg_matches), ("France", fr_matches),
# #     apply_multiple_correction=False
# # )

In [23]:
def get_cliffs_delta_explanation(cliff_d, ci_lower, ci_upper):
    """
    Provide explanation for Cliff's Delta results
    """
    try:
        cliff_d = float(cliff_d)
        ci_lower = float(ci_lower)
        ci_upper = float(ci_upper)
        
        if ci_lower > 0:
            return "Significantly faster recovery after events (CI entirely above 0)"
        elif ci_upper < 0:
            return "Significantly slower recovery after events (CI entirely below 0)"
        else:
            return "No significant difference (CI includes 0)"
    except (ValueError, TypeError):
        return "Unable to determine (invalid data)"

def apply_benjamini_hochberg_correction(mannwhitney_results, alpha=0.05):
    """
    Apply Benjamini-Hochberg correction to Mann-Whitney test results
    """
    from statsmodels.stats.multitest import multipletests
    import numpy as np
    
    # Extract p-values
    p_values = []
    df_names = []
    
    for df_name, results in mannwhitney_results.items():
        if 'mw_p' in results and not pd.isna(results['mw_p']):
            p_values.append(results['mw_p'])
            df_names.append(df_name)
    
    if not p_values:
        print("No valid p-values found for correction.")
        return mannwhitney_results
    
    # Apply BH correction
    rejected, corrected_p_values, _, _ = multipletests(p_values, alpha=alpha, method='fdr_bh')
    
    # Update results
    for i, df_name in enumerate(df_names):
        mannwhitney_results[df_name]['corrected_p_value'] = corrected_p_values[i]
        mannwhitney_results[df_name]['significant_corrected'] = rejected[i]
        mannwhitney_results[df_name]['correction_method'] = f"Benjamini-Hochberg (FDR), α={alpha}"
    
    # Print summary
    original_significant = sum(1 for p in p_values if p <= alpha)
    corrected_significant = sum(rejected)
    
    print(f"BH Correction Applied:")
    print(f"  Total tests: {len(p_values)}")
    print(f"  Originally significant (α={alpha}): {original_significant}")
    print(f"  Significant after BH correction: {corrected_significant}")
    print(f"  Tests no longer significant: {original_significant - corrected_significant}")
    
    return mannwhitney_results

def analyze_event_recovery_times(*team_dicts, output_dir="event_recovery_analysis", window_minutes=5, apply_multiple_correction=True, alpha=0.05):
    """
    Analyze recovery times around different event types in football matches.
    
    Uses separate Benjamini-Hochberg corrections for:
    - Match-level analysis: Controls FDR within each individual match
    - Team-level analysis: Controls FDR within each team's aggregated data
    
    Args:
        *team_dicts: Variable number of tuples, each containing (team_name, matches_dict)
        output_dir: Directory to save output CSVs (default: "event_recovery_analysis")
        window_minutes: Time window in minutes before and after events to analyze (default: 5)
        apply_multiple_correction: Whether to apply Benjamini-Hochberg correction (default: True)
        alpha: Significance level for corrections (default: 0.05)
    
    Returns:
        Dictionary containing dataframes with recovery times for each team and event type
    """
    import numpy as np
    import pandas as pd
    import os
    from scipy import stats
    from statsmodels.stats.multitest import multipletests
    
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    # Dictionary to store all dataframes
    all_dataframes = {}
    
    # Dictionary to store Shapiro-Wilk test results
    normality_results = {}
    
    # Dictionary to store Mann-Whitney test results (for backward compatibility)
    mannwhitney_results = {}
    
    # Storage for separate analysis levels
    match_level_tests = []  # Individual match analyses
    team_level_tests = []   # Team-aggregated analyses
    
    # Store impact results for summary reporting
    match_impact_results = []
    team_impact_results = []
    
    print(f"\n{'='*80}")
    print("EVENT RECOVERY ANALYSIS WITH SEPARATE BH CORRECTION")
    print('='*80)
    print(f"BH Correction Strategy:")
    print(f"  Match-level: Separate correction within each individual match")
    print(f"  Team-level: Separate correction within each team's aggregated data")
    print(f"  Alpha level: {alpha}")
    print(f"  Apply correction: {apply_multiple_correction}")
    print('='*80)
    
    # Process all teams and their matches
    processed_matches = set()
    
    def cliffs_delta(group1, group2, n_bootstrap=1000, random_state=None):
        """
        Calculate Cliff's Delta effect size with bootstrap confidence intervals
        """
        import numpy as np
        
        # Convert to numpy arrays
        group1 = np.array(group1)
        group2 = np.array(group2)
        
        n1 = len(group1)
        n2 = len(group2)
        
        if n1 == 0 or n2 == 0:
            return float('nan'), "Not applicable", float('nan'), float('nan')
        
        # Set random seed if provided
        if random_state is not None:
            np.random.seed(random_state)
        
        def calculate_cliffs_delta(g1, g2):
            """Helper function to calculate Cliff's Delta for any two groups"""
            dominance = 0
            total_comparisons = len(g1) * len(g2)
            
            for x in g1:
                for y in g2:
                    if x > y:
                        dominance += 1
                    elif x < y:
                        dominance -= 1
            
            return dominance / total_comparisons
        
        # Calculate original Cliff's Delta
        delta = calculate_cliffs_delta(group1, group2)
        
        # Bootstrap procedure
        bootstrap_deltas = []
        
        for i in range(n_bootstrap):
            # Resample both groups with replacement
            bootstrap_group1 = np.random.choice(group1, size=n1, replace=True)
            bootstrap_group2 = np.random.choice(group2, size=n2, replace=True)
            
            # Calculate Cliff's Delta for bootstrap sample
            bootstrap_delta = calculate_cliffs_delta(bootstrap_group1, bootstrap_group2)
            bootstrap_deltas.append(bootstrap_delta)
        
        bootstrap_deltas = np.array(bootstrap_deltas)
        
        # Calculate 95% confidence intervals using percentile method
        ci_lower = np.percentile(bootstrap_deltas, 2.5)
        ci_upper = np.percentile(bootstrap_deltas, 97.5)
        
        # Ensure CI is within [-1, 1] bounds
        ci_lower = max(-1, ci_lower)
        ci_upper = min(1, ci_upper)
        
        # Interpret magnitude according to Romano et al. (2006)
        if abs(delta) < 0.147:
            magnitude = "Negligible"
        elif abs(delta) < 0.33:
            magnitude = "Small"
        elif abs(delta) < 0.474:
            magnitude = "Medium"
        else:
            magnitude = "Large"
        
        return delta, magnitude, ci_lower, ci_upper
        
    for team_data in team_dicts:
        # Unpack the team data
        team_name, matches_dict = team_data
        
        # Create team-specific directory
        team_output_dir = os.path.join(output_dir, team_name.replace(' ', '_').lower())
        os.makedirs(team_output_dir, exist_ok=True)
        
        # Create team-level subfolder
        team_level_dir = os.path.join(team_output_dir, "team_level")
        os.makedirs(team_level_dir, exist_ok=True)
        
        # Create stats folder in team-level directory
        team_stats_dir = os.path.join(team_level_dir, "stats")
        os.makedirs(team_stats_dir, exist_ok=True)
        
        print(f"\nProcessing matches for {team_name}...")
        
        # Initialize dictionaries for this team (aggregated across all matches)
        team_event_recoveries = {
            'all_goals': {'times': [], 'timing': [], 'count': 0, 'team': []},
            'substitutions': {'times': [], 'timing': [], 'count': 0, 'team': []},
            'tactical_shifts': {'times': [], 'timing': [], 'count': 0, 'team': []},
            'injuries': {'times': [], 'timing': [], 'count': 0, 'team': []},
            'yellow_cards': {'times': [], 'timing': [], 'count': 0, 'team': []},
            'red_cards': {'times': [], 'timing': [], 'count': 0, 'team': []}
        }
        
        # Team-specific normality results
        team_normality_results = []
        
        # Team-specific tests (to be corrected together)
        team_tests_for_this_team = []
        
        # Process all matches for this team
        for match_id, details in matches_dict.items():
            print(f"  Processing match {match_id} ({details['team']})...")
            
            # Create variable names based on team names
            team1 = details['team'].replace(' ', '_').lower()  # opposing team
            team2 = team_name.replace(' ', '_').lower()  # current team
            
            # Store the opposing team's name for reference
            opposing_team_name = details['team']
            
            # Get the time-to-recover DataFrame
            df_var_name = f"df_time_to_recover_{team2}_vs_{team1}"
            if df_var_name not in globals():
                print(f"    No time-to-recover data found for {df_var_name}")
                continue
                
            time_to_recover_df = globals()[df_var_name]
            
            # Ensure loss_time is datetime
            time_to_recover_df['loss_time'] = pd.to_datetime(time_to_recover_df['loss_time'])
            
            # Get events from the StatsBomb API
            events = sb.events(match_id=match_id)
            events['timestamp'] = pd.to_datetime(events['timestamp'], errors='coerce')
            
            # Filter for periods 1-4
            events = events[events['period'].isin([1, 2, 3, 4])]
            
            if events.empty:
                print(f"    No events found for periods 1-4 in Match {match_id}. Skipping...")
                continue
            
            # Create match-specific directory
            match_output_dir = os.path.join(team_output_dir, f"match_{match_id}")
            os.makedirs(match_output_dir, exist_ok=True)
            
            # Create stats folder in match directory
            match_stats_dir = os.path.join(match_output_dir, "stats")
            os.makedirs(match_stats_dir, exist_ok=True)
            
            # Initialize dictionaries for this match
            match_event_recoveries = {
                'all_goals': {'times': [], 'timing': [], 'count': 0, 'team': []},
                'substitutions': {'times': [], 'timing': [], 'count': 0, 'team': []},
                'tactical_shifts': {'times': [], 'timing': [], 'count': 0, 'team': []},
                'injuries': {'times': [], 'timing': [], 'count': 0, 'team': []},
                'yellow_cards': {'times': [], 'timing': [], 'count': 0, 'team': []},
                'red_cards': {'times': [], 'timing': [], 'count': 0, 'team': []}
            }
            
            # Match-specific normality results
            match_normality_results = []
            
            # Match-specific tests (to be corrected together)
            match_tests_for_this_match = []
            
            # Define event type filters
            event_types = {
                'all_goals': events[((events['type'] == 'Shot') & (events['shot_outcome'] == 'Goal')) | 
                                   (events['type'] == 'Own Goal For')].copy(),
                'substitutions': events[events['type'] == 'Substitution'].copy(),
                'tactical_shifts': events[events['type'] == 'Tactical Shift'].copy(),
                'injuries': events[events['type'] == 'Injury Stoppage'].copy(),
                'yellow_cards': events[(events['type'] == 'Foul Committed') & 
                                      (events['foul_committed_card'] == 'Yellow Card')].copy(),
                'red_cards': events[(events['type'] == 'Foul Committed') & 
                                   (events['foul_committed_card'] == 'Red Card')].copy()
            }
            
            # Process each type of event
            for event_type, event_events in event_types.items():
                # Count the number of events of this type (team-specific and match-specific)
                team_event_recoveries[event_type]['count'] += len(event_events)
                match_event_recoveries[event_type]['count'] += len(event_events)
                
                for _, event in event_events.iterrows():
                    event_time = event['timestamp']
                    event_period = event['period']
                    
                    # Define time window around the event
                    window_start = event_time - pd.Timedelta(minutes=window_minutes)
                    window_end = event_time + pd.Timedelta(minutes=window_minutes)
                    
                    # Get all ball loss and recovery events in the window
                    window_events = time_to_recover_df[
                        (time_to_recover_df['loss_time'] >= window_start) &
                        (time_to_recover_df['loss_time'] <= window_end) &
                        (time_to_recover_df['period'] == event_period) &
                        (time_to_recover_df['time_to_recover'] >= 1)
                    ].copy()
                    
                    # Label events as "before event" or "after event"
                    window_events['event_timing'] = window_events['loss_time'].apply(
                        lambda x: "before event" if x < event_time else "after event"
                    )
                    
                    # Add to both team-specific and match-specific data
                    for _, row in window_events.iterrows():
                        # Team data (aggregated)
                        team_event_recoveries[event_type]['times'].append(row['time_to_recover'])
                        team_event_recoveries[event_type]['timing'].append(row['event_timing'])
                        team_event_recoveries[event_type]['team'].append(row['team'])
                        
                        # Match data (specific to this match)
                        match_event_recoveries[event_type]['times'].append(row['time_to_recover'])
                        match_event_recoveries[event_type]['timing'].append(row['event_timing'])
                        match_event_recoveries[event_type]['team'].append(row['team'])
            
            # MATCH-LEVEL ANALYSIS: Create and analyze match-specific dataframes
            for event_type, data in match_event_recoveries.items():
                if data['times']:
                    # Create DataFrame for this event type in this match
                    df_name = f"df_{team_name.replace(' ', '_').lower()}_match_{match_id}_{event_type}"
                    
                    df = pd.DataFrame({
                        'time_to_recover': data['times'],
                        'event_timing': data['timing'],
                        'team': data['team']
                    })
                    
                    # Save the dataframe
                    globals()[df_name] = df
                    all_dataframes[df_name] = df
                    
                    # Save to match-specific directory
                    csv_path = f"{match_output_dir}/{event_type}_recovery_times.csv"
                    df.to_csv(csv_path, index=False)
                    
                    # Filter data to only include the specified team for statistical analysis
                    team_data_df = df[df['team'] == team_name]
                    team_recovery_times = team_data_df['time_to_recover'].tolist()
                    
                    # Perform Shapiro-Wilk test if we have sufficient data
                    if len(team_recovery_times) >= 8:
                        normality_results[df_name] = {}
                        
                        if len(team_recovery_times) < 5000:
                            shapiro_stat, shapiro_p = stats.shapiro(team_recovery_times)
                            normality_results[df_name]['shapiro_stat'] = shapiro_stat
                            normality_results[df_name]['shapiro_p'] = shapiro_p
                            normality_results[df_name]['shapiro_normal'] = shapiro_p > 0.05
                            normality_results[df_name]['sample_size'] = len(team_recovery_times)
                            
                            match_result = {
                                'Event Type': event_type.replace('_', ' ').title(),
                                'Sample Size': len(team_recovery_times),
                                'Shapiro-Wilk Statistic': f"{shapiro_stat:.4f}",
                                'p-value': f"{shapiro_p:.18f}",
                                'Normal Distribution': "Yes" if shapiro_p > 0.05 else "No"
                            }
                            match_normality_results.append(match_result)
                    
                    # Split data into before and after events for Mann-Whitney test
                    before_times = team_data_df[team_data_df['event_timing'] == 'before event']['time_to_recover'].tolist()
                    after_times = team_data_df[team_data_df['event_timing'] == 'after event']['time_to_recover'].tolist()
                    
                    # Calculate averages
                    avg_before = np.mean(before_times) if len(before_times) > 0 else float('nan')
                    avg_after = np.mean(after_times) if len(after_times) > 0 else float('nan')
                    faster_after = avg_after < avg_before if not (pd.isna(avg_before) or pd.isna(avg_after)) else False
                    
                    # Calculate Cliff's Delta
                    cliff_d = float('nan')
                    cliff_magnitude = "Not applicable"
                    ci_lower = float('nan')
                    ci_upper = float('nan')
                    
                    if len(before_times) > 0 and len(after_times) > 0:
                        cliff_d, cliff_magnitude, ci_lower, ci_upper = cliffs_delta(before_times, after_times, random_state=8)
                    
                    # Perform Mann-Whitney U test if there's enough data
                    if len(before_times) >= 5 and len(after_times) >= 5:
                        mw_stat, mw_p = stats.mannwhitneyu(before_times, after_times, alternative='two-sided')
                        
                        # Store match-level test result
                        match_test = {
                            'analysis_level': 'match',
                            'team': team_name,
                            'match_id': match_id,
                            'opposing_team': opposing_team_name,
                            'event_type': event_type,
                            'p_value': mw_p,
                            'test_statistic': mw_stat,
                            'before_n': len(before_times),
                            'after_n': len(after_times),
                            'avg_before': avg_before,
                            'avg_after': avg_after,
                            'faster_after': faster_after,
                            'cliffs_delta': cliff_d,
                            'ci_lower': ci_lower,
                            'ci_upper': ci_upper,
                            'effect_size': cliff_magnitude,
                            'significant_uncorrected': mw_p <= alpha
                        }
                        
                        match_tests_for_this_match.append(match_test)
                        match_level_tests.append(match_test)
                        
                        # Store in mannwhitney_results for backward compatibility
                        mannwhitney_results[df_name] = {
                            'mw_stat': mw_stat,
                            'mw_p': mw_p,
                            'before_sample_size': len(before_times),
                            'after_sample_size': len(after_times),
                            'avg_before': avg_before,
                            'avg_after': avg_after,
                            'faster_after': faster_after,
                            'significant': mw_p <= 0.05,
                            'cliffs_delta': cliff_d,
                            'ci_lower': ci_lower,
                            'ci_upper': ci_upper,
                            'effect_size': cliff_magnitude
                        }
            
            # APPLY BH CORRECTION TO THIS MATCH'S TESTS
            if apply_multiple_correction and match_tests_for_this_match:
                print(f"    Applying BH correction to {len(match_tests_for_this_match)} tests in Match {match_id}...")
                
                match_p_values = [test['p_value'] for test in match_tests_for_this_match]
                rejected, corrected_p_values, _, _ = multipletests(
                    match_p_values, alpha=alpha, method='fdr_bh'
                )
                
                # Update match test results with correction
                for i, test in enumerate(match_tests_for_this_match):
                    test['bh_corrected_p'] = corrected_p_values[i]
                    test['significant_after_bh'] = rejected[i]
                    test['correction_scope'] = f"Within Match {match_id}"
                    test['correction_method'] = f"Benjamini-Hochberg (Match-level, α={alpha})"
                
                # Print match correction summary
                original_sig = sum(1 for p in match_p_values if p <= alpha)
                corrected_sig = sum(rejected)
                print(f"    Match {match_id}: {original_sig} → {corrected_sig} significant tests after BH correction")
                
                # Create match impact results
                for test in match_tests_for_this_match:
                    if test['significant_after_bh']:
                        match_impact = {
                            'Team': team_name,
                            'Match ID': match_id,
                            'Opposing Team': opposing_team_name,
                            'Event Type': test['event_type'].replace('_', ' ').title(),
                            'Significant Impact': 'Yes',
                            'Avg TRB Before (s)': f"{test['avg_before']:.2f}",
                            'Avg TRB After (s)': f"{test['avg_after']:.2f}",
                            'Faster After Event': 'Yes' if test['faster_after'] else 'No',
                            'Original p-value': f"{test['p_value']:.6f}",
                            'BH Corrected p-value': f"{test['bh_corrected_p']:.6f}",
                            'Correction Method': test['correction_method']
                        }
                        match_impact_results.append(match_impact)
            
            # Save match-specific normality test results
            if match_normality_results:
                match_results_df = pd.DataFrame(match_normality_results)
                match_normality_csv_path = f"{match_stats_dir}/shapiro_wilk_test_results.csv"
                match_results_df.to_csv(match_normality_csv_path, index=False)
            
            # Save match-specific test results
            if match_tests_for_this_match:
                match_test_df = pd.DataFrame(match_tests_for_this_match)
                match_test_csv_path = f"{match_stats_dir}/mann_whitney_test_results.csv"
                match_test_df.to_csv(match_test_csv_path, index=False)
        
        # TEAM-LEVEL ANALYSIS: Create and analyze team-aggregated dataframes
        print(f"  Performing team-level analysis for {team_name}...")
        
        for event_type, data in team_event_recoveries.items():
            if data['times']:
                # Create DataFrame for this event type aggregated across all matches
                df_name = f"df_{team_name.replace(' ', '_').lower()}_{event_type}"
                
                df = pd.DataFrame({
                    'time_to_recover': data['times'],
                    'event_timing': data['timing'],
                    'team': data['team']
                })
                
                # Save the dataframe
                globals()[df_name] = df
                all_dataframes[df_name] = df
                
                # Save to team-level directory
                csv_path = f"{team_level_dir}/{event_type}_recovery_times.csv"
                df.to_csv(csv_path, index=False)
                
                # Filter data to only include the specified team for statistical analysis
                team_data_df = df[df['team'] == team_name]
                team_recovery_times = team_data_df['time_to_recover'].tolist()
                
                # Perform Shapiro-Wilk test if we have sufficient data
                if len(team_recovery_times) >= 8:
                    normality_results[df_name] = {}
                    
                    if len(team_recovery_times) < 5000:
                        shapiro_stat, shapiro_p = stats.shapiro(team_recovery_times)
                        normality_results[df_name]['shapiro_stat'] = shapiro_stat
                        normality_results[df_name]['shapiro_p'] = shapiro_p
                        normality_results[df_name]['shapiro_normal'] = shapiro_p > 0.05
                        normality_results[df_name]['sample_size'] = len(team_recovery_times)
                        
                        team_result = {
                            'Event Type': event_type.replace('_', ' ').title(),
                            'Sample Size': len(team_recovery_times),
                            'Shapiro-Wilk Statistic': f"{shapiro_stat:.4f}",
                            'p-value': f"{shapiro_p:.18f}",
                            'Normal Distribution': "Yes" if shapiro_p > 0.05 else "No"
                        }
                        team_normality_results.append(team_result)
                
                # Split data into before and after events for Mann-Whitney test
                before_times = team_data_df[team_data_df['event_timing'] == 'before event']['time_to_recover'].tolist()
                after_times = team_data_df[team_data_df['event_timing'] == 'after event']['time_to_recover'].tolist()
                
                # Calculate averages
                avg_before = np.mean(before_times) if len(before_times) > 0 else float('nan')
                avg_after = np.mean(after_times) if len(after_times) > 0 else float('nan')
                faster_after = avg_after < avg_before if not (pd.isna(avg_before) or pd.isna(avg_after)) else False
                
                # Calculate Cliff's Delta
                cliff_d = float('nan')
                cliff_magnitude = "Not applicable"
                ci_lower = float('nan')
                ci_upper = float('nan')
                
                if len(before_times) > 0 and len(after_times) > 0:
                    cliff_d, cliff_magnitude, ci_lower, ci_upper = cliffs_delta(before_times, after_times, random_state=8)
                
                # Perform Mann-Whitney U test if there's enough data
                if len(before_times) >= 5 and len(after_times) >= 5:
                    mw_stat, mw_p = stats.mannwhitneyu(before_times, after_times, alternative='two-sided')
                    
                    # Store team-level test result
                    team_test = {
                        'analysis_level': 'team',
                        'team': team_name,
                        'match_id': 'all_matches',
                        'event_type': event_type,
                        'p_value': mw_p,
                        'test_statistic': mw_stat,
                        'before_n': len(before_times),
                        'after_n': len(after_times),
                        'avg_before': avg_before,
                        'avg_after': avg_after,
                        'faster_after': faster_after,
                        'cliffs_delta': cliff_d,
                        'ci_lower': ci_lower,
                        'ci_upper': ci_upper,
                        'effect_size': cliff_magnitude,
                        'significant_uncorrected': mw_p <= alpha
                    }
                    # Store in mannwhitney_results for backward compatibility
                    mannwhitney_results[df_name] = {
                        'mw_stat': mw_stat,
                        'mw_p': mw_p,
                        'before_sample_size': len(before_times),
                        'after_sample_size': len(after_times),
                        'avg_before': avg_before,
                        'avg_after': avg_after,
                        'faster_after': faster_after,
                        'significant': mw_p <= 0.05,
                        'cliffs_delta': cliff_d,
                        'ci_lower': ci_lower,
                        'ci_upper': ci_upper,
                        'effect_size': cliff_magnitude
                    }
                    team_tests_for_this_team.append(team_test)
                    team_level_tests.append(team_test)
        
        # APPLY BH CORRECTION TO THIS TEAM'S TESTS
        if apply_multiple_correction and team_tests_for_this_team:
            print(f"  Applying BH correction to {len(team_tests_for_this_team)} team-level tests for {team_name}...")
            
            team_p_values = [test['p_value'] for test in team_tests_for_this_team]
            rejected, corrected_p_values, _, _ = multipletests(
                team_p_values, alpha=alpha, method='fdr_bh'
            )
            
            # Update team test results with correction
            for i, test in enumerate(team_tests_for_this_team):
                test['bh_corrected_p'] = corrected_p_values[i]
                test['significant_after_bh'] = rejected[i]
                test['correction_scope'] = f"Within {team_name} Team"
                test['correction_method'] = f"Benjamini-Hochberg (Team-level, α={alpha})"
            
            # Print team correction summary
            original_sig = sum(1 for p in team_p_values if p <= alpha)
            corrected_sig = sum(rejected)
            print(f"  {team_name}: {original_sig} → {corrected_sig} significant tests after BH correction")
            
            # Create team impact results
            for test in team_tests_for_this_team:
                if test['significant_after_bh']:
                    team_impact = {
                        'Team': team_name,
                        'Match ID': 'All Matches',
                        'Event Type': test['event_type'].replace('_', ' ').title(),
                        'Significant Impact': 'Yes',
                        'Avg TRB Before (s)': f"{test['avg_before']:.2f}",
                        'Avg TRB After (s)': f"{test['avg_after']:.2f}",
                        'Faster After Event': 'Yes' if test['faster_after'] else 'No',
                        'Original p-value': f"{test['p_value']:.6f}",
                        'BH Corrected p-value': f"{test['bh_corrected_p']:.6f}",
                        'Correction Method': test['correction_method']
                    }
                    team_impact_results.append(team_impact)
        
        # Save team-specific normality test results
        if team_normality_results:
            team_results_df = pd.DataFrame(team_normality_results)
            team_normality_csv_path = f"{team_stats_dir}/shapiro_wilk_test_results.csv"
            team_results_df.to_csv(team_normality_csv_path, index=False)
        
        # Save team-specific test results
        if team_tests_for_this_team:
            team_test_df = pd.DataFrame(team_tests_for_this_team)
            team_test_csv_path = f"{team_stats_dir}/mann_whitney_test_results.csv"
            team_test_df.to_csv(team_test_csv_path, index=False)
    
    # SAVE COMPREHENSIVE RESULTS
    
    # Save normality test results
    if normality_results:
        results_data = []
        for df_name, results in normality_results.items():
            team_name = df_name.split('_')[1] if len(df_name.split('_')) > 1 else "Unknown"
            
            if "match_" in df_name:
                match_id = df_name.split('_match_')[1].split('_')[0]
                event_type = '_'.join(df_name.split(f'_match_{match_id}_')[1:])
                
                if event_type != "all_events":
                    results_data.append({
                        'Team': team_name.replace('_', ' ').title(),
                        'Match ID': match_id,
                        'Event Type': event_type.replace('_', ' ').title(),
                        'Sample Size': results.get('sample_size', "N/A"),
                        'Shapiro-Wilk Statistic': f"{results.get('shapiro_stat', 'N/A'):.4f}" if isinstance(results.get('shapiro_stat'), (float, int)) else "N/A",
                        'p-value': f"{results.get('shapiro_p', 'N/A'):.18f}" if isinstance(results.get('shapiro_p'), (float, int)) else "N/A",
                        'Normal Distribution': "Yes" if results.get('shapiro_normal', False) else "No"
                    })
            else:
                event_type = '_'.join(df_name.split('_')[2:])
                
                if event_type != "all_events":
                    results_data.append({
                        'Team': team_name.replace('_', ' ').title(),
                        'Match ID': "All Matches",
                        'Event Type': event_type.replace('_', ' ').title(),
                        'Sample Size': results.get('sample_size', "N/A"),
                        'Shapiro-Wilk Statistic': f"{results.get('shapiro_stat', 'N/A'):.4f}" if isinstance(results.get('shapiro_stat'), (float, int)) else "N/A",
                        'p-value': f"{results.get('shapiro_p', 'N/A'):.18f}" if isinstance(results.get('shapiro_p'), (float, int)) else "N/A",
                        'Normal Distribution': "Yes" if results.get('shapiro_normal', False) else "No"
                    })
        
        if results_data:
            results_df = pd.DataFrame(results_data)
            normality_csv_path = f"{output_dir}/shapiro_wilk_test_results.csv"
            results_df.to_csv(normality_csv_path, index=False)
    
    # Save match-level results
    if match_level_tests:
        match_df = pd.DataFrame(match_level_tests)
        match_csv_path = f"{output_dir}/match_level_mann_whitney_results.csv"
        match_df.to_csv(match_csv_path, index=False)
        
        # Save significant match results
        if apply_multiple_correction:
            sig_match = match_df[match_df['significant_after_bh'] == True]
        else:
            sig_match = match_df[match_df['significant_uncorrected'] == True]
        
        if not sig_match.empty:
            sig_match_csv = f"{output_dir}/significant_match_level_results.csv"
            sig_match.to_csv(sig_match_csv, index=False)
    
    # Save team-level results
    if team_level_tests:
        team_df = pd.DataFrame(team_level_tests)
        team_csv_path = f"{output_dir}/team_level_mann_whitney_results.csv"
        team_df.to_csv(team_csv_path, index=False)
        
        # Save significant team results
        if apply_multiple_correction:
            sig_team = team_df[team_df['significant_after_bh'] == True]
        else:
            sig_team = team_df[team_df['significant_uncorrected'] == True]
        
        if not sig_team.empty:
            sig_team_csv = f"{output_dir}/significant_team_level_results.csv"
            sig_team.to_csv(sig_team_csv, index=False)
    
    # Save impact results
    all_impacts = match_impact_results + team_impact_results
    if all_impacts:
        all_impacts_df = pd.DataFrame(all_impacts)
        all_impacts_csv = f"{output_dir}/all_significant_impacts.csv"
        all_impacts_df.to_csv(all_impacts_csv, index=False)
        
        # Save separate files for match and team impacts
        if match_impact_results:
            match_impacts_df = pd.DataFrame(match_impact_results)
            match_impacts_csv = f"{output_dir}/significant_match_impacts.csv"
            match_impacts_df.to_csv(match_impacts_csv, index=False)
        
        if team_impact_results:
            team_impacts_df = pd.DataFrame(team_impact_results)
            team_impacts_csv = f"{output_dir}/significant_team_impacts.csv"
            team_impacts_df.to_csv(team_impacts_csv, index=False)
    
    # PRINT FINAL SUMMARY
    print(f"\n{'='*80}")
    print("FINAL ANALYSIS SUMMARY")
    print('='*80)
    
    if match_level_tests:
        match_original = sum(1 for test in match_level_tests if test['significant_uncorrected'])
        match_corrected = sum(1 for test in match_level_tests if test.get('significant_after_bh', False))
        print(f"Match-level tests: {len(match_level_tests)} total")
        print(f"  Originally significant: {match_original}")
        print(f"  Significant after BH correction: {match_corrected}")
    
    if team_level_tests:
        team_original = sum(1 for test in team_level_tests if test['significant_uncorrected'])
        team_corrected = sum(1 for test in team_level_tests if test.get('significant_after_bh', False))
        print(f"Team-level tests: {len(team_level_tests)} total")
        print(f"  Originally significant: {team_original}")
        print(f"  Significant after BH correction: {team_corrected}")
    
    # Print significant findings
    if apply_multiple_correction:
        if team_impact_results:
            print(f"\nSIGNIFICNT TEAM-LEVEL FINDINGS (After BH Correction):")
            for impact in team_impact_results:
                result_type = "FASTER" if impact['Faster After Event'] == 'Yes' else "SLOWER"
                print(f"  • {impact['Team']} recovered {result_type} after {impact['Event Type']} (p={impact['BH Corrected p-value']})")
        
        if match_impact_results:
            print(f"\nSIGNIFICANT MATCH-LEVEL FINDINGS (After BH Correction):")
            for impact in match_impact_results:
                result_type = "FASTER" if impact['Faster After Event'] == 'Yes' else "SLOWER"
                print(f"  • {impact['Team']} recovered {result_type} after {impact['Event Type']} in Match {impact['Match ID']} (p={impact['BH Corrected p-value']})")
    
    print('='*80)
    
    return all_dataframes, normality_results, mannwhitney_results, match_level_tests, team_level_tests

# Usage example:
dataframes, normality_results, mannwhitney_results, match_tests, team_tests = analyze_event_recovery_times(
    ("Argentina", arg_matches), 
    ("France", fr_matches),
    apply_multiple_correction=True,
    alpha=0.05
)




EVENT RECOVERY ANALYSIS WITH SEPARATE BH CORRECTION
BH Correction Strategy:
  Match-level: Separate correction within each individual match
  Team-level: Separate correction within each team's aggregated data
  Alpha level: 0.05
  Apply correction: True

Processing matches for Argentina...
  Processing match 3857264 (Poland)...
    Applying BH correction to 1 tests in Match 3857264...
    Match 3857264: 0 → 0 significant tests after BH correction
  Processing match 3857289 (Mexico)...
    Applying BH correction to 3 tests in Match 3857289...
    Match 3857289: 1 → 0 significant tests after BH correction
  Processing match 3857300 (Saudi Arabia)...
    Applying BH correction to 3 tests in Match 3857300...
    Match 3857300: 2 → 2 significant tests after BH correction
  Processing match 3869151 (Australia)...
    Applying BH correction to 2 tests in Match 3869151...
    Match 3869151: 0 → 0 significant tests after BH correction
  Processing match 3869321 (Netherlands)...
    Applying BH

# Dashboard

In [28]:
# Import required libraries
import os
import pandas as pd
import numpy as np
from scipy import stats
import json
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
from datetime import datetime


# Stub function for add_explanations_to_all_plots
def add_explanations_to_all_plots(output_dir):
    """Stub function for adding explanations to plots."""
    print("Note: Using stub function for add_explanations_to_all_plots")
    pass

def test_recovery_time_normality(*team_dicts, output_dir="visuals"): 
    """
    Performs normality tests on recovery time data for matches involving teams in parameters.
    Creates QQ plots and KDE plots for each match and period.
    Features sequential team-match filtering where match options depend on team selection.
    
    Args:
        *team_dicts: Variable number of tuples, each containing (team_name, team_color, matches_dict)
        output_dir: Directory to save the output plots and reports
    
    Returns:
        Dictionary with normality test results for each match and period
    """
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Define periods to consider, including "All" for combined data
    numeric_periods = [1, 2, 3, 4]
    periods = numeric_periods + ["All"]
    
    # Dictionary to store test results
    normality_results = {}
    
    # Create a set of teams we're explicitly analyzing (from parameters)
    target_teams = {team_name for team_name, _, _ in team_dicts}
    print(f"Analyzing only matches involving these teams: {', '.join(target_teams)}")
    
    # Get team colors
    team_colors = {team_name: team_color for team_name, team_color, _ in team_dicts}
    
    # Define a set of distinct colors for mean lines that are visually different from typical team colors
    mean_line_colors = [
        "#FF4500",  # OrangeRed
        "#32CD32",  # LimeGreen
        "#9932CC",  # DarkOrchid
        "#FF8C00",  # DarkOrange
        "#1E90FF",  # DodgerBlue
        "#FF1493",  # DeepPink
        "#8A2BE2",  # BlueViolet
        "#FF6347",  # Tomato
        "#20B2AA",  # LightSeaGreen
        "#CD5C5C",  # IndianRed
    ]

    # Create a dictionary to store the mean line color for each team
    team_mean_colors = {}

    # Assign colors to teams in a deterministic way
    for i, team_name in enumerate(sorted(target_teams)):
        team_mean_colors[team_name] = mean_line_colors[i % len(mean_line_colors)]
    
    # Define a set of different dash styles for each period
    period_dash_styles = {
        "All": "solid",
        1: "dash",
        2: "dot",
        3: "dashdot",
        4: "longdash"
    }

    # Define line widths for better visual differentiation
    period_line_widths = {
        "All": 3,
        1: 2,
        2: 2,
        3: 2,
        4: 2
    }
    
    # Store match data for the filtering feature
    match_data = {}  # Will store all match information
    team_matches = {}  # Will map teams to their matches
    results_data = []  # Will store all result rows
    
    # Process all teams and their matches
    processed_matches = set()
    
    for team_name, team_color, matches_dict in team_dicts:
        print(f"\nProcessing normality tests for {team_name}...")
        
        # Initialize team_matches entry
        if team_name not in team_matches:
            team_matches[team_name] = []
        
        # Process all matches for this team
        for match_id, details in matches_dict.items():
            # Skip if we've already processed this match
            if match_id in processed_matches:
                print(f"Skipping already processed Match ID: {match_id}")
                continue
                
            try:
                print(f"Processing Match ID: {match_id} ({details['team']})...")
                
                # Mark this match as processed
                processed_matches.add(match_id)
                
                # Get the team names
                primary_team = details['team']
                
                # Create variable names based on team names
                team1 = primary_team.replace(' ', '_').lower()
                team2 = team_name.replace(' ', '_').lower()
                
                # Initialize match results in the dictionary
                if match_id not in normality_results:
                    normality_results[match_id] = {
                        'primary_team': primary_team,
                        'team_name': team_name,
                        'periods': {}
                    }
                
                # Retrieve the main team data 
                df_main = globals()[f"df_time_to_recover_{team2}_vs_{team1}"]
                
                # Get all teams in this match
                match_teams = df_main['team'].unique()
                match_teams_in_target = [t for t in match_teams if t in target_teams]
                
                # Store match information for filtering
                opponent_team = [t for t in match_teams if t in target_teams and t != primary_team]
                opponent_name = opponent_team[0] if opponent_team else "Unknown"
                match_name = f"{primary_team} vs {opponent_name}"
                
                # Store match data
                match_data[match_id] = {
                    'match_id': match_id,
                    'match_name': match_name,
                    'teams': match_teams_in_target
                }
                
                # Add this match to each team's list
                for t in match_teams_in_target:
                    if t not in team_matches:
                        team_matches[t] = []
                    team_matches[t].append(match_id)
                
                # Create output folder for this match
                match_folder = os.path.join(output_dir, f"match_{match_id}")
                os.makedirs(match_folder, exist_ok=True)
                
                # Create KDE plot for the entire match (all teams, all periods)
                # First gather all recovery time data
                all_data = []
                team_period_data = {}
                
                for current_team in match_teams:
                    # Skip teams not in our target list
                    if current_team not in target_teams:
                        continue
                    
                    # Initialize team data
                    team_period_data[current_team] = {}
                    
                    # Get the team color
                    team_color = team_colors.get(current_team, "#808080")
                    
                    # Process each numeric period
                    combined_data = pd.DataFrame()
                    
                    for p in numeric_periods:
                        period_data = df_main[(df_main['team'] == current_team) & (df_main['period'] == p)]
                        
                        # Store period data if available
                        if len(period_data) >= 3:
                            recovery_times = period_data['time_to_recover'].dropna().values
                            team_period_data[current_team][p] = recovery_times
                            combined_data = pd.concat([combined_data, period_data])
                            
                            # Add to all data collection for match-wide KDE
                            all_data.append({
                                'team': current_team,
                                'period': p,
                                'color': team_color,
                                'data': recovery_times
                            })
                    
                    # Add combined "All periods" data if we have anything
                    if not combined_data.empty and len(combined_data) >= 3:
                        recovery_times = combined_data['time_to_recover'].dropna().values
                        team_period_data[current_team]["All"] = recovery_times
                        # Add to all data collection
                        all_data.append({
                            'team': current_team,
                            'period': "All",
                            'color': team_color,
                            'data': recovery_times
                        })
                
                # Create match-wide KDE plot comparing all periods for all teams
                if all_data:
                    # Sort data entries to ensure consistent order (teams, then periods)
                    all_data.sort(key=lambda x: (x['team'], 0 if x['period'] == "All" else x['period']))
                    
                    # Create KDE figure
                    fig_kde = go.Figure()
                    
                    # Process each team
                    for team in set(entry['team'] for entry in all_data):
                        team_color = next(entry['color'] for entry in all_data if entry['team'] == team)
                        team_data = [entry for entry in all_data if entry['team'] == team]
                        
                        # First add histogram with low opacity for each team's "All Periods" data only
                        all_periods_data = next((entry for entry in team_data if entry['period'] == "All"), None)
                        if all_periods_data:
                            fig_kde.add_trace(go.Histogram(
                                x=all_periods_data['data'],
                                histnorm='probability density',
                                name=f"{team} (Histogram)",
                                marker=dict(
                                    color=team_color, 
                                    opacity=0.2,
                                    line=dict(color=team_color, width=1)
                                ),
                                legendgroup=team,
                                legendgrouptitle=dict(text=team),
                                showlegend=True
                            ))
                        
                        # Then update the part where KDE traces are added for each period:
                        for data_entry in team_data:
                            period = data_entry['period']
                            period_label = "All Periods" if period == "All" else f"Period {period}"
                            
                            # Create KDE
                            x_range = np.linspace(min(data_entry['data']) - 0.5, max(data_entry['data']) + 0.5, 1000)
                            kde = stats.gaussian_kde(data_entry['data'])
                            
                            # Use the defined dash styles and line widths based on period
                            dash_style = period_dash_styles[period]
                            line_width = period_line_widths[period]
                            
                            fig_kde.add_trace(go.Scatter(
                                x=x_range,
                                y=kde(x_range),
                                mode='lines',
                                name=f"{team} - {period_label}",
                                line=dict(color=team_color, width=line_width, dash=dash_style),
                                legendgroup=team
                            ))
                            
                            # Add data points as scatter for "All Periods" only
                            if period == "All":
                                # Create y-positions that follow the KDE curve
                                scatter_y = []
                                for t in data_entry['data']:
                                    # Find closest point in x_range
                                    idx = np.abs(x_range - t).argmin()
                                    # Get corresponding y-value from KDE and adjust with random component
                                    y_val = kde(x_range)[idx] * np.random.uniform(0.05, 0.15)
                                    scatter_y.append(y_val)
                                
                                fig_kde.add_trace(go.Scatter(
                                    x=data_entry['data'],
                                    y=scatter_y,
                                    mode='markers',
                                    name=f"{team} - Data Points",  # Include team name
                                    marker=dict(
                                        color=team_color, 
                                        size=8, 
                                        symbol='circle-open',
                                        line=dict(width=1.5, color=team_color)
                                    ),
                                    legendgroup=team,
                                    showlegend=True  # Show in legend
                                ))

                        # Add vertical line at mean for each team's "All Periods" data
                        all_periods_data = next((entry for entry in team_data if entry['period'] == "All"), None)
                        if all_periods_data:
                            mean_val = np.mean(all_periods_data['data'])
                            
                            # Create KDE for this data to get max height
                            x_range = np.linspace(min(all_periods_data['data']) - 0.5, max(all_periods_data['data']) + 0.5, 1000)
                            kde = stats.gaussian_kde(all_periods_data['data'])
                            max_kde_height = max(kde(x_range))
                            
                            # Use the assigned mean color for this team
                            mean_color = team_mean_colors.get(team, "#808080")  # Default to gray if team not found
                            
                            fig_kde.add_trace(go.Scatter(
                                x=[mean_val, mean_val],
                                y=[0, max_kde_height],
                                mode='lines',
                                name=f"{team} - Mean",
                                line=dict(color=mean_color, width=2, dash='dot'),
                                legendgroup=team,
                                showlegend=True
                            ))
                    
                    # Update layout
                    fig_kde.update_layout(
                        title=f"Recovery Time Distributions: {match_name}",
                        xaxis_title="Recovery Time (seconds)",
                        yaxis_title="Density",
                        legend_title="Teams and Periods",
                        template="plotly_white",
                        hovermode="closest",
                        width=1100,
                        height=700,
                        legend=dict(
                            groupclick="toggleitem",  # Changed from togglegroup to toggleitem
                            tracegroupgap=10,
                            orientation="h",
                            y=-0.2,  # Adjusted position to accommodate more legend items
                            x=0.5,
                            xanchor="center"
                        )
                    )
                    
                    # Save the KDE plot
                    kde_file = os.path.join(match_folder, f"match_{match_id}_kde_plot.html")
                    pio.write_html(fig_kde, kde_file)
                    print(f"Created KDE plot for Match {match_id}")
                    
                # Process each team in the match
                for current_team in match_teams:
                    # Skip teams not in our target list
                    if current_team not in target_teams:
                        print(f"Skipping {current_team} (not in target teams list)")
                        continue
                        
                    # Get the team color
                    team_color = team_colors.get(current_team, "#808080")  # Default to gray if not found
                    
                    # Prepare combined data across all periods
                    combined_data = pd.DataFrame()
                    
                    # Find available periods with data
                    available_periods = []
                    
                    for p in numeric_periods:
                        period_data = df_main[(df_main['team'] == current_team) & (df_main['period'] == p)]
                        
                        # Only include periods with enough data
                        has_data = len(period_data) >= 3
                        
                        if has_data:
                            available_periods.append(p)
                            combined_data = pd.concat([combined_data, period_data])
                    
                    # Add "All" periods if we have data for any period
                    if len(combined_data) >= 3:
                        available_periods.append("All")
                    
                    # Sort periods (numeric first, then "All")
                    available_periods.sort(key=lambda x: float('inf') if x == "All" else x)
                    
                    if not available_periods:
                        print(f"No data available for {current_team} in any period in Match ID {match_id}")
                        continue
                    
                    # Create team-specific KDE comparing all periods
                    if current_team in team_period_data and len(team_period_data[current_team]) > 0:
                        fig_team_kde = go.Figure()
                        
                        # First add histograms (with lower opacity)
                        for period, data in team_period_data[current_team].items():
                            period_label = "All Periods" if period == "All" else f"Period {period}"
                            
                            # Add histogram with low opacity
                            fig_team_kde.add_trace(go.Histogram(
                                x=data,
                                histnorm='probability density',
                                name=f'Histogram: {period_label}',
                                marker=dict(
                                    color=team_color, 
                                    opacity=0.2,
                                    line=dict(color=team_color, width=1)
                                ),
                                legendgroup=f'period_{period}',
                                legendgrouptitle=dict(text=period_label),
                                showlegend=(period == "All")  # Only show histogram for "All" to reduce legend clutter
                            ))
                        
                        # Then add KDE curves
                        for period, data in team_period_data[current_team].items():
                            # Create KDE
                            x_range = np.linspace(min(data) - 0.5, max(data) + 0.5, 1000)
                            kde = stats.gaussian_kde(data)
                            
                            period_label = "All Periods" if period == "All" else f"Period {period}"
                            
                            # Use solid line for "All Periods", dashed for individual periods
                            dash_style = 'solid' if period == "All" else 'dash'
                            line_width = 3 if period == "All" else 2
                            
                            fig_team_kde.add_trace(go.Scatter(
                                x=x_range,
                                y=kde(x_range),
                                mode='lines',
                                name=f'KDE: {period_label}',
                                line=dict(color=team_color, width=line_width, dash=dash_style),
                                legendgroup=f'period_{period}'
                            ))
                            
                            # Add scatter points for actual data
                            # Create y-positions for scatter that follow the KDE curve
                            scatter_y = []
                            for t in data:
                                # Find closest point in x_range
                                idx = np.abs(x_range - t).argmin()
                                # Get corresponding y-value from KDE and adjust with a small random component for visibility
                                y_val = kde(x_range)[idx] * np.random.uniform(0.05, 0.15)  
                                scatter_y.append(y_val)
                            
                            fig_team_kde.add_trace(go.Scatter(
                                x=data,
                                y=scatter_y,
                                mode='markers',
                                name=f'Data Points: {period_label}',
                                marker=dict(
                                    color=team_color, 
                                    size=7, 
                                    symbol='circle-open',
                                    line=dict(width=1, color=team_color)
                                ),
                                legendgroup=f'period_{period}',
                                showlegend=(period == "All")  # Only show for "All" to reduce legend clutter
                            ))
                            
                            # Add normal distribution overlay for comparison
                            mean_val = np.mean(data)
                            std_dev = np.std(data)
                            normal_y = stats.norm.pdf(x_range, mean_val, std_dev)
                            
                            # Scale normal distribution to match KDE height
                            scale_factor = max(kde(x_range)) / max(normal_y)
                            
                            fig_team_kde.add_trace(go.Scatter(
                                x=x_range,
                                y=normal_y * scale_factor,
                                mode='lines',
                                name=f'Normal Dist: {period_label}',
                                line=dict(color=team_color, width=1, dash='dot'),
                                legendgroup=f'period_{period}'
                            ))
                            
                            # Add vertical line at mean for main periods only
                            if period == "All":  # Only add vertical lines for All Periods to avoid clutter
                                fig_team_kde.add_trace(go.Scatter(
                                    x=[mean_val, mean_val],
                                    y=[0, max(kde(x_range))],
                                    mode='lines',
                                    name=f'Mean: {mean_val:.2f}s',
                                    line=dict(color='green', dash='dot', width=2),
                                    legendgroup='stats',
                                    legendgrouptitle=dict(text="Statistics")
                                ))
                                
                                # Add vertical line at median
                                median_val = np.median(data)
                                fig_team_kde.add_trace(go.Scatter(
                                    x=[median_val, median_val],
                                    y=[0, max(kde(x_range))],
                                    mode='lines',
                                    name=f'Median: {median_val:.2f}s',
                                    line=dict(color='blue', dash='dot', width=2),
                                    legendgroup='stats'
                                ))
                        
                        # Update layout
                        fig_team_kde.update_layout(
                            title=f"Recovery Time Distributions: {current_team} ({match_name})",
                            xaxis_title="Recovery Time (seconds)",
                            yaxis_title="Density",
                            legend_title="Period",
                            template="plotly_white",
                            width=1000,
                            height=600,
                            legend=dict(
                                groupclick="togglegroup",
                                tracegroupgap=10,
                                orientation="h",
                                y=-0.2,
                                x=0.5,
                                xanchor="center"
                            )
                        )
                        
                        # Save the team KDE plot
                        team_kde_file = os.path.join(
                            match_folder, 
                            f"{current_team.replace(' ', '_').lower()}_kde_plot.html"
                        )
                        pio.write_html(fig_team_kde, team_kde_file)
                        print(f"Created KDE plot for {current_team} in Match {match_id}")
                        
                    # Process each period
                    for period in available_periods:
                        # Initialize period results
                        if period not in normality_results[match_id]['periods']:
                            normality_results[match_id]['periods'][period] = {}
                        
                        # Get data for this period
                        if period == "All":
                            team_data = combined_data
                        else:
                            team_data = df_main[(df_main['team'] == current_team) & (df_main['period'] == period)]
                        
                        # Skip if no data for this period
                        if len(team_data) < 3:
                            print(f"No data or insufficient data for {current_team} in Period {period}")
                            normality_results[match_id]['periods'][period][current_team] = {
                                'has_data': False,
                                'sample_size': len(team_data) if not team_data.empty else 0
                            }
                            continue
                        
                        # Get recovery time data
                        recovery_times = team_data['time_to_recover'].dropna().values
                        
                        if len(recovery_times) < 3:
                            print(f"Too few samples for {current_team} in Period {period} (n={len(recovery_times)})")
                            normality_results[match_id]['periods'][period][current_team] = {
                                'has_data': False,
                                'sample_size': len(recovery_times)
                            }
                            continue
                        
                        # Perform Shapiro-Wilk test
                        shapiro_test = stats.shapiro(recovery_times)
                        is_normal = shapiro_test.pvalue > 0.05
                        
                        # Calculate statistics
                        mean_val = np.mean(recovery_times)
                        median_val = np.median(recovery_times)
                        std_dev = np.std(recovery_times)
                        
                        # Store results
                        normality_results[match_id]['periods'][period][current_team] = {
                            'has_data': True,
                            'shapiro_stat': shapiro_test.statistic,
                            'shapiro_pvalue': shapiro_test.pvalue,
                            'is_normal': is_normal,
                            'sample_size': len(recovery_times),
                            'mean': mean_val,
                            'median': median_val,
                            'std_dev': std_dev
                        }
                        
                        # Create QQ plot data
                        qq = stats.probplot(recovery_times, dist="norm")
                        x = qq[0][0]  # x-points
                        y = qq[0][1]  # y-points
                        slope = qq[1][0]
                        intercept = qq[1][1]
                        
                        # Create combined QQ plot and KDE plot for this period and team
                        fig = make_subplots(
                            rows=1, cols=2,
                            subplot_titles=["QQ Plot", "Density Plot (KDE)"],
                            column_widths=[0.45, 0.55]  # Give more space to KDE plot
                        )
                        
                        # Add QQ plot data points
                        fig.add_trace(
                            go.Scatter(
                                x=x, 
                                y=y, 
                                mode='markers',
                                name='Data Points',
                                marker=dict(color=team_color, size=8),
                                legendgroup='qq',
                                legendgrouptitle=dict(text="QQ Plot")
                            ),
                            row=1, col=1
                        )
                        
                        # Add QQ plot reference line
                        fig.add_trace(
                            go.Scatter(
                                x=x,
                                y=slope * x + intercept,
                                mode='lines',
                                name='Reference Line',
                                line=dict(color='red', width=2),
                                legendgroup='qq',
                            ),
                            row=1, col=1
                        )
                        
                        # Create KDE for the second subplot
                        x_range = np.linspace(min(recovery_times) - 0.5, max(recovery_times) + 0.5, 1000)
                        kde = stats.gaussian_kde(recovery_times)
                        
                        # Add KDE curve
                        fig.add_trace(
                            go.Scatter(
                                x=x_range,
                                y=kde(x_range),
                                mode='lines',
                                name='KDE (Density)',
                                line=dict(color=team_color, width=3),
                                legendgroup='kde',
                                legendgrouptitle=dict(text="Density Plot")
                            ),
                            row=1, col=2
                        )
                        
                        # Add histogram of actual data points under the KDE
                        fig.add_trace(
                            go.Histogram(
                                x=recovery_times,
                                histnorm='probability density',
                                name='Data Histogram',
                                marker=dict(color=team_color, opacity=0.3),
                                legendgroup='kde',
                                showlegend=True
                            ),
                            row=1, col=2
                        )
                        
                        # Add normal distribution curve for comparison
                        normal_y = stats.norm.pdf(x_range, mean_val, std_dev)
                        
                        # Scale normal distribution to match KDE height
                        scale_factor = max(kde(x_range)) / max(normal_y)
                        
                        fig.add_trace(
                            go.Scatter(
                                x=x_range,
                                y=normal_y * scale_factor,
                                mode='lines',
                                name='Normal Distribution',
                                line=dict(color='black', dash='dash', width=2),
                                legendgroup='kde',
                            ),
                            row=1, col=2
                        )
                        
                        # Add vertical line at mean
                        fig.add_trace(
                            go.Scatter(
                                x=[mean_val, mean_val],
                                y=[0, max(kde(x_range))],
                                mode='lines',
                                name=f'Mean: {mean_val:.2f}s',
                                line=dict(color='green', dash='dot', width=2),
                                legendgroup='stats',
                                legendgrouptitle=dict(text="Statistics")
                            ),
                            row=1, col=2
                        )
                        
                        # Add vertical line at median
                        fig.add_trace(
                            go.Scatter(
                                x=[median_val, median_val],
                                y=[0, max(kde(x_range))],
                                mode='lines',
                                name=f'Median: {median_val:.2f}s',
                                line=dict(color='blue', dash='dot', width=2),
                                legendgroup='stats',
                            ),
                            row=1, col=2
                        )
                        
                        # Add data points as a scatter plot on top of KDE
                        # Create y-positions for scatter plot that follow the KDE curve
                        scatter_y = []
                        for t in recovery_times:
                            # Find closest point in x_range
                            idx = np.abs(x_range - t).argmin()
                            # Get corresponding y-value from KDE and adjust with a small random component for visibility
                            y_val = kde(x_range)[idx] * np.random.uniform(0.1, 0.3)  
                            scatter_y.append(y_val)
                        
                        fig.add_trace(
                            go.Scatter(
                                x=recovery_times,
                                y=scatter_y,
                                mode='markers',
                                name='Individual Data Points',
                                marker=dict(color=team_color, size=9, symbol='circle-open', line=dict(width=2, color=team_color)),
                                legendgroup='kde',
                            ),
                            row=1, col=2
                        )
                        
                        # Update layout
                        period_label = "All Periods" if period == "All" else f"Period {period}"
                        fig.update_layout(
                            title=f"Normality Analysis: {current_team} - {period_label} (p-value: {shapiro_test.pvalue:.4f})",
                            showlegend=True,
                            height=600,
                            width=1100,
                            template="plotly_white",
                            legend=dict(
                                groupclick="toggleitem",
                                tracegroupgap=5,
                                orientation="h",
                                y=-0.2,
                                x=0.5,
                                xanchor="center"
                            )
                        )
                        
                        # Update subplot axes
                        fig.update_xaxes(title_text="Theoretical Quantiles", row=1, col=1)
                        fig.update_yaxes(title_text="Sample Quantiles", row=1, col=1)
                        fig.update_xaxes(title_text="Recovery Time (seconds)", row=1, col=2)
                        fig.update_yaxes(title_text="Density", row=1, col=2)
                        
                        # Save the individual plot
                        period_str = "all_periods" if period == "All" else f"period_{period}"
                        plot_file = os.path.join(
                            match_folder, 
                            f"{current_team.replace(' ', '_').lower()}_{period_str}_qq_kde_plot.html"
                        )
                        pio.write_html(fig, plot_file)
                        
                        # Determine status based on normality and sample size
                        status = ""
                        if len(recovery_times) < 8:
                            status = "warning"  # Small sample size (yellow)
                        elif is_normal:
                            status = "normal"   # Normal distribution (green)
                        else:
                            status = "not-normal"  # Not normal (red)
                        
                        # Get opponent team
                        other_teams = [t for t in match_teams if t != current_team]
                        opponent = other_teams[0] if other_teams else "Unknown"
                        
                        # Store all result data for the filtering feature
                        results_data.append({
                            'match_id': match_id,
                            'match_name': match_name,
                            'team': current_team,
                            'team_color': team_color,
                            'opponent': opponent,
                            'period': "All" if period == "All" else period,
                            'sample_size': len(recovery_times),
                            'shapiro_pvalue': shapiro_test.pvalue,
                            'is_normal': is_normal,
                            'status': status,
                            'mean': mean_val,
                            'median': median_val,
                            'std_dev': std_dev,
                            'plot_path': plot_file.replace(output_dir + os.sep, ''),
                            'kde_match_path': f"match_{match_id}/match_{match_id}_kde_plot.html",
                            'match_plots_path': f"match_{match_id}/match_{match_id}_all_periods.html",
                            'match_stats_path': f"match_{match_id}/{current_team.replace(' ', '_').lower()}_analysis.html" 
                        })
                
            except Exception as e:
                print(f"Error processing Match ID {match_id}: {e}")
                import traceback
                traceback.print_exc()
    
    # Create final HTML with sequential filtering
    html_content = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>Recovery Time Normality Test Results</title>
        <style>
        :root {
            --primary: #1e88e5;
            --secondary: #26a69a;
            --success: #43a047;
            --danger: #e53935;
            --warning: #ffb300;
            --light: #f5f5f5;
            --dark: #2c3e50;
            --background: #ffffff;
        }
            body {
                font-family: Arial, sans-serif;
                line-height: 1.6;
                margin: 0;
                padding: 20px;
                color: var(--dark);
                background-color: var(--light);
            }
            .container {
                max-width: 1200px;
                margin: 0 auto;
                background-color: var(--background);
                box-shadow: 0 4px 20px rgba(0,0,0,0.1);
                border-radius: 8px;
                padding: 25px;
            }
            button, .view-btn {
                background-color: var(--primary);
                transition: all 0.2s;
            }
            button:hover, .view-btn:hover {
                background-color: #1565c0;
                transform: translateY(-2px);
            }
            .normal { background-color: var(--success); }
            .not-normal { background-color: var(--danger); }
            .warning { background-color: var(--warning); }
            h1, h2, h3 {
                color: #2c3e50;
            }
            table {
                border-collapse: collapse;
                width: 100%;
                margin-top: 20px;
            }
            th, td {
                border: 1px solid #ddd;
                padding: 8px 12px;
                text-align: left;
            }
            th {
                background-color: #f2f2f2;
                position: sticky;
                top: 0;
                z-index: 10;
            }
            tbody tr:nth-child(even) {
                background-color: #f9f9f9;
            }
            tbody tr:hover {
                background-color: #f1f1f1;
            }
            .team-color {
                display: inline-block;
                width: 15px;
                height: 15px;
                margin-right: 5px;
                border-radius: 50%;
                vertical-align: middle;
            }
            .status-indicator {
                display: inline-block;
                width: 10px;
                height: 10px;
                margin-right: 5px;
                border-radius: 50%;
            }


            label {
                display: inline-block;
                width: 80px;
                font-weight: bold;
            }
            .filters {
                background-color: #f8f9fa;
                padding: 20px;
                margin-bottom: 25px;
                border-radius: 8px;
                border: none;
                box-shadow: 0 2px 10px rgba(0,0,0,0.05);
            }
            .filter-group {
                display: flex;
                align-items: center;
                margin-bottom: 15px;
            }
            select {
                padding: 8px 12px;
                border-radius: 6px;
                border: 1px solid #ddd;
                min-width: 220px;
                box-shadow: inset 0 1px 3px rgba(0,0,0,0.05);
                transition: border-color 0.2s;
            }
            select:focus {
                border-color: var(--primary);
                outline: none;
            }
            #reset-filters {
                padding: 8px 18px;
                font-weight: 600;
                border-radius: 6px;
            }
            button {
                background-color: #4CAF50;
                color: white;
                padding: 8px 15px;
                border: none;
                border-radius: 4px;
                cursor: pointer;
                margin-top: 10px;
            }
            button:hover {
                background-color: #45a049;
            }
            .legend {
                display: flex;
                margin: 15px 0;
                flex-wrap: wrap;
            }
            .legend-item {
                display: flex;
                align-items: center;
                margin-right: 20px;
                margin-bottom: 5px;
            }
            .disabled {
                opacity: 0.6;
                cursor: not-allowed;
            }
            .step-indicator {
                display: inline-block;
                background-color: #ccc;
                color: white;
                border-radius: 50%;
                width: 20px;
                height: 20px;
                text-align: center;
                line-height: 20px;
                margin-right: 5px;
            }
            .current-step {
                background-color: #4CAF50;
            }
            .btn-group {
                margin-top: 10px;
            }
            .btn-group button {
                margin-right: 10px;
            }
            .button-legend {
                background-color: white;
                padding: 20px;
                margin: 20px 0;
                border-radius: 8px;
                border: none;
                box-shadow: 0 2px 10px rgba(0,0,0,0.05);
            }
            .button-info {
                display: flex;
                align-items: flex-start;
                margin-bottom: 15px;
            }
            .button-sample {
                background-color: var(--primary);
                color: white;
                padding: 6px 12px;
                border-radius: 4px;
                margin-right: 20px;
                font-size: 12px;
                min-width: 90px;
                text-align: center;
                font-weight: 600;
            }
            .button-description {
                font-size: 14px;
                color: #555;
                line-height: 1.5;
            }
            .dashboard-header {
                text-align: center;
                margin-bottom: 30px;
                padding-bottom: 20px;
                border-bottom: 1px solid #eee;
            }
            .logo {
                font-size: 32px;
                margin-bottom: 10px;
            }
            .subtitle {
                color: #666;
                font-size: 16px;
                margin-top: 5px;
            }
            .stats-cards {
                display: grid;
                grid-template-columns: repeat(auto-fill, minmax(250px, 1fr));
                gap: 20px;
                margin-bottom: 30px;
            }
            .stat-card {
                background-color: white;
                border-radius: 8px;
                padding: 20px;
                box-shadow: 0 4px 10px rgba(0,0,0,0.05);
                display: flex;
                align-items: center;
                transition: transform 0.2s, box-shadow 0.2s;
            }
            .stat-card:hover {
                transform: translateY(-3px);
                box-shadow: 0 6px 12px rgba(0,0,0,0.1);
            }
            .stat-icon {
                font-size: 28px;
                margin-right: 15px;
            }
            .stat-value {
                font-size: 24px;
                font-weight: bold;
                color: var(--primary);
            }
            .stat-label {
                color: #666;
                font-size: 14px;
            }
            #reset-filters, 
            .reset-filters, 
            button[id="reset-filters"], 
            button.reset-filters {
                background-color: var(--primary, #1e88e5);
                color: white;
                border: none;
                transition: all 0.2s;
            }

            #reset-filters:hover, 
            .reset-filters:hover, 
            button[id="reset-filters"]:hover, 
            button.reset-filters:hover {
                background-color: #1565c0;
                transform: translateY(-2px);
            }
            .step-indicator {
                display: inline-block;
                background-color: #ccc;  /* Inactive color remains gray */
                color: white;
                border-radius: 50%;
                width: 20px;
                height: 20px;
                text-align: center;
                line-height: 20px;
                margin-right: 5px;
            }

            /* Change active indicator from green to blue */
            .current-step {
                background-color: var(--primary, #1e88e5);  /* Use blue instead of #4CAF50 */
            }

            /* Optional: add a subtle hover effect */
            .step-indicator:hover {
                opacity: 0.9;
}
/* Enhanced Table Styling */
table#results-table {
    border-collapse: separate;
    border-spacing: 0;
    width: 100%;
    margin-top: 25px;
    border-radius: 8px;
    overflow: hidden;
    box-shadow: 0 2px 10px rgba(0,0,0,0.08);
    font-size: 14px;
}

#results-table th {
    background-color: var(--primary, #1e88e5);
    color: white;
    position: sticky;
    top: 0;
    z-index: 10;
    padding: 14px 16px;
    font-weight: 600;
    text-transform: uppercase;
    font-size: 13px;
    letter-spacing: 0.5px;
    text-align: left;
    border-bottom: 2px solid rgba(0,0,0,0.05);
}

#results-table td {
    padding: 12px 16px;
    border-bottom: 1px solid #eee;
    vertical-align: middle;
}

#results-table tbody tr:nth-child(even) {
    background-color: rgba(0,0,0,0.02);
}

#results-table tbody tr:hover {
    background-color: rgba(30, 136, 229, 0.05);
}

/* Keep specific styling for the status indicators */
.status-indicator {
    display: inline-block;
    width: 12px;
    height: 12px;
    margin-right: 8px;
    border-radius: 50%;
    vertical-align: middle;
}

/* Team color indicator enhancement */
.team-color {
    display: inline-block;
    width: 16px;
    height: 16px;
    margin-right: 8px;
    border-radius: 50%;
    vertical-align: middle;
    box-shadow: 0 0 4px rgba(0,0,0,0.2);
}

/* View Plots column - preserve arrangement but enhance buttons */
#results-table td:last-child {
    text-align: right;
    padding-right: 12px;
}

/* Enhance the buttons without changing arrangement */
.btn-group {
    display: inline-flex;
    gap: 6px;
}

.view-btn {
    padding: 6px 10px;
    font-size: 12px;
    font-weight: 500;
    border-radius: 4px;
    border: none;
    background-color: var(--primary, #1e88e5);
    color: white;
    cursor: pointer;
    transition: all 0.2s;
    box-shadow: 0 1px 3px rgba(0,0,0,0.1);
    text-align: center;
}

.view-btn:hover {
    background-color: #1565c0;
    transform: translateY(-1px);
    box-shadow: 0 2px 5px rgba(0,0,0,0.15);
}
/* Help panel styling */
.help-container {
  position: relative;
  margin: 20px 0;
}

.help-button {
  background-color: var(--primary, #1e88e5);
  color: white;
  border: none;
  border-radius: 4px;
  padding: 8px 16px;
  display: flex;
  align-items: center;
  gap: 8px;
  cursor: pointer;
  font-size: 14px;
  transition: all 0.2s;
}

.help-button:hover {
  background-color: #1565c0;
  transform: translateY(-1px);
}

.help-icon {
  font-size: 16px;
}

.help-panel {
  position: absolute;
  top: 100%;
  right: 0;
  background-color: white;
  border-radius: 8px;
  box-shadow: 0 4px 20px rgba(0,0,0,0.15);
  width: 600px;
  max-width: 90vw;
  z-index: 100;
  max-height: 80vh;
  overflow-y: auto;
}

.help-header {
  display: flex;
  justify-content: space-between;
  align-items: center;
  padding: 15px 20px;
  border-bottom: 1px solid #eee;
}

.help-header h2 {
  margin: 0;
  font-size: 20px;
  color: var(--primary, #1e88e5);
}

.close-button,
.close-button:focus,
.close-button:hover,
.close-button:active {
  outline: none;
  box-shadow: none;
  background: transparent;   
  color: #666;               
}

.accordion {
  padding: 15px;
}

.accordion-item {
  margin-bottom: 10px;
  border: 1px solid #eee;
  border-radius: 6px;
  overflow: hidden;
}

.accordion-header {
  background-color: #f8f9fa;
  padding: 12px 15px;
  font-weight: 600;
  cursor: pointer;
  display: flex;
  align-items: center;
  gap: 10px;
  transition: background-color 0.2s;
}

.accordion-header:hover {
  background-color: #f1f3f5;
}

.accordion-icon {
  font-size: 18px;
}

.accordion-content {
  padding: 15px;
  background-color: white;
}

.help-list {
  padding-left: 20px;
  margin: 10px 0;
}

.help-list li {
  margin-bottom: 8px;
  line-height: 1.5;
}

.help-note {
  background-color: #f8f9fa;
  padding: 10px;
  border-radius: 4px;
  border-left: 4px solid var(--primary, #1e88e5);
  margin: 15px 0 5px 0;
}

/* Team Analysis Navigation */
.team-analytics-navigation {
    background-color: white;
    border-radius: 8px;
    padding: 20px;
    margin: 20px 0;
    box-shadow: 0 2px 10px rgba(0,0,0,0.08);
}
.team-buttons {
    display: flex;
    flex-wrap: wrap;
    gap: 10px;
    margin-top: 15px;
}
.team-btn {
    background-color: var(--primary);
    color: white;
    padding: 10px 20px;
    border-radius: 6px;
    border: none;
    cursor: pointer;
    transition: all 0.2s;
    font-weight: 500;
    min-width: 180px;
    text-align: center;
}
.team-btn:hover {
    background-color: #1565c0;
    transform: translateY(-2px);
    box-shadow: 0 4px 8px rgba(0,0,0,0.1);
}
        </style>
    </head>
    <body>
        <div class="container">
            <header class="dashboard-header">
                <div class="logo">⚽📊</div>
                <h1>Football Recovery Analytics Dashboard</h1>
                <div class="subtitle">Performance analysis of recovery times across matches and periods</div>
            </header>
            <!-- Add this where you want the help button to appear (perhaps in the header or near the filters) -->
<div class="help-container">
  <button id="toggleHelp" class="help-button">
    <i class="help-icon">❓</i> Help & Documentation
  </button>
  
  <div id="helpPanel" class="help-panel" style="display: none;">
    <div class="help-header">
      <h2>Analysis Documentation</h2>
      <button id="closeHelp" class="close-button">×</button>
    </div>
    
    <div class="accordion">
      <div class="accordion-item">
        <div class="accordion-header">
          <span class="accordion-icon">📊</span> Interpretation
        </div>
        <div class="accordion-content">
          <ul class="help-list">
            <li><strong>Shapiro-Wilk p-value > 0.05:</strong> Data can be considered normally distributed</li>
            <li><strong>Shapiro-Wilk p-value <= 0.05:</strong> Data significantly deviates from normal distribution</li>
            <li><strong>Warning:</strong> Results for small sample sizes (n < 8) should be interpreted with caution</li>
          </ul>
          <p class="help-note"><strong>Note:</strong> The "All Periods" analysis combines data across all periods to provide a more comprehensive assessment with larger sample sizes.</p>
        </div>
      </div>
      
      <div class="accordion-item">
        <div class="accordion-header">
          <span class="accordion-icon">📈</span> QQ Plots
        </div>
        <div class="accordion-content">
          <p>QQ plots compare the observed data distribution to the theoretical normal distribution:</p>
          <ul class="help-list">
            <li>Points following the red reference line closely suggest a normal distribution</li>
            <li>Systematic deviations from the line suggest non-normality</li>
            <li>S-shaped patterns indicate skewness in the data</li>
          </ul>
        </div>
      </div>
      
      <div class="accordion-item">
        <div class="accordion-header">
          <span class="accordion-icon">🔍</span> KDE Plots
        </div>
        <div class="accordion-content">
          <p>Kernel Density Estimation (KDE) plots show the estimated probability density of recovery times:</p>
          <ul class="help-list">
            <li>Bell-shaped curves suggest normal distributions</li>
            <li>Multi-modal curves (multiple peaks) suggest mixed distributions</li>
            <li>Skewed curves (asymmetric) indicate non-normal distributions</li>
            <li>Dotted line shows theoretical normal distribution for comparison</li>
            <li>Green vertical line shows mean, blue vertical line shows median</li>
          </ul>
        </div>
      </div>
      
      
      <div class="accordion-item">
        <div class="accordion-header">
          <span class="accordion-icon">⚽</span> Match Events
        </div>
        <div class="accordion-content">
          <p>These plots show match recovery time trends alongside match events:</p>
          <ul class="help-list">
            <li>Interactive timeline with team recovery times</li>
            <li>Match events (goals, cards, substitutions) overlaid on recovery data</li>
            <li>Team formation details and tactical shifts</li>
            <li>Toggle between different periods or view all periods</li>
          </ul>
        </div>
      </div>
    </div>
  </div>
</div>
            <div class="filters">
                <h3>Filter Results</h3>
                <div class="filter-group">
                    <span class="step-indicator current-step">1</span>
                    <label for="team-filter">Team:</label>
                    <select id="team-filter">
                        <option value="">-- Select a Team --</option>

            
    """
    
    # Add team options
    for team_name in sorted(target_teams):
        html_content += f'                        <option value="{team_name}">{team_name}</option>\n'
    
    html_content += """
                    </select>
                </div>
                <div class="filter-group">
                    <span class="step-indicator">2</span>
                    <label for="match-filter">Match:</label>
                    <select id="match-filter" disabled>
                        <option value="">-- Select a Match --</option>
                    </select>
                </div>
                <div class="filter-group">
                    <span class="step-indicator">3</span>
                    <label for="period-filter">Period:</label>
                    <select id="period-filter" disabled>
                        <option value="all">All Periods</option>
                        <option value="1">Period 1</option>
                        <option value="2">Period 2</option>
                        <option value="3">Period 3</option>
                        <option value="4">Period 4</option>
                        <option value="All">Combined (All Periods)</option>
                    </select>
                </div>
                <button id="reset-filters">Reset Filters</button>
            </div>
            <div class="team-analytics-navigation">
                <h3>Team Analysis Reports</h3>
                <p>Access detailed analytics reports for each team:</p>
                <div class="team-buttons">
"""

    # Add buttons for each team
    for team_name in sorted(target_teams):
        team_slug = team_name.replace(' ', '_').lower()
        html_content += f"""
                    <button onclick="window.open('team_analysis/{team_slug}_analysis.html', '_blank')" class="view-btn team-btn">
                        {team_name} Analysis
                    </button>
        """

    html_content += """
                </div>
            </div>
            <div class="button-legend">
                <h4>Visualization Buttons</h4>
                <div class="button-info">
                    <span class="button-sample">QQ+KDE</span>
                    <span class="button-description">Shows QQ plots and density plots for assessing normality of team recovery times for a specific period</span>
                </div>
                <div class="button-info">
                    <span class="button-sample">Match Events</span>
                    <span class="button-description">Shows interactive timeline plot and table with match events, team formations, and recovery time plots</span>
                </div>
                <div class="button-info">
                    <span class="button-sample">Match Stats</span>
                    <span class="button-description">Shows detailed statistical analysis for the team, including event impact analysis, normality tests, and significance of recovery time changes</span>
                </div>
            </div>
            <div class="stats-cards">
    <div class="stat-card">
        <div class="stat-icon">📊</div>
        <div class="stat-content">
            <div class="stat-value" id="total-records">0</div>
            <div class="stat-label">Total Records</div>
        </div>
    </div>
    <div class="stat-card">
        <div class="stat-icon">✅</div>
        <div class="stat-content">
            <div class="stat-value" id="normal-count">0</div>
            <div class="stat-label">Normal Distributions</div>
        </div>
    </div>
    <div class="stat-card">
        <div class="stat-icon">⚠️</div>
        <div class="stat-content">
            <div class="stat-value" id="warning-count">0</div>
            <div class="stat-label">Small Samples</div>
        </div>
    </div>
</div>
            <div class="legend">
                <div class="legend-item">
                    <span class="status-indicator normal"></span> Normal Distribution (p > 0.05)
                </div>
                <div class="legend-item">
                    <span class="status-indicator not-normal"></span> Not Normal (p <= 0.05)
                </div>
                <div class="legend-item">
                    <span class="status-indicator warning"></span> Small Sample (n < 8)
                </div>
            </div>
            
            <table id="results-table">
                <thead>
                    <tr>
                        <th>Match</th>
                        <th>Period</th>
                        <th>Team</th>
                        <th>Sample Size</th>
                        <th>Shapiro-Wilk p-value</th>
                        <th>Is Normal?</th>
                        <th>Mean (sec)</th>
                        <th>Median (sec)</th>
                        <th>Std Dev (sec)</th>
                        <th>View Plots and Tables</th>
                    </tr>
                </thead>
                <tbody>
    """
    
    # Add rows for each result
    for result in results_data:
        # Create the team slug for the file path
        team_slug = result['team'].replace(' ', '_').lower()
        # Update the match_stats_path to point to the team-specific analysis HTML
        stats_path = f"match_{result['match_id']}/{team_slug}_analysis.html"
        
        html_content += f"""
                        <tr data-match="{result['match_id']}" data-team="{result['team']}" data-period="{result['period']}">
                            <td>{result['match_name']}</td>
                            <td>{result['period']}</td>
                            <td>
                                <span class="team-color" style="background-color: {result['team_color']};"></span>
                                {result['team']}
                            </td>
                            <td>{result['sample_size']}</td>
                            <td>{result['shapiro_pvalue']:.4f}</td>
                            <td>
                                <span class="status-indicator {result['status']}"></span>
                                {"Yes" if result['is_normal'] else "No"}{" (small sample)" if result['sample_size'] < 8 else ""}
                            </td>
                            <td>{result['mean']:.2f}</td>
                            <td>{result['median']:.2f}</td>
                            <td>{result['std_dev']:.2f}</td>
                            <td>
                                <div class="btn-group">
                                    <button onclick="window.open('{result['plot_path']}', '_blank')" class="view-btn">QQ+KDE</button>
                                    <button onclick="window.open('{result['match_plots_path']}', '_blank')" class="view-btn">Match Events</button>
                                    <button onclick="window.open('{stats_path}', '_blank')" class="view-btn" title="View detailed team analytics">Match Stats</button>
                                </div>
                            </td>
                        </tr>
            """


    html_content += """
                </tbody>
            </table>
            
            <script>
            document.addEventListener('DOMContentLoaded', function() {
                // Get filter elements
                const teamFilter = document.getElementById('team-filter');
                const matchFilter = document.getElementById('match-filter');
                const periodFilter = document.getElementById('period-filter');
                const resetButton = document.getElementById('reset-filters');
                const table = document.getElementById('results-table');
                const rows = table.querySelectorAll('tbody tr');
                const stepIndicators = document.querySelectorAll('.step-indicator');
                
                // Store team-match relationships
                const teamMatches = JSON.parse('""" + json.dumps(team_matches) + """');
                const matchData = JSON.parse('""" + json.dumps(match_data) + """');
                // Help panel functionality
                const toggleHelpBtn = document.getElementById('toggleHelp');
                const closeHelpBtn = document.getElementById('closeHelp');
                const helpPanel = document.getElementById('helpPanel');


                toggleHelpBtn.addEventListener('click', function() {
                    helpPanel.style.display = helpPanel.style.display === 'none' ? 'block' : 'none';
                });
                
                closeHelpBtn.addEventListener('click', function() {
                    helpPanel.style.display = 'none';
                });

                
                // Team filter change event
                teamFilter.addEventListener('change', function() {
                    // Reset and disable subsequent filters
                    resetMatchFilter();
                    resetPeriodFilter();
                    
                    const selectedTeam = this.value;
                    
                    if (!selectedTeam) {
                        // Reset all to initial state if no team selected
                        resetAllFilters(false);
                        return;
                    }
                    
                    // Populate match filter with matches for this team
                    const matches = teamMatches[selectedTeam] || [];
                    
                    // Enable match filter
                    matchFilter.disabled = false;
                    matchFilter.innerHTML = '<option value="">-- Select a Match --</option>';
                    
                    // Add match options
                    matches.forEach(matchId => {
                        const match = matchData[matchId];
                        const option = document.createElement('option');
                        option.value = matchId;
                        option.textContent = match.match_name;
                        matchFilter.appendChild(option);
                    });
                    
                    // Update step indicators
                    stepIndicators[0].classList.remove('current-step');
                    stepIndicators[1].classList.add('current-step');
                    
                    // Filter table to show only rows with the selected team
                    filterTable();
                });
                
                // Match filter change event
                matchFilter.addEventListener('change', function() {
                    resetPeriodFilter();
                    
                    const selectedMatch = this.value;
                    
                    if (!selectedMatch) {
                        // If no match selected, only filter by team
                        periodFilter.disabled = true;
                        stepIndicators[1].classList.add('current-step');
                        stepIndicators[2].classList.remove('current-step');
                    } else {
                        // Enable period filter
                        periodFilter.disabled = false;
                        stepIndicators[1].classList.remove('current-step');
                        stepIndicators[2].classList.add('current-step');
                    }
                    
                    // Filter table
                    filterTable();
                });
                
                // Period filter change event
                periodFilter.addEventListener('change', function() {
                    // Filter table
                    filterTable();
                });
                
                // Reset button click event
                resetButton.addEventListener('click', function() {
                    resetAllFilters(true);
                });
                
                // Filter table based on current selections
                function filterTable() {
                    const selectedTeam = teamFilter.value;
                    const selectedMatch = matchFilter.value;
                    const selectedPeriod = periodFilter.value;
                    
                    rows.forEach(row => {
                        let showRow = true;
                        
                        // Apply team filter
                        if (selectedTeam && row.getAttribute('data-team') !== selectedTeam) {
                            showRow = false;
                        }
                        
                        // Apply match filter
                        if (selectedMatch && row.getAttribute('data-match') !== selectedMatch) {
                            showRow = false;
                        }
                        
                        // Apply period filter
                        if (selectedPeriod && selectedPeriod !== 'all' && row.getAttribute('data-period') !== selectedPeriod) {
                            showRow = false;
                        }
                        
                        row.style.display = showRow ? '' : 'none';
                    });
                    
                    // Update stats after filtering
                    updateStats();
                }
                
                // Reset match filter
                function resetMatchFilter() {
                    matchFilter.innerHTML = '<option value="">-- Select a Match --</option>';
                    matchFilter.disabled = true;
                }
                
                // Reset period filter
                function resetPeriodFilter() {
                    periodFilter.value = 'all';
                    periodFilter.disabled = true;
                }
                
                // Reset all filters
                function resetAllFilters(includeTeam) {
                    if (includeTeam) {
                        teamFilter.value = '';
                    }
                    
                    resetMatchFilter();
                    resetPeriodFilter();
                    
                    // Reset step indicators
                    stepIndicators.forEach((indicator, index) => {
                        indicator.classList.remove('current-step');
                        if (index === 0) {
                            indicator.classList.add('current-step');
                        }
                    });
                    
                    // Show all rows
                    rows.forEach(row => {
                        row.style.display = '';
                    });
                    
                    // Update stats after resetting
                    updateStats();
                }
                
                // Make rows clickable to open plots
                rows.forEach(row => {
                    row.style.cursor = 'pointer';
                    row.addEventListener('click', function(e) {
                        if (e.target.tagName !== 'BUTTON' && e.target.parentNode.tagName !== 'BUTTON') {
                            const qqKdeBtn = this.querySelector('.view-btn');
                            if (qqKdeBtn) {
                                window.open(qqKdeBtn.getAttribute('onclick').match(/'([^']+)'/)[1], '_blank');
                            }
                        }
                    });
                });
                
                // Function to update the stats cards
                function updateStats() {
                    const rows = document.querySelectorAll('#results-table tbody tr:not([style*="display: none"])');
                    const totalRecords = rows.length;
                    const normalCount = document.querySelectorAll('#results-table tbody tr:not([style*="display: none"]) .normal').length;
                    const warningCount = document.querySelectorAll('#results-table tbody tr:not([style*="display: none"]) .warning').length;
                    
                    document.getElementById('total-records').textContent = totalRecords;
                    document.getElementById('normal-count').textContent = normalCount;
                    document.getElementById('warning-count').textContent = warningCount;
                }
                
                // Initialize stats when page loads
                updateStats();
            });
            </script>
        </div>
    </body>
    </html>
    """
    
    # Write the final HTML file
    index_file = os.path.join(output_dir, "dashboard.html")
    with open(index_file, 'w', encoding='utf-8') as f:
        f.write(html_content)
    
    print(f"\nNormality test report created at: {index_file}")
    print("All normality tests completed!")
    add_explanations_to_all_plots(output_dir)
    
    return normality_results


def generate_html_files(team_dicts, all_dataframes, normality_results, mannwhitney_results, impact_results, match_level_tests, team_level_tests, output_dir="visuals"):
    """
    Generate HTML files for the analysis results from the two functions:
    - analyze_event_recovery_times
    - analyze_recovery_impact
    
    Args:
        team_dicts: Dictionary of team names and their match dictionaries
        all_dataframes: Dictionary of all dataframes from analyze_event_recovery_times
        normality_results: Dictionary of normality test results from analyze_event_recovery_times
        mannwhitney_results: Dictionary of Mann-Whitney test results from analyze_event_recovery_times
        impact_results: DataFrame of impact analysis results from analyze_recovery_impact
        match_level_tests: List of match-level test results with BH correction
        team_level_tests: List of team-level test results with BH correction
        output_dir: Directory to save HTML files
    """
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    # Create team_analysis directory
    team_dir = os.path.join(output_dir, "team_analysis")
    os.makedirs(team_dir, exist_ok=True)
    
    # Get list of teams and matches
    teams = [team_name for team_name, _ in team_dicts]
    
    # Dictionary to store match IDs and their details
    all_matches = {}
    for team_name, matches_dict in team_dicts:
        for match_id, match_details in matches_dict.items():
            if match_id not in all_matches:
                all_matches[match_id] = {
                    'teams': [team_name, match_details['team']]
                }
            
            # Create individual match directory
            match_dir = os.path.join(output_dir, f"match_{match_id}")
            os.makedirs(match_dir, exist_ok=True)
    
    # Create dictionaries to quickly look up BH correction results
    match_bh_lookup = {}
    team_bh_lookup = {}
    
    # Build lookup for match-level BH corrections
    for test in match_level_tests:
        key = f"{test['team'].replace(' ', '_').lower()}_match_{test['match_id']}_{test['event_type']}"
        match_bh_lookup[key] = {
            'bh_corrected_p': test.get('bh_corrected_p', 'N/A'),
            'significant_after_bh': test.get('significant_after_bh', False)
        }
    
    # Build lookup for team-level BH corrections
    for test in team_level_tests:
        key = f"{test['team'].replace(' ', '_').lower()}_{test['event_type']}"
        team_bh_lookup[key] = {
            'bh_corrected_p': test.get('bh_corrected_p', 'N/A'),
            'significant_after_bh': test.get('significant_after_bh', False)
        }
    
    # Define CSS styles
    css_styles = """
    <style>
        body {
            font-family: 'Segoe UI', Arial, sans-serif;
            line-height: 1.6;
            color: #333;
            margin: 0;
            padding: 20px;
            background-color: #f5f5f5;
        }
        .container {
            width: 100%;
            max-width: 100%;
            margin: 0 auto;
        }
        h1 {
            color: #1d3557;
            border-bottom: 2px solid #457b9d;
            padding-bottom: 10px;
            margin-top: 30px;
        }
        h2 {
            color: #1d3557;
            margin-top: 25px;
        }
        h3 {
            color: #457b9d;
            margin-top: 20px;
        }
        table {
            border-collapse: collapse;
            width: 100%;
            margin: 20px 0;
            background-color: white;
            box-shadow: 0 1px 3px rgba(0,0,0,0.12), 0 1px 2px rgba(0,0,0,0.24);
            table-layout: auto;
        }
        th, td {
            text-align: left;
            padding: 12px;
            border-bottom: 1px solid #ddd;
            white-space: nowrap;
        }
        th {
            background-color: #457b9d;
            color: white;
            position: sticky;
            top: 0;
            z-index: 10;
        }
        tr:hover {
            background-color: #f1f1f1;
        }
        .positive {
            color: #2a9d8f;
            font-weight: bold;
        }
        .negative {
            color: #e63946;
            font-weight: bold;
        }
        .highlight {
            background-color: #ffffd9;
        }
        .highlight-bh {
            background-color: #e8f5e9;
        }
        .summary-box {
            background-color: #e1f3f8;
            border-left: 5px solid #457b9d;
            padding: 15px;
            margin: 20px 0;
            border-radius: 0 5px 5px 0;
        }
        .event-card {
            background-color: white;
            border-radius: 5px;
            padding: 15px;
            margin: 15px 0;
            box-shadow: 0 1px 3px rgba(0,0,0,0.12), 0 1px 2px rgba(0,0,0,0.24);
        }
        .footer {
            margin-top: 40px;
            padding-top: 20px;
            border-top: 1px solid #ddd;
            text-align: center;
            font-size: 0.9em;
            color: #777;
        }
        .chart-container {
            background-color: white;
            border-radius: 5px;
            padding: 20px;
            margin: 20px 0;
            box-shadow: 0 1px 3px rgba(0,0,0,0.12), 0 1px 2px rgba(0,0,0,0.24);
            height: 400px;
            position: relative;
        }
        /* Help button styles */
        .help-button {
            position: fixed;
            bottom: 30px;
            right: 30px;
            background-color: #457b9d;
            color: white;
            border: none;
            border-radius: 50%;
            width: 60px;
            height: 60px;
            font-size: 24px;
            cursor: pointer;
            box-shadow: 0 2px 5px rgba(0,0,0,0.3);
            z-index: 1000;
            display: flex;
            align-items: center;
            justify-content: center;
        }
        .help-button:hover {
            background-color: #1d3557;
        }
        .help-modal {
            display: none;
            position: fixed;
            top: 0;
            left: 0;
            right: 0;
            bottom: 0;
            background-color: rgba(0,0,0,0.5);
            z-index: 1001;
            align-items: center;
            justify-content: center;
        }
        .help-content {
            background-color: white;
            padding: 30px;
            border-radius: 8px;
            max-width: 800px;
            max-height: 80vh;
            overflow-y: auto;
            box-shadow: 0 3px 7px rgba(0,0,0,0.3);
        }
        .help-content h2 {
            color: #1d3557;
            margin-top: 0;
        }
        .help-close {
            position: absolute;
            top: 20px;
            right: 20px;
            font-size: 24px;
            cursor: pointer;
            color: #666;
        }
        .help-close:hover {
            outline: none;
            box-shadow: none;
            background: transparent;
            color: #666;
        }
        .term-definition {
            margin-bottom: 20px;
            padding-bottom: 10px;
            border-bottom: 1px solid #ddd;
        }
        .term-definition h3 {
            margin-bottom: 5px;
        }
        .team-flag {
            font-size: 24px;
            margin-right: 8px;
            vertical-align: middle;
        }
        /* Remove table wrapper constraints */
        .table-wrapper {
            width: 100%;
            overflow-x: auto;
        }
        /* Make tables responsive but use full width */
        @media screen and (max-width: 1200px) {
            .table-wrapper {
                overflow-x: scroll;
            }
        }
        .no-data {
            color: #999;
            font-style: italic;
            background-color: #f9f9f9;
        }
    </style>
    """
    
    # JavaScript for help button and modal
    help_js = """
    <script>
        // Wait for the DOM to be fully loaded
        document.addEventListener('DOMContentLoaded', function() {
            // Get modal elements
            var helpModal = document.getElementById('helpModal');
            var helpBtn = document.getElementById('helpBtn');
            var closeBtn = document.getElementById('closeHelp');
            
            // Open modal when help button is clicked
            helpBtn.onclick = function() {
                helpModal.style.display = 'flex';
            }
            
            // Close modal when close button is clicked
            closeBtn.onclick = function() {
                helpModal.style.display = 'none';
            }
            
            // Close modal when clicking outside the content
            window.onclick = function(event) {
                if (event.target == helpModal) {
                    helpModal.style.display = 'none';
                }
            }
        });
    </script>
    """
    
    # Helper function to format event type names for display
    def format_event_type(event_type):
        """Format event type for display, with special handling for tactical shifts"""
        if event_type in ("tactical_shifts", "shifts"):
            return "Tactical Shifts"
        else:
            return event_type.replace('_', ' ').title()
            
    # Help button and modal HTML with updated BH correction information
    def create_help_button_and_modal():
        help_html = """
        <!-- Help Button -->
        <button id="helpBtn" class="help-button">?</button>
        
        <!-- Help Modal -->
        <div id="helpModal" class="help-modal">
            <div class="help-content">
                <span id="closeHelp" class="help-close">&times;</span>
                <h2>Football Analysis Guide</h2>
                <p>This guide explains the statistical terms and analysis used in this dashboard.</p>
                
                <div class="term-definition">
                    <h3>TRB (Time to Recover Ball)</h3>
                    <p>The time (in seconds) it takes for a team to regain possession after losing the ball. Lower times indicate better defensive pressing or recovery.</p>
                </div>
                
                <div class="term-definition">
                    <h3>Event Types</h3>
                    <p><strong>All Goals:</strong> Includes goals scored by either team</p>
                    <p><strong>Substitutions:</strong> Player replacements during the match</p>
                    <p><strong>Tactical Shifts:</strong> Changes in team formation or strategy</p>
                    <p><strong>Injuries:</strong> Stoppages due to player injuries</p>
                    <p><strong>Yellow/Red Cards:</strong> Disciplinary actions by the referee</p>
                </div>
                
                <div class="term-definition">
                    <h3>Shapiro-Wilk Test</h3>
                    <p>A statistical test to check if data follows a normal distribution (bell curve).</p>
                    <p><strong>p-value:</strong> If this number is less than 0.05, the data is NOT normally distributed.</p>
                    <p><strong>What it means for coaching:</strong> Non-normal data suggests there are inconsistencies in recovery times - some very fast recoveries and some very slow ones, rather than most being around the average.</p>
                </div>
                
                <div class="term-definition">
                    <h3>Mann-Whitney U Test</h3>
                    <p>A statistical test that compares recovery times before and after events to see if there's a significant difference.</p>
                    <p><strong>p-value:</strong> If this number is less than 0.05, there IS a significant difference in recovery times before vs. after the event.</p>
                    <p><strong>BH Corrected p-value:</strong> When multiple tests are performed, the Benjamini-Hochberg (BH) correction adjusts p-values to control the false discovery rate. This helps avoid finding spurious significant results.</p>
                    <p><strong>Significant After BH:</strong> Shows whether the result remains significant after applying the BH correction. This is a more conservative and reliable indicator of true effects.</p>
                    <p><strong>Cliff's Delta:</strong> Measures the size of the effect:</p>
                    <ul>
                        <li>Negligible: Less than 0.147</li>
                        <li>Small: 0.147 to 0.33</li>
                        <li>Medium: 0.33 to 0.474</li>
                        <li>Large: Greater than 0.474</li>
                    </ul>
                    <p><strong>Direction of Cliff's Delta:</strong></p>
                    <ul>
                        <li><strong>Positive values (+):</strong> Recovery times BEFORE the event are generally higher than after the event. This means players take LESS time to recover the ball following the event (faster recovery).</li>
                        <li><strong>Negative values (-):</strong> Recovery times BEFORE the event are generally lower than after the event. This means players take MORE time to recover the ball following the event (slower recovery).</li>
                    </ul>
                    <p><strong>Confidence Interval (CI):</strong></p>
                    <ul>
                        <li>The CI Lower and CI Upper values show the 95% confidence interval for Cliff's Delta.</li>
                        <li>This means we're 95% confident the true effect size is somewhere between these two values.</li>
                        <li><strong>For Coaches:</strong> If the CI includes zero (e.g., -0.1 to +0.2), the effect might not be reliable. If it excludes zero (e.g., +0.1 to +0.3), you can be more confident in the effect's direction.</li>
                        <li>Wider intervals indicate less certainty about the exact effect size.</li>
                        <li>Narrower intervals indicate more reliable estimates of the true effect.</li>
                    </ul>
                    <p><strong>What it means for coaching:</strong> Focus on results that are significant after BH correction, as these are the most reliable findings. These indicate events that truly affect your team's recovery patterns.</p>
                </div>
                
                <div class="term-definition">
                    <h3>Recovery Impact Analysis</h3>
                    <p>Shows whether teams recover the ball faster or slower after specific events.</p>
                    <p><strong>Avg TRB Before/After:</strong> Average recovery time in seconds before and after events</p>
                    <p><strong>Faster After Event:</strong> "Yes" means recovery was quicker after the event</p>
                    <p><strong>Significant Impact:</strong> "Yes" means the difference is statistically meaningful</p>
                    <p><strong>What it means for coaching:</strong> Focus on events with significant impacts. If recovery is faster after goals, your team might be more motivated. If slower after cards, you may need to adjust tactics after disciplinary actions.</p>
                </div>
                
                <div class="term-definition">
                    <h3>Highlighted Rows</h3>
                    <p>Yellow highlighted rows indicate statistically significant findings that deserve special attention from coaching staff.</p>
                </div>
                
                <div class="term-definition">
                    <h3>How To Use This Analysis</h3>
                    <p><strong>For Training:</strong> Focus on improving recovery times after events where your team slows down</p>
                    <p><strong>For Tactics:</strong> Consider formation changes after events that negatively impact recovery</p>
                    <p><strong>For Psychology:</strong> Address mental aspects if certain events (like conceding goals) consistently slow recovery</p>
                    <p><strong>For Opposition Analysis:</strong> Look for patterns in how opponents' recovery times change after events</p>
                </div>
            </div>
        </div>
        """
        return help_html
        
    # Helper function to get significant findings for a team
    def get_significant_findings(team_name):
        findings = []
        
        # Filter impact results for this team
        if not impact_results.empty:
            team_impacts = impact_results[impact_results['Team'] == team_name]
            significant_impacts = team_impacts[team_impacts['Significant Impact'] == 'Yes']
            
            for _, row in significant_impacts.iterrows():
                event_type = row['Event Type']
                faster = row['Faster After Event'] == 'Yes'
                effect = "faster" if faster else "slower"
                
                findings.append(f"<strong>{event_type}:</strong> Recovery times are significantly {effect} after {event_type.lower()}.")
        
        return findings
    
    # Generate team-level HTML files
    for team_name in teams:
        team_slug = team_name.replace(' ', '_').lower()
        flag_emoji = get_flag_emoji(team_name)
        
        # Debug: Print available dataframes for this team
        print(f"\nGenerating HTML for {team_name}...")
        team_dataframes = [df for df in mannwhitney_results.keys() if team_slug in df and "match_" not in df]
        print(f"Found {len(team_dataframes)} team-level dataframes for {team_name}: {team_dataframes}")
        
        # Get significant findings for this team
        significant_findings = get_significant_findings(team_name)

        # Create team level analysis HTML file
        with open(os.path.join(team_dir, f"{team_slug}_analysis.html"), "w") as f:
            f.write(f"""<!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>{team_name} - Team Analysis</title>
            {css_styles}
            {help_js}
        </head>
        <body style="max-width: 100%; width: 100%;">
            <h1><span class="team-flag">{flag_emoji}</span> {team_name} - Team Analysis</h1>
                        
            <div class="summary-box">
                <h3>Key Findings</h3>
            """)
            
            if significant_findings:
                # Show the significant findings first
                f.write("<ul>\n")
                for finding in significant_findings:
                    f.write(f"    <li>{finding}</li>\n")
                f.write("</ul>\n")
                
                # Add detailed event-specific analysis
                f.write("<h4>Detailed Event Analysis (All Matches)</h4>\n")
                
                # Add team-level impact analysis details
                if not impact_results.empty:
                    team_impacts = impact_results[
                        (impact_results['Team'] == team_name) & 
                        (impact_results['Match ID'] == 'All Matches')
                    ]
                    
                    for _, row in team_impacts.iterrows():
                        event_type = row['Event Type']
                        avg_before = row['Avg TRB Before (s)']
                        avg_after = row['Avg TRB After (s)']
                        faster = row['Faster After Event'] == "Yes"
                        p_value = row['p-value']
                        significant = row['Significant Impact'] == "Yes"
                        
                        # Create event analysis
                        effect_word = "faster" if faster else "slower"
                        significance_phrase = "significantly " if significant else ""
                        effect_size = ""
                        
                        # Convert to internal event type format for consistent display
                        internal_event_type = event_type.lower().replace(' ', '_')
                        event_display = format_event_type(internal_event_type)
                        
                        # Find the effect size from Mann-Whitney results if available
                        for df_name, mw_results in mannwhitney_results.items():
                            # Find team-level results (not match-specific)
                            if "match_" not in df_name and team_slug in df_name:
                                parts = df_name.split('_')
                                if len(parts) > 2:
                                    test_event_type = '_'.join(parts[2:])
                                    if test_event_type == internal_event_type:
                                        if 'effect_size' in mw_results:
                                            effect_size = f" with a {mw_results['effect_size'].lower()} effect size (Cliff's Delta: {mw_results.get('cliffs_delta', 'N/A'):.4f})" if isinstance(mw_results.get('cliffs_delta'), (float, int)) else ""
                                        break
                        
                        # Write event analysis
                        f.write(f"""
                        <div style="margin-left: 20px; margin-bottom: 15px; padding: 10px; background-color: #f9f9f9; border-left: 3px solid {'#4CAF50' if significant else '#ddd'}; border-radius: 4px;">
                            <strong>{event_display}:</strong>
                    """)
                        
                        if significant:
                            f.write(f"""
                            Across all matches, the team has {significance_phrase}{effect_word} recovery times after {event_type.lower()}{effect_size}. 
                            Average recovery time {'decreased' if faster else 'increased'} from {avg_before} seconds before to {avg_after} seconds after {event_type.lower()}.
                            """)
                        else:
                            f.write(f"""
                            No significant overall impact found for {event_type.lower()}, though there was a {'decrease' if faster else 'increase'} 
                            in average recovery time ({avg_before} to {avg_after}).
                            """)
                        
                        f.write("</div>\n")
            else:
                f.write("    <p>No significant findings were identified for this team.</p>\n")
            
            f.write("""
                        </div>

                        <h2>Recovery Times - Team Level</h2>
                    """)
            
            # Generate tables for team-level analysis (rest of the function continues...)
            # NOTE: This continues with the exact same table generation code from the original document
            # 1. Normality Test Results
            f.write("""
                <h3>Normality Test Results</h3>
                <div class="table-wrapper">
                <table>
                    <tr>
                        <th>Event Type</th>
                        <th>Sample Size</th>
                        <th>Shapiro-Wilk Statistic</th>
                        <th>p-value</th>
                        <th>Normal Distribution</th>
                    </tr>
            """)
            
            # Define all event types to ensure complete table
            event_types = ['all_goals', 'substitutions', 'tactical_shifts', 'injuries', 'yellow_cards', 'red_cards']
            
            # Collect all rows first
            rows_with_data = []
            rows_without_data = []
            
            # Process each event type
            for event_type in event_types:
                found = False
                
                # Look for exact dataframe name pattern: df_[team_slug]_[event_type]
                expected_df_name = f"df_{team_slug}_{event_type}"
                
                # Find team-level normality results for this event type
                if expected_df_name in normality_results:
                    results = normality_results[expected_df_name]
                    found = True
                    
                    # Format for display
                    event_display = format_event_type(event_type)
                    
                    # Check for normality
                    is_normal = results.get('shapiro_normal', False)
                    normal_class = "positive" if is_normal else "negative"
                    normal_text = "Yes" if is_normal else "No"
                    
                    # Sample size
                    sample_size = results.get('sample_size', "N/A")
                    
                    # Test statistic
                    shapiro_stat = results.get('shapiro_stat', "N/A")
                    if isinstance(shapiro_stat, (float, int)):
                        shapiro_stat = f"{shapiro_stat:.4f}"
                        
                    # p-value
                    shapiro_p = results.get('shapiro_p', "N/A")
                    if isinstance(shapiro_p, (float, int)):
                        shapiro_p = f"{shapiro_p:.6f}"
                    
                    row_html = f"""
                <tr>
                    <td>{event_display}</td>
                    <td>{sample_size}</td>
                    <td>{shapiro_stat}</td>
                    <td>{shapiro_p}</td>
                    <td class="{normal_class}">{normal_text}</td>
                </tr>
                    """
                    rows_with_data.append(row_html)
                

            
            # Write rows with data first, then rows without data
            for row in rows_with_data:
                f.write(row)
            # for row in rows_without_data:
            #     f.write(row)
            
            f.write("""
                </table>
                </div>
            """)
            
            # 2. Mann-Whitney Test Results with BH Correction
            f.write("""
                <h3>Mann-Whitney U Test Results</h3>
                <div class="table-wrapper">
                <table>
                    <tr>
                        <th>Event Type</th>
                        <th>Before Sample Size</th>
                        <th>After Sample Size</th>
                        <th>Mann-Whitney U</th>
                        <th>p-value</th>
                        <th>Significant Difference</th>
                        <th>BH Corrected p-value</th>
                        <th>Significant After BH</th>
                        <th>Cliff's Delta</th>
                        <th>CI Lower</th>
                        <th>CI Upper</th>
                        <th>Effect Size</th>
                        <th>Explanation</th>
                    </tr>
            """)
            
            # Define all event types to ensure complete table
            event_types = ['all_goals', 'substitutions', 'tactical_shifts', 'injuries', 'yellow_cards', 'red_cards']
            
            # Collect all rows first
            rows_with_data = []
            rows_without_data = []
            
            # Process each event type
            for event_type in event_types:
                found = False
                
                # Look for exact dataframe name pattern: df_[team_slug]_[event_type]
                expected_df_name = f"df_{team_slug}_{event_type}"
                
                # Search through Mann-Whitney results
                if expected_df_name in mannwhitney_results:
                    results = mannwhitney_results[expected_df_name]
                    found = True
                    
                    # Format for display
                    event_display = format_event_type(event_type)
                    
                    # Look up BH correction results
                    lookup_key = f"{team_slug}_{event_type}"
                    bh_data = team_bh_lookup.get(lookup_key, {})
                    bh_corrected_p = bh_data.get('bh_corrected_p', 'N/A')
                    significant_after_bh = bh_data.get('significant_after_bh', False)
                    
                    # Check for significance
                    is_significant = results.get('significant', False)
                    sig_class = "positive" if is_significant else "negative"
                    sig_text = "Yes" if is_significant else "No"
                    
                    # BH significance class
                    bh_sig_class = "positive" if significant_after_bh else "negative"
                    bh_sig_text = "Yes" if significant_after_bh else "No"
                    
                    # Highlight row based on BH significance
                    row_class = 'class="highlight"' if significant_after_bh else ''
                    
                    # Sample sizes
                    before_size = results.get('before_sample_size', "N/A")
                    after_size = results.get('after_sample_size', "N/A")
                    
                    # Test statistic
                    mw_stat = results.get('mw_stat', "N/A")
                    if isinstance(mw_stat, (float, int)):
                        mw_stat = f"{mw_stat:.4f}"
                        
                    # p-value
                    mw_p = results.get('mw_p', "N/A")
                    if isinstance(mw_p, (float, int)):
                        mw_p = f"{mw_p:.6f}"
                    
                    # Format BH corrected p-value
                    if isinstance(bh_corrected_p, (float, int)):
                        bh_corrected_p_str = f"{bh_corrected_p:.6f}"
                    else:
                        bh_corrected_p_str = str(bh_corrected_p)
                        
                    # Cliff's Delta
                    cliff_d = results.get('cliffs_delta', "N/A")
                    if isinstance(cliff_d, (float, int)):
                        cliff_d = f"{cliff_d:.4f}"
                    
                    # Cliff's Delta CI Lower
                    ci_lower = results.get('ci_lower', "N/A")
                    if isinstance(ci_lower, (float, int)):
                        ci_lower = f"{ci_lower:.4f}"

                    # Cliff's Delta CI Upper
                    ci_upper = results.get('ci_upper', "N/A")
                    if isinstance(ci_upper, (float, int)):
                        ci_upper = f"{ci_upper:.4f}"
                        
                    # Effect size
                    effect_size = results.get('effect_size', "N/A")
                    
                    row_html = f"""
                        <tr {row_class}>
                            <td>{event_display}</td>
                            <td>{before_size}</td>
                            <td>{after_size}</td>
                            <td>{mw_stat}</td>
                            <td>{mw_p}</td>
                            <td class="{sig_class}">{sig_text}</td>
                            <td>{bh_corrected_p_str}</td>
                            <td class="{bh_sig_class}">{bh_sig_text}</td>
                            <td>{cliff_d}</td>
                            <td>{ci_lower}</td>
                            <td>{ci_upper}</td>
                            <td>{effect_size}</td>
                            <td>{get_cliffs_delta_explanation(results.get('cliffs_delta', 'N/A'), 
                                    results.get('ci_lower', 'N/A'), 
                                    results.get('ci_upper', 'N/A'))}</td>
                        </tr>
                    """
                    rows_with_data.append(row_html)
                

            
            # Write rows with data first, then rows without data
            for row in rows_with_data:
                f.write(row)
            # for row in rows_without_data:
            #     f.write(row)
            
            f.write("""
                </table>
                </div>
            """)
            
            # 3. Recovery Impact Analysis
            f.write("""
                <h3>Recovery Impact Analysis</h3>
                <div class="table-wrapper">
                <table>
                    <tr>
                        <th>Event Type</th>
                        <th>Avg TRB Before (s)</th>
                        <th>Avg TRB After (s)</th>
                        <th>Faster After Event</th>
                    </tr>
            """)
            
            # Define all event types to ensure complete table
            event_types_display = {
                'all_goals': 'All Goals',
                'substitutions': 'Substitutions', 
                'tactical_shifts': 'Tactical Shifts',
                'injuries': 'Injuries',
                'yellow_cards': 'Yellow Cards',
                'red_cards': 'Red Cards'
            }
            
            # Collect all rows first
            rows_with_data = []
            rows_without_data = []
            
            # Process each event type
            for event_type, event_display in event_types_display.items():
                found = False
                
                # Find team-level impact results
                if not impact_results.empty:
                    team_impacts = impact_results[
                        (impact_results['Team'] == team_name) & 
                        (impact_results['Match ID'] == 'All Matches')
                    ]
                    
                    for _, row in team_impacts.iterrows():
                        if row['Event Type'] == event_display:
                            found = True
                            
                            avg_before = row['Avg TRB Before (s)']
                            avg_after = row['Avg TRB After (s)']
                            faster = row['Faster After Event']
                            p_value = row['p-value']
                            significant = row['Significant Impact']
                            
                            # CSS classes
                            faster_class = "positive" if faster == "Yes" else "negative"
                            sig_class = "positive" if significant == "Yes" else "negative"
                            
                            # Highlight row if significant
                            row_class = 'class="highlight"' if significant == "Yes" else ''
                            
                            row_html = f"""
                            <tr {row_class}>
                                <td>{event_display}</td>
                                <td>{avg_before}</td>
                                <td>{avg_after}</td>
                                <td class="{faster_class}">{faster}</td>
                            </tr>
                            """
                            rows_with_data.append(row_html)
                            break
                

            
            # Write rows with data first, then rows without data
            for row in rows_with_data:
                f.write(row)
            # for row in rows_without_data:
            #     f.write(row)
            
            f.write("""
                </table>
                </div>
            """)
            
            # Generate match list for this team
            f.write("""
                <h2>Match-specific Analysis</h2>
                <div class="event-card">
                    <p>Select a match to view detailed analysis:</p>
                    <ul>
            """)
            
            # Get matches for this team
            team_matches = {}
            for team_name_data, matches_dict in team_dicts:
                if team_name_data == team_name:
                    for match_id, match_details in matches_dict.items():
                        team_matches[match_id] = match_details
            
            for match_id, match_details in team_matches.items():
                opposing_team = match_details['team']
                opposing_flag = get_flag_emoji(opposing_team)
                # Update link to point to match-specific directory
                f.write(f'        <li><a href="../match_{match_id}/{team_slug}_analysis.html"><span class="team-flag">{flag_emoji}</span> {team_name} vs <span class="team-flag">{opposing_flag}</span> {opposing_team} (Match ID: {match_id})</a></li>\n')
            
            f.write("""
                    </ul>
                </div>
                
                <div class="footer">
                </div>
                
                """ + create_help_button_and_modal() + """
            </body>
            </html>
            """)
    
    # Helper function to get significant findings for a team in a specific match
    def get_match_significant_findings(team_name, match_id):
        findings = []
        
        # Filter impact results for this team and match
        if not impact_results.empty:
            match_impacts = impact_results[
                (impact_results['Team'] == team_name) & 
                (impact_results['Match ID'] == match_id)
            ]
            significant_impacts = match_impacts[match_impacts['Significant Impact'] == 'Yes']
            
            for _, row in significant_impacts.iterrows():
                event_type = row['Event Type']
                faster = row['Faster After Event'] == 'Yes'
                effect = "faster" if faster else "slower"
                
                findings.append(f"<strong>{event_type}:</strong> Recovery times are significantly {effect} after {event_type.lower()} in this match.")
        
        return findings
    
    # Generate match-specific HTML files
    for team_name, matches_dict in team_dicts:
        team_slug = team_name.replace(' ', '_').lower()
        flag_emoji = get_flag_emoji(team_name)
        
        for match_id, match_details in matches_dict.items():
            opposing_team = match_details['team']
            opposing_flag = get_flag_emoji(opposing_team)
            
            # Get significant findings for this match
            match_findings = get_match_significant_findings(team_name, match_id)
            match_dir = os.path.join(output_dir, f"match_{match_id}")
            
            # Create match-specific HTML file in the match directory
            with open(os.path.join(match_dir, f"{team_slug}_analysis.html"), "w") as f:
                f.write(f"""<!DOCTYPE html>
            <html lang="en">
            <head>
                <meta charset="UTF-8">
                <meta name="viewport" content="width=device-width, initial-scale=1.0">
                <title>{team_name} vs. {opposing_team} - Match Analysis</title>
                {css_styles}
                {help_js}
            </head>
            <body style="max-width: 100%; width: 100%;">
                <h1><span class="team-flag">{flag_emoji}</span> {team_name} vs <span class="team-flag">{opposing_flag}</span> {opposing_team} - Match Analysis</h1>
                <p>Match ID: {match_id}</p>
                
                <div class="summary-box">
                    <h3>Key Findings</h3>
            """)
                
                if match_findings:
                    # Show the significant findings first
                    f.write("<ul>\n")
                    for finding in match_findings:
                        f.write(f"    <li>{finding}</li>\n")
                    f.write("</ul>\n")
                    
                    # Add detailed event-specific analysis
                    f.write("<h4>Detailed Event Analysis</h4>\n")
                    
                    # Find match-specific impact results for event details
                    if not impact_results.empty:
                        match_impacts = impact_results[
                            (impact_results['Team'] == team_name) & 
                            (impact_results['Match ID'] == match_id)
                        ]
                        
                        for _, row in match_impacts.iterrows():
                            event_type = row['Event Type']
                            avg_before = row['Avg TRB Before (s)']
                            avg_after = row['Avg TRB After (s)']
                            faster = row['Faster After Event'] == "Yes"
                            p_value = float(row['p-value']) if isinstance(row['p-value'], str) else row['p-value']
                            significant = row['Significant Impact'] == "Yes"
                            
                            # Create event analysis
                            effect_word = "faster" if faster else "slower"
                            significance_phrase = "significantly " if significant else ""
                            effect_size = ""
                            
                            # Use the formatted event type display
                            event_display = format_event_type(event_type.lower().replace(' ', '_'))
                            
                            # extract the full event type from the dataframe name
                            for df_name, mw_results in mannwhitney_results.items():
                                if f"match_{match_id}" in df_name and team_slug in df_name:
                                    # Extract the full event type from the dataframe name
                                    prefix = f"df_{team_slug}_match_{match_id}_"
                                    if df_name.startswith(prefix):
                                        extracted_event_type = df_name[len(prefix):]
                                        # Compare with the current event type
                                        if extracted_event_type == event_type.lower().replace(' ', '_'):
                                            if 'effect_size' in mw_results:
                                                effect_size = f" with a {mw_results['effect_size'].lower()} effect size (Cliff's Delta: {mw_results.get('cliffs_delta', 'N/A'):.4f})" if isinstance(mw_results.get('cliffs_delta'), (float, int)) else ""
                                            break
                            
                            # Write event analysis
                            f.write(f"""
                            <div style="margin-left: 20px; margin-bottom: 15px; padding: 10px; background-color: #f9f9f9; border-left: 3px solid {'#4CAF50' if significant else '#ddd'}; border-radius: 4px;">
                                <strong>{event_display}:</strong>
                        """)
                            
                            if significant:
                                f.write(f"""
                                The team has {significance_phrase}{effect_word} recovery times after {event_type.lower()} in this match{effect_size}. 
                                Average recovery time {'decreased' if faster else 'increased'} from {avg_before} seconds before to {avg_after} seconds after {event_type.lower()}.
                                """)
                            else:
                                f.write(f"""
                                No significant impact found for {event_type.lower()}, though there was a {'decrease' if faster else 'increase'} 
                                in average recovery time ({avg_before} to {avg_after}).
                                """)
                            
                            f.write("</div>\n")
                else:
                    f.write("    <p>No significant findings were identified for this match.</p>\n")
                
                f.write("""
                    </div>

                    <h2>Recovery Times - Match Level</h2>
                """)
                
                # Generate tables for match-level analysis
                
                # 1. Normality Test Results
                f.write("""
                    <h3>Normality Test Results</h3>
                    <div class="table-wrapper">
                    <table>
                        <tr>
                            <th>Event Type</th>
                            <th>Sample Size</th>
                            <th>Shapiro-Wilk Statistic</th>
                            <th>p-value</th>
                            <th>Normal Distribution</th>
                        </tr>
                """)
                
                # Define all event types to ensure complete table
                event_types = ['all_goals', 'substitutions', 'tactical_shifts', 'injuries', 'yellow_cards', 'red_cards']
                
                # Process each event type for match-specific normality results
                for event_type in event_types:
                    found = False
                    
                    # Find match-specific normality results
                    for df_name, results in normality_results.items():
                        # Only include match-specific results for this match
                        if f"match_{match_id}" in df_name and team_slug in df_name:
                            # Get event type
                            parts = df_name.split('_')
                            if len(parts) > 4:  
                                match_idx = df_name.find(f"_match_{match_id}_")
                                if match_idx != -1:
                                    test_event_type = df_name[match_idx + len(f"_match_{match_id}_"):]
                                else:
                                    test_event_type = parts[-1]
                                
                                if test_event_type == event_type:
                                    found = True
                                    
                                    # Format for display
                                    event_display = format_event_type(event_type)

                                    # Check for normality
                                    is_normal = results.get('shapiro_normal', False)
                                    normal_class = "positive" if is_normal else "negative"
                                    normal_text = "Yes" if is_normal else "No"
                                    
                                    # Sample size
                                    sample_size = results.get('sample_size', "N/A")
                                    
                                    # Test statistic
                                    shapiro_stat = results.get('shapiro_stat', "N/A")
                                    if isinstance(shapiro_stat, (float, int)):
                                        shapiro_stat = f"{shapiro_stat:.4f}"
                                        
                                    # p-value
                                    shapiro_p = results.get('shapiro_p', "N/A")
                                    if isinstance(shapiro_p, (float, int)):
                                        shapiro_p = f"{shapiro_p:.6f}"
                                    
                                    f.write(f"""
                                <tr>
                                    <td>{event_display}</td>
                                    <td>{sample_size}</td>
                                    <td>{shapiro_stat}</td>
                                    <td>{shapiro_p}</td>
                                    <td class="{normal_class}">{normal_text}</td>
                                </tr>
                                    """)
                                    break
                    
                    # # If no results found for this event type, add empty row
                    # if not found:
                    #     event_display = format_event_type(event_type)
                    #     f.write(f"""
                    #         <tr>
                    #             <td>{event_display}</td>
                    #             <td colspan="4" style="text-align: center; color: #999;">No data available</td>
                    #         </tr>
                    #     """)
                
                f.write("""
                    </table>
                    </div>
                """)
                
                # 2. Mann-Whitney Test Results with BH Correction
                f.write("""
                    <h3>Mann-Whitney U Test Results</h3>
                    <div class="table-wrapper">
                    <table>
                        <tr>
                            <th>Event Type</th>
                            <th>Before Sample Size</th>
                            <th>After Sample Size</th>
                            <th>Mann-Whitney U</th>
                            <th>p-value</th>
                            <th>Significant Difference</th>
                            <th>BH Corrected p-value</th>
                            <th>Significant After BH</th>
                            <th>Cliff's Delta</th>
                            <th>CI Lower</th>
                            <th>CI Upper</th>
                            <th>Effect Size</th>
                            <th>Explanation</th>
                        </tr>
                """)
                
                # Process each event type for match-specific Mann-Whitney results
                for event_type in event_types:
                    found = False
                    
                    # Find match-specific Mann-Whitney results
                    for df_name, results in mannwhitney_results.items():
                        # Only include match-specific results for this match
                        if f"match_{match_id}" in df_name and team_slug in df_name:
                            # Get event type
                            parts = df_name.split('_')
                            if len(parts) > 4:  
                                match_idx = df_name.find(f"_match_{match_id}_")
                                if match_idx != -1:
                                    test_event_type = df_name[match_idx + len(f"_match_{match_id}_"):]
                                else:
                                    test_event_type = parts[-1]
                                
                                if test_event_type == event_type:
                                    found = True
                                    
                                    # Format for display
                                    event_display = format_event_type(event_type)
                                    
                                    # Look up BH correction results
                                    lookup_key = f"{team_slug}_match_{match_id}_{event_type}"
                                    bh_data = match_bh_lookup.get(lookup_key, {})
                                    bh_corrected_p = bh_data.get('bh_corrected_p', 'N/A')
                                    significant_after_bh = bh_data.get('significant_after_bh', False)
                                    
                                    # Check for significance
                                    is_significant = results.get('significant', False)
                                    sig_class = "positive" if is_significant else "negative"
                                    sig_text = "Yes" if is_significant else "No"
                                    
                                    # BH significance class
                                    bh_sig_class = "positive" if significant_after_bh else "negative"
                                    bh_sig_text = "Yes" if significant_after_bh else "No"
                                    
                                    # Highlight row based on BH significance
                                    row_class = 'class="highlight"' if significant_after_bh else ''
                                    
                                    # Sample sizes
                                    before_size = results.get('before_sample_size', "N/A")
                                    after_size = results.get('after_sample_size', "N/A")
                                    
                                    # Test statistic
                                    mw_stat = results.get('mw_stat', "N/A")
                                    if isinstance(mw_stat, (float, int)):
                                        mw_stat = f"{mw_stat:.4f}"
                                        
                                    # p-value
                                    mw_p = results.get('mw_p', "N/A")
                                    if isinstance(mw_p, (float, int)):
                                        mw_p = f"{mw_p:.6f}"
                                    
                                    # Format BH corrected p-value
                                    if isinstance(bh_corrected_p, (float, int)):
                                        bh_corrected_p_str = f"{bh_corrected_p:.6f}"
                                    else:
                                        bh_corrected_p_str = str(bh_corrected_p)
                                        
                                    # Cliff's Delta
                                    cliff_d = results.get('cliffs_delta', "N/A")
                                    if isinstance(cliff_d, (float, int)):
                                        cliff_d = f"{cliff_d:.4f}"
                                        
                                    # Cliff's Delta CI Lower
                                    ci_lower = results.get('ci_lower', "N/A")
                                    if isinstance(ci_lower, (float, int)):
                                        ci_lower = f"{ci_lower:.4f}"

                                    # Cliff's Delta CI Upper
                                    ci_upper = results.get('ci_upper', "N/A")
                                    if isinstance(ci_upper, (float, int)):
                                        ci_upper = f"{ci_upper:.4f}"
                                    
                                    # Effect size
                                    effect_size = results.get('effect_size', "N/A")
                                    
                                    f.write(f"""
                                        <tr {row_class}>
                                            <td>{event_display}</td>
                                            <td>{before_size}</td>
                                            <td>{after_size}</td>
                                            <td>{mw_stat}</td>
                                            <td>{mw_p}</td>
                                            <td class="{sig_class}">{sig_text}</td>
                                            <td>{bh_corrected_p_str}</td>
                                            <td class="{bh_sig_class}">{bh_sig_text}</td>
                                            <td>{cliff_d}</td>
                                            <td>{ci_lower}</td>
                                            <td>{ci_upper}</td>
                                            <td>{effect_size}</td>
                                            <td>{get_cliffs_delta_explanation(results.get('cliffs_delta', 'N/A'), 
                                                results.get('ci_lower', 'N/A'), 
                                                results.get('ci_upper', 'N/A'))}</td>
                                        </tr>
                                    """)
                                    break
                    
                    # # If no results found for this event type, add empty row
                    # if not found:
                    #     event_display = format_event_type(event_type)
                    #     f.write(f"""
                    #         <tr>
                    #             <td>{event_display}</td>
                    #             <td colspan="12" style="text-align: center; color: #999;">No data available</td>
                    #         </tr>
                    #     """)
                
                f.write("""
                    </table>
                    </div>
                """)
                
                # 3. Recovery Impact Analysis
                f.write("""
                    <h3>Recovery Impact Analysis</h3>
                    <div class="table-wrapper">
                    <table>
                        <tr>
                            <th>Event Type</th>
                            <th>Avg TRB Before (s)</th>
                            <th>Avg TRB After (s)</th>
                            <th>Faster After Event</th>
                        </tr>
                """)
                
                # Define all event types to ensure complete table
                event_types_display = {
                    'all_goals': 'All Goals',
                    'substitutions': 'Substitutions', 
                    'tactical_shifts': 'Tactical Shifts',
                    'injuries': 'Injuries',
                    'yellow_cards': 'Yellow Cards',
                    'red_cards': 'Red Cards'
                }
                
                # Process each event type for match-specific impact results
                for event_type, event_display in event_types_display.items():
                    found = False
                    
                    # Find match-specific impact results
                    if not impact_results.empty:
                        match_impacts = impact_results[
                            (impact_results['Team'] == team_name) & 
                            (impact_results['Match ID'] == match_id)
                        ]
                        
                        for _, row in match_impacts.iterrows():
                            if row['Event Type'] == event_display:
                                found = True
                                
                                avg_before = row['Avg TRB Before (s)']
                                avg_after = row['Avg TRB After (s)']
                                faster = row['Faster After Event']
                                p_value = row['p-value']
                                significant = row['Significant Impact']
                                
                                # CSS classes
                                faster_class = "positive" if faster == "Yes" else "negative"
                                sig_class = "positive" if significant == "Yes" else "negative"
                                
                                # Highlight row if significant
                                row_class = 'class="highlight"' if significant == "Yes" else ''
                                
                                f.write(f"""
                                <tr {row_class}>
                                    <td>{event_display}</td>
                                    <td>{avg_before}</td>
                                    <td>{avg_after}</td>
                                    <td class="{faster_class}">{faster}</td>
                                </tr>
                                """)
                                break
                    
                    # # If no data found, add empty row
                    # if not found:
                    #     f.write(f"""
                    #         <tr>
                    #             <td>{event_display}</td>
                    #             <td colspan="5
                    #             " style="text-align: center; color: #999;">No data available</td>
                    #         </tr>
                    #     """)
                
                f.write("""
                    </table>
                    </div>
                """)
                
                f.write("""
                    <div class="footer">
                    </div>
                    
                    """ + create_help_button_and_modal() + """
                </body>
                </html>
                """)
    
    print(f"HTML files generated in {output_dir}")
    return "HTML generation complete"


def get_cliffs_delta_explanation(cliff_d, ci_lower, ci_upper):
    """
    Provide explanation for Cliff's Delta results
    """
    try:
        cliff_d = float(cliff_d)
        ci_lower = float(ci_lower)
        ci_upper = float(ci_upper)
        
        if ci_lower > 0:
            return "Significantly faster recovery after events (CI entirely above 0)"
        elif ci_upper < 0:
            return "Significantly slower recovery after events (CI entirely below 0)"
        else:
            return "No significant difference (CI includes 0)"
    except (ValueError, TypeError):
        return "Unable to determine (invalid data)"


def run_football_analysis(team_dicts, output_dir="visuals"):
    """
    Main function to run the football analysis and generate HTML files
    
    Args:
        team_dicts: List of tuples containing team names and match dictionaries
        output_dir: Directory to save HTML files
    """
    # First, run the analyze_event_recovery_times function
    print("Running analyze_event_recovery_times...")
    all_dataframes, normality_results, mannwhitney_results, match_level_tests, team_level_tests = analyze_event_recovery_times(*team_dicts)
    
    # Extract impact results from the test results
    print("Extracting impact results...")
    
    impact_data = []
    
    # Process match-level tests
    for test in match_level_tests:
        if test.get('before_n', 0) >= 5 and test.get('after_n', 0) >= 5:
            impact_row = {
                'Team': test['team'],
                'Match ID': test['match_id'],
                'Event Type': test['event_type'].replace('_', ' ').title(),
                'Avg TRB Before (s)': f"{test['avg_before']:.2f}" if not pd.isna(test['avg_before']) else "N/A",
                'Avg TRB After (s)': f"{test['avg_after']:.2f}" if not pd.isna(test['avg_after']) else "N/A",
                'Faster After Event': "Yes" if test.get('faster_after', False) else "No",
                'p-value': f"{test['p_value']:.6f}" if not pd.isna(test['p_value']) else "N/A",
                'Significant Impact': "Yes" if test.get('significant_after_bh', test.get('significant_uncorrected', False)) else "No"
            }
            impact_data.append(impact_row)
    
    # Process team-level tests
    for test in team_level_tests:
        if test.get('before_n', 0) >= 5 and test.get('after_n', 0) >= 5:
            impact_row = {
                'Team': test['team'],
                'Match ID': 'All Matches',
                'Event Type': test['event_type'].replace('_', ' ').title(),
                'Avg TRB Before (s)': f"{test['avg_before']:.2f}" if not pd.isna(test['avg_before']) else "N/A",
                'Avg TRB After (s)': f"{test['avg_after']:.2f}" if not pd.isna(test['avg_after']) else "N/A",
                'Faster After Event': "Yes" if test.get('faster_after', False) else "No",
                'p-value': f"{test['p_value']:.6f}" if not pd.isna(test['p_value']) else "N/A",
                'Significant Impact': "Yes" if test.get('significant_after_bh', test.get('significant_uncorrected', False)) else "No"
            }
            impact_data.append(impact_row)
    
    # Create DataFrame
    impact_results = pd.DataFrame(impact_data)
    
    # Sort by team, then match ID, then event type
    if not impact_results.empty:
        impact_results = impact_results.sort_values(['Team', 'Match ID', 'Event Type'])
    
    # Finally, generate the HTML files with the new parameters
    print("Generating HTML files...")
    result = generate_html_files(team_dicts, all_dataframes, normality_results, mannwhitney_results, 
                                impact_results, match_level_tests, team_level_tests, output_dir)
    
    return result


def integrated_football_analysis(team_dicts, output_dir="visuals"):
    """
    Run both the recovery time normality analysis and the football analytics
    functions to generate a complete dashboard with linked HTML files.
    
    Args:
        team_dicts: List of tuples, each containing (team_name, team_color, matches_dict)
        output_dir: Directory to save the output plots and reports
    
    Returns:
        Result of both analyses
    """
    # First, run the recovery time normality test
    print("Running recovery time normality analysis...")
    normality_results = test_recovery_time_normality(*team_dicts, output_dir=output_dir)
    
    # Convert team_dicts format for the run_football_analysis function
    # The first function takes (team_name, team_color, matches_dict)
    # The second function takes (team_name, matches_dict)
    football_team_dicts = [(team_name, matches_dict) for team_name, team_color, matches_dict in team_dicts]
    
    # Then, run the football analytics function
    print("Running football analytics...")
    football_results = run_football_analysis(football_team_dicts, output_dir=output_dir)
    
    print("All analyses complete!")
    return {
        "normality_results": normality_results,
        "football_results": football_results
    }

# Run the analysis
print("Starting integrated football analysis...")
integrated_football_analysis([
    ("Argentina", "#89CFF0", arg_matches), 
    ("France", "blue", fr_matches)
], output_dir="visuals")

Starting integrated football analysis...
Running recovery time normality analysis...
Analyzing only matches involving these teams: France, Argentina

Processing normality tests for Argentina...
Processing Match ID: 3857264 (Poland)...
Created KDE plot for Match 3857264
Created KDE plot for Argentina in Match 3857264
Skipping Poland (not in target teams list)
Processing Match ID: 3857289 (Mexico)...
Created KDE plot for Match 3857289
Created KDE plot for Argentina in Match 3857289
Skipping Mexico (not in target teams list)
Processing Match ID: 3857300 (Saudi Arabia)...
Created KDE plot for Match 3857300
Skipping Saudi Arabia (not in target teams list)
Created KDE plot for Argentina in Match 3857300
Processing Match ID: 3869151 (Australia)...
Created KDE plot for Match 3869151
Skipping Australia (not in target teams list)
Created KDE plot for Argentina in Match 3869151
Processing Match ID: 3869321 (Netherlands)...
Created KDE plot for Match 3869321
Skipping Netherlands (not in target tea

{'normality_results': {3857264: {'primary_team': 'Poland',
   'team_name': 'Argentina',
   'periods': {1: {'Argentina': {'has_data': True,
      'shapiro_stat': 0.9303066464583367,
      'shapiro_pvalue': 0.2202281031694888,
      'is_normal': True,
      'sample_size': 17,
      'mean': 71.6625294117647,
      'median': 52.917,
      'std_dev': 51.93217943922591}},
    2: {'Argentina': {'has_data': True,
      'shapiro_stat': 0.8060034340618834,
      'shapiro_pvalue': 0.008039955982563683,
      'is_normal': False,
      'sample_size': 13,
      'mean': 88.22461538461538,
      'median': 64.396,
      'std_dev': 87.40546956001046}},
    'All': {'Argentina': {'has_data': True,
      'shapiro_stat': 0.8375029681198324,
      'shapiro_pvalue': 0.0003431815480460831,
      'is_normal': False,
      'sample_size': 30,
      'mean': 78.83943333333335,
      'median': 58.6565,
      'std_dev': 70.04407597015056}}}},
  3857289: {'primary_team': 'Mexico',
   'team_name': 'Argentina',
   'peri